<a href="https://colab.research.google.com/github/Datarian723/AutoHPOJupyter/blob/main/AutoHpo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Prep

In [1]:
"""
(1)TODO: Decouple ConfigSpace <-> ObjectiveExecutor. CHANGE the previous ObjectiveExecutor to ObjectiveExecutorExecutor.
         NOTE: ConfigSpace, ObjectiveExecutor are for user-interaction
               ObjectiveExecutorExecutor is NOT supposed to interact with the user(less granular checking)
(2)TODO: Modify 'Auto' s.t it accepts pre-determined trials & Returns ALL the .get_loadable_action(s) from each ModelAction(var_type -> dict(all the loaded models))
         1-Do internal check in Auto to DROP all the keys that don't appear in init_config.var_types | init_config.sig_var_types
          (Additionally, make sure that each and every one of them falls into the 'valid' value range for each configuration space)
         2-Do internal check & assembly in HpoTrainer according to pre-determined trials.
           (If there's any missing signature parameter in the process of assembly, WARN the user and use a randomly sampled value from .distribution(indices=False))
         3-Multiple models being returned can further be used for things like K-fold voting/pred for producing the best output.
"""

"\n(1)TODO: Decouple ConfigSpace <-> ObjectiveExecutor. CHANGE the previous ObjectiveExecutor to ObjectiveExecutorExecutor.\n         NOTE: ConfigSpace, ObjectiveExecutor are for user-interaction\n               ObjectiveExecutorExecutor is NOT supposed to interact with the user(less granular checking)\n(2)TODO: Modify 'Auto' s.t it accepts pre-determined trials & Returns ALL the .get_loadable_action(s) from each ModelAction(var_type -> dict(all the loaded models))\n         1-Do internal check in Auto to DROP all the keys that don't appear in init_config.var_types | init_config.sig_var_types\n          (Additionally, make sure that each and every one of them falls into the 'valid' value range for each configuration space)\n         2-Do internal check & assembly in HpoTrainer according to pre-determined trials.\n           (If there's any missing signature parameter in the process of assembly, WARN the user and use a randomly sampled value from .distribution(indices=False))\n         

In [2]:
!pip install fastai --upgrade --quiet
!pip install optuna --upgrade --quiet
!pip install optuna-integration --upgrade --quiet
!pip install scipy --upgrade --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 54.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires scipy<1.14.0,>=1.7.0, but you have scipy 1.14.1 which is incompatible.


In [3]:
# ---------------------------
#Special Imports
# ---------------------------
from __future__ import annotations  #Enable forward references for type hints

# ---------------------------
#Standard Libraries
# ---------------------------
import os  # Operating system utilities
import gc  # Garbage collection for memory management
import logging  # Logging utilities
import sys  # System-specific parameters and functions
import unittest  # For unit-testing Python code
import copy  # For deep and shallow copying of objects
import math  # Math operations
import random  # Random number generation
import gzip  # For compressing and decompressing data
import ast #For building an Abstract Syntax Tree; used in class 'ObjectiveExecutor'.
import warnings  # For handling warnings
import textwrap #For dedenting texts.
import logging #For log information. Used in Objective.
import contextlib #For context management. Used to reduce message clutter.
from functools import partial  # Partial function application
from collections import defaultdict  # Dictionary subclass with default values
from itertools import chain # Used for 'flattening' Collection of Collections.
from sortedcontainers import SortedSet  # Sorted set implementation
from unittest.mock import Mock, patch  # For mocking and patching in unit tests
from inspect import signature  # Inspecting callable signatures
from typing import Callable, Union  # Type annotations for type hints
from ast import Is  # Abstract syntax tree utilities
from abc import ABC, abstractmethod  # Abstract Base Classes for inheritance
from contextlib import contextmanager # contextmanager.

# ---------------------------
# Scientific Libraries
# ---------------------------
import numpy as np  # Array and matrix operations
import pandas as pd  # Data manipulation and analysis
import matplotlib.pyplot as plt  # Plotting and visualization

# ---------------------------
# PyTorch and Related Modules
# ---------------------------
import torch  # Core PyTorch library for tensor operations
import torch.nn as nn  # Neural network modules
import torch.nn.functional as F  # Functional API for neural networks
import torchvision.models as models  # Pre-trained models from torchvision
from torch.multiprocessing import Pool, set_start_method  # Multiprocessing utilities for PyTorch

# ---------------------------
# FastAI Libraries
# ---------------------------
from fastai.vision.all import *  # All-in-one import for vision-specific modules
from fastai.metrics import *  # Evaluation metrics for model performance
from fastai.callback.hook import *  # Hooks for capturing intermediate model states
from fastai.callback.tracker import *  # Training callback trackers (e.g., early stopping)
from fastai.learner import Learner  # Core Learner object for model training

# ---------------------------
# Optuna for Hyperparameter Optimization
# ---------------------------
import optuna  # Core Optuna library for optimization
from optuna.integration import FastAIPruningCallback  # Pruning callback for FastAI integration
from optuna.trial import Trial, FrozenTrial  # Trial classes for defining and managing trials

# ---------------------------
# Google Colab Utilities
# ---------------------------
from google.colab import files  # File utilities for Colab
from google.colab import drive  # Mounting Google Drive in Colab

#Class: Var

In [4]:
class Var:
    def __init__(self, var_type: str, source: dict, belongs_to: str=None):
        sample_method = source['sample']
        distribution = source['params']['choices'] if sample_method=='categorical' else source['params']
        #ERROR CHECK:
        self._validate_inputs(distribution, sample_method)

        #Member initializations:
        self.var_type = var_type
        self.sample_method = sample_method
        self._belongs_to = belongs_to
        #frozen_dist is used for value conversion, in case value NOT currently in 'self.get_dist()' is requested for conversion.
        self._frozen_dist = distribution.copy()
        self._dist = list(range(len(self._frozen_dist))) if sample_method=='categorical' else distribution.copy()

    def _validate_inputs(self, distribution, sample_method: str):
        if not isinstance(distribution, (list, dict)):
            raise ValueError(f"Input 'distribution' MUST be of type in (list,dict) but found: {type(distribution)}")
        if sample_method not in ['categorical', 'float', 'int']:
            raise ValueError(f"sample method MUST be one of ['categorical', 'float', 'int'] but found: {sample_method}")
        if sample_method!='categorical' and not isinstance(distribution, dict) or sample_method=='categorical' and not isinstance(distribution, list):
            raise ValueError(f"sample_method: {sample_method} is not compatible with distribution: {distribution}. Categorical sampling requires a list, rest requires a dictionary")
        cat_valid_params = sample_method!='categorical' or len(distribution)
        non_cat_valid_params = sample_method=='categorical' or (distribution.get('low', 0) <= distribution.get('high', -1))
        if not cat_valid_params or not non_cat_valid_params:
            msg = f"Categorical distribution needs at least one value in it: {distribution}" if sample_method=='categorical' else f"distribution needs (low,high) as keys and distribution[low]<=distribution[high] but found: {distribution}"
            raise ValueError(msg)

    def __hash__(self):
        return hash((self.var_type, self._belongs_to))

    def __eq__(self, other: Var):
        return all([
            isinstance(other, Var),
            self.var_type == other.var_type,
            self.sample_method == other.sample_method,
            self._belongs_to == other._belongs_to,
            self._frozen_dist == other._frozen_dist,
            self._dist == other._dist
        ])

    def __str__(self):
        if self.sample_method == 'categorical':
            config_space = f"'choices' = {[self._frozen_dist[idx] for idx in self._dist]}"
        else:
            config_space = (
                f"'low': {self._dist['low']}, "
                f"'high': {self._dist['high']}, "
                f"'log': {self._dist.get('log', False)}, "
                f"'step': {self._dist.get('step', 'unspecified')}"
            )
        output = f"Var of type: {self.var_type}, signature of: {self._belongs_to}\nConfiguration Space:\n{config_space}\n"
        return output

    def is_fixed(self):
        return len(self._dist)==1 if self.sample_method=='categorical' else self._dist['low']==self._dist['high']

    def is_sig_var(self):
        return self._belongs_to!='None'

    def belongs_to(self):
        return self._belongs_to

    def contain_idx(self, idx, current: bool = False):
        if self.sample_method!='categorical' and type(idx)!=eval(self.sample_method) or self.sample_method=='categorical' and type(idx)!=int:
            return False
        if self.sample_method!='categorical':
            dist = self._dist if current else self._frozen_dist
            return dist['low'] <= idx <= dist['high']
        return idx in self._dist if current else 0 <= idx < len(self._frozen_dist)

    def contain_val(self, val, current: bool = False):
        if self.sample_method != 'categorical':
            return self.contain_idx(idx=val, current=current)
        return val in self.distribution() if current else val in self._frozen_dist

    def convert_idx_to_val(self, idx):
        if not self.contain_idx(idx=idx, current=False):
            raise KeyError(f"Input index '{idx}' is NOT a proper index for conversion via Var:{self.var_type}'s distribution: {list(range(len(self._frozen_dist))) if self.sample_method=='categorical' else self._frozen_dist}")
        if self.sample_method!='categorical':
            return idx
        return self._frozen_dist[idx]

    def convert_val_to_idx(self, val):
        if not self.contain_val(val=val, current=False):
            raise ValueError(f"Input value '{val}' is NOT a proper value for conversion via Var:{self.var_type}'s distribution: {self._frozen_dist}")
        if self.sample_method!='categorical':
            return val
        return self._frozen_dist.index(val)

    def distribution(self, indices: bool = False, current: bool = True):
        if self.sample_method!='categorical':
            return self._dist if current else self._frozen_dist
        if indices:
            return self._dist if current else list(range(len(self._frozen_dist)))
        return [self._frozen_dist[idx] for idx in self._dist] if current else self._frozen_dist

    def sample(self, trial):
        #Ensure: trial is an instance of Trial, FrozenTrial.
        if not isinstance(trial, (Trial, FrozenTrial)):
            raise ValueError(f"Input 'trial' must be of type in (optuna::Trial, optuna::FrozenTrial, Mock) but found: {trial}")

        name=self.var_type+'>'+str(self._belongs_to)

        sampled_val = None
        if isinstance(trial, FrozenTrial):
            #idx_or_val <- FrozenTrial[name] or randomly chosen value from self._dist
            idx_or_val = trial.params.get(name, self.random_sample())
            #sample_method=='categorical' <-> retrieved value is an index NOT an actual value.
            sampled_val = self.convert_idx_to_val(idx_or_val) if self.sample_method=='categorical' else idx_or_val
        elif self.sample_method == 'categorical':
            idx = trial.suggest_categorical(name, choices=self._dist)
            sampled_val = self._frozen_dist[idx]
        elif self.sample_method == 'float':
            sampled_val = trial.suggest_float(name, **self._dist)
        else: #Guaranteed that it's of 'int'.
            sampled_val = trial.suggest_int(name, **self._dist)
        return sampled_val

    def random_sample(self):
        if self.sample_method=='categorical':
            return random.choice(self._dist)
        if self.sample_method=='int':
            return random.choice(list(range(self._dist['low'], self._dist['high']+1, self._dist.get('step', 1))))
        if self._dist.get('log', False):
            return math.exp(random.uniform(math.log(self._dist['low']), math.log(self._dist['high'])))
        return random.uniform(self._dist['low'], self._dist['high'])

    def update(self, vals: list):
        #Proior processing:
        if not len(vals):
            return
        if not all(isinstance(val, (int,float)) for val in vals):
            raise ValueError(f"Var of type: {self.var_type} with sample_method: {self.sample_method} recieved update request with 'vals': {vals} which contain element(s) NOT of type in (int,float)")
        vals = sorted(vals.copy()) if self.sample_method=='categorical' else vals

        set_vals = set(vals)
        if len(set_vals)==1:
            vals = [vals[0]]

        #ERROR CHECKS:
        if self.sample_method!='categorical' and (min(vals)<self._dist['low'] or max(vals)>self._dist['high']):
            raise ValueError(f"Var of type: {self.var_type} with sample_method: {self.sample_method} recieved update request with 'vals': {vals} NOT in valid range: [{self._dist['low']},{self._dist['high']}]. Note the initial search space: [{self._frozen_dist['low']},{self._frozen_dist['high']}].")
        if self.sample_method=='categorical' and not set_vals.issubset(set(self._dist)):
            raise ValueError(f"Var of type: {self.var_type} with sample_method: {self.sample_method} recieved update request with 'vals': {vals} NOT in valid range: {self._dist}. Note the initial search space: {self._frozen_dist}")

        #UPDATE: configuration space.
        if self.sample_method=='categorical':
            self._dist = vals
        else:
            self._dist['low'] = min(vals)
            self._dist['high'] = max(vals)

#Test: Var

In [5]:
class TestVar(unittest.TestCase):
    def setUp(self):
        self.categorical_source = {
            'params': {'choices': [1, 2, 3, 4, 5]},
            'sample': 'categorical'
        }
        self.float_source = {
            'params': {'low': 0.1, 'high': 1.0},
            'sample': 'float'
        }
        self.int_source = {
            'params': {'low': 1, 'high': 10},
            'sample': 'int'
        }

    def test_init_categorical(self):
        var = Var(var_type='test_var', source=self.categorical_source)
        self.assertEqual(var.var_type, 'test_var')
        self.assertEqual(var.sample_method, 'categorical')
        self.assertEqual(var._frozen_dist, [1, 2, 3, 4, 5])
        self.assertEqual(var._dist, [0, 1, 2, 3, 4])
        self.assertFalse(var.is_fixed())

    def test_init_float(self):
        var = Var(var_type='test_var', source=self.float_source)
        self.assertEqual(var.var_type, 'test_var')
        self.assertEqual(var.sample_method, 'float')
        self.assertEqual(var._frozen_dist, {'low': 0.1, 'high': 1.0})
        self.assertEqual(var._dist, {'low': 0.1, 'high': 1.0})
        self.assertFalse(var.is_fixed())

    def test_init_int(self):
        var = Var(var_type='test_var', source=self.int_source)
        self.assertEqual(var.var_type, 'test_var')
        self.assertEqual(var.sample_method, 'int')
        self.assertEqual(var._frozen_dist, {'low': 1, 'high': 10})
        self.assertEqual(var._dist, {'low': 1, 'high': 10})
        self.assertFalse(var.is_fixed())

    def test_init_error_invalid_sample_method(self):
        with self.assertRaises(ValueError):
            Var(var_type='test_var', source={'params': {'choices': [1, 2]}, 'sample': 'invalid'})

    def test_init_error_invalid_distribution_type(self):
        with self.assertRaises(ValueError):
            Var(var_type='test_var', source={'params': {'choices': {'key': 'value'}}, 'sample': 'categorical'})

    def test_contain_idx(self):
        var = Var(var_type='test_var', source=self.categorical_source)
        self.assertTrue(var.contain_idx(2))
        self.assertFalse(var.contain_idx(5))

        var_float = Var(var_type='test_var', source=self.float_source)
        self.assertTrue(var_float.contain_idx(0.5))
        self.assertFalse(var_float.contain_idx(1.1))

    def test_contain_val(self):
        var = Var(var_type='test_var', source=self.categorical_source)
        self.assertTrue(var.contain_val(3))
        self.assertFalse(var.contain_val(6))

        var_float = Var(var_type='test_var', source=self.float_source)
        self.assertTrue(var_float.contain_val(0.5))
        self.assertFalse(var_float.contain_val(1.1))

    def test_convert_idx_to_val(self):
        var = Var(var_type='test_var', source=self.categorical_source)
        self.assertEqual(var.convert_idx_to_val(2), 3)

        var_float = Var(var_type='test_var', source=self.float_source)
        self.assertEqual(var_float.convert_idx_to_val(0.5), 0.5)

    def test_convert_val_to_idx(self):
        var = Var(var_type='test_var', source=self.categorical_source)
        self.assertEqual(var.convert_val_to_idx(3), 2)

        with self.assertRaises(ValueError):
            var.convert_val_to_idx(6)

    def test_distribution(self):
        var = Var(var_type='test_var', source=self.categorical_source)
        self.assertEqual(var.distribution(), [1, 2, 3, 4, 5])

        var_float = Var(var_type='test_var', source=self.float_source)
        self.assertEqual(var_float.distribution(), {'low': 0.1, 'high': 1.0})

    def test_sample_categorical(self):
        var = Var(var_type='test_var', source=self.categorical_source)
        trial = optuna.create_study(direction='minimize').ask()
        trial.suggest_categorical = Mock(return_value=2)
        sampled_val = var.sample(trial)
        self.assertEqual(sampled_val, 3)

    def test_sample_float(self):
        var = Var(var_type='test_var', source=self.float_source)
        trial = optuna.create_study(direction='minimize').ask()
        trial.suggest_float = Mock(return_value=0.5)
        sampled_val = var.sample(trial)
        self.assertEqual(sampled_val, 0.5)

    def test_sample_error(self):
        var = Var(var_type='test_var', source=self.categorical_source)
        with self.assertRaises(ValueError):
            var.sample('not_a_trial')

    def test_update_categorical(self):
        var = Var(var_type='test_var', source=self.categorical_source)
        var.update([1, 2])
        self.assertEqual(var.distribution(indices=True), [1, 2])
        self.assertEqual(var.distribution(), [2, 3])
        self.assertFalse(var.is_fixed())

    def test_update_float(self):
        var = Var(var_type='test_var', source=self.float_source)
        var.update([0.2, 0.9])
        self.assertEqual(var._dist['low'], 0.2)
        self.assertEqual(var._dist['high'], 0.9)

    def test_update_error_out_of_range(self):
        var = Var(var_type='test_var', source=self.categorical_source)
        with self.assertRaises(ValueError):
            var.update([0, 6])

    def test_str(self):
        var = Var(var_type='test_var', source=self.categorical_source)
        expected_str = f"Var of type: test_var, signature of: {'None'}\nConfiguration Space:\n'choices' = [1, 2, 3, 4, 5]\n"
        self.assertEqual(str(var), expected_str)

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

..........[I 2024-10-13 21:16:19,125] A new study created in memory with name: no-name-984b08c4-2021-495e-831d-46c2b0b5ba39
..[I 2024-10-13 21:16:19,130] A new study created in memory with name: no-name-810e69d6-9974-4d2c-ada6-8080de293174
.....
----------------------------------------------------------------------
Ran 17 tests in 0.024s

OK


#Class: Action, ModelAction

In [6]:
class Action:
    def __init__(self, action_func: Callable, sig_dict: dict):
        if not all(isinstance(var,Var) for var in sig_dict.values()):
            raise ValueError(f"Constructor of object 'Action' with action_func: '{action_func.__name__}' received an invalid dictionary sig_dict: {sig_dict}")
        self.action_func = action_func
        self.sig_dict = sig_dict
        self.sig = frozenset(sig_dict.keys())
        self.params = {}
        self.action_state = None
        self._model_loader = None

    def __len__(self):
        return len(self.sig_dict)

    def __hash__(self):
        return hash(self.action_func)

    def __eq__(self, other: Action):
        """Two Action objects are considered 'equal'
        IFF (1)other is Action object
            (2)self.action_func == other.action_func
            (3)self.params == other.params
            (4)self.action_state == other.action_state
        """
        #TYPE CHECK:
        if (isinstance(self, ModelAction) and not isinstance(other, ModelAction)) or not isinstance(other, Action):
            return False
        #Model loader comparison:
        self_model_params = self._model_loader if self._model_loader is None else self._model_loader.keywords
        other_model_params = other._model_loader if other._model_loader is None else other._model_loader.keywords
        if self_model_params != other_model_params:
            return False

        #Compare action states if both are dictionaries
        if isinstance(self.action_state, dict) and isinstance(other.action_state, dict):
            if self.action_state.keys() != other.action_state.keys():
                return False
            if not all(torch.equal(self.action_state[key], other.action_state[key]) for key in self.action_state.keys()):
                return False
        elif self.action_state != other.action_state: #IF one is None and the other is not None.
            return False

        return all([
            self.action_func == other.action_func,
            self.sig_dict == other.sig_dict,
        ])

    def __str__(self):
        if self._model_loader:
            parameters = " ".join([f"{key}={val}" for key,val in self._model_loader.keywords.items()])
        else:
            parameters = " ".join([f"{key}" for key in self.sig_dict.keys()])
        return f"Action of func: {self.action_func.__name__}\n" + f"Model loader parameters: {parameters}\n"

    def get_var(self, var_type):
        if var_type not in self.keys():
            raise KeyError(f"'{var_type}' is not a valid signature key for Action: '{self.action_func.__name__}'. Available keys: {set(self.keys())}")
        return self.sig_dict[var_type]

    def keys(self):
        return self.sig_dict.keys()

    def values(self):
        return self.sig_dict.values()

    def items(self):
        return self.sig_dict.items()

    def is_loadable(self):
        return self._model_loader is not None

    def is_fixed(self):
        return all(var.fixed for var in self.sig_dict.values())

    def parameterize(self, trial: Trial):
        """
        ACCEPTS:
        trial := Active optuna trial objet.
                 IF input 'manual_action_params' is not empty, THEN 'trial' will NOT be used.
        RETURNS: void.
                 The function parameterizes the caller Action object.
                 Without any saved action_state, the caller Action object needs to be parameterized every time before .__call__()
        """
        #ERROR CHECKS:
        if self.action_state is not None and not self.params:
            raise RuntimeError(f"Action: {self.action_func.__name__} has a saved action state(self.action_state!=None) but NO saved params(self.params==None).")
        if self.action_state is None and trial is None:
            raise ValueError(f"Action: {self.action_func.__name__} has no saved action state(self.action_state==None) and no input trial passed in(self.trial==None). No sampling can be done.")

        if self.action_state is None:
            self.params = {cat:var.sample(trial=trial) for cat,var in self.items()}

        #self._model_loader <- Fully parameterized model with self.params
        #NOTE: IF it has a saved action state, the consistent self.params will restore the recenmost state(based on the recentmost top sampling results)
        self._model_loader = partial(self.action_func, **self.params)

    def __call__(self, device: torch.device=None):
        if self.action_state is not None and not self.params:
            raise RuntimeError(f"Action: {self.action_func.__name__} has a saved action state(self.action_state!=None) but no saved params exists(self.params empty). Likely an internal logic issue")
        if self._model_loader is None:
            raise RuntimeError(f"Action: {self.action_func.__name__} cannot be called. Call Action.parameterize() first and then try it again.")

        #Try: Applying 'device' to the model_loader and load.
        try:
            model = self._model_loader(device=device)
        except Exception as e: #IF fails: Load the model w/o device.
            model = self._model_loader()

        #IF 'self.action_state' exists, THEN load the saved weights from the state dictionary(=self.action_state) to the model
        if self.action_state:
            try:
                name = self.action_func.__name__
                model.load_state_dict(self.action_state)
            except torch.nn.modules.module.ModuleAttributeError as e:
                raise ValueError(f"Attribute error: {e}. The state dictionary of Action: '{name}' may not match the model.")
            except RuntimeError as e:
                raise RuntimeError(f"Runtime error: {e}. The state dictionary of Action: '{name}' may have unmatching layer sizes or missing keys.")
            except Exception as e:
                raise ValueError(f"An unexpected error occurred while loading state dictionary to Action '{name}': {e}")

        #Last but not least, return the model.
        return model

    def update_sig_vars(self, update_dict: dict):
        #IF Action (1)has a saved 'action_state' OR (2)0 elements in the 'list_params'
        #THEN NO update shall be done.
        if self.action_state or not len(update_dict):
            return

        leftovers = set(self.keys()) - set(update_dict.keys())
        if leftovers:
            raise RuntimeError(f"Action: {self.action_func.__name__} requires following missing keys in 'update_dict': {leftovers}")

        #Update: signature configuration space(=='self.sig_dict')
        for var_type,var in self.items():
            var.update(vals=update_dict[var_type])

    #Calling Action.update_state() will raise a helpful Error.
    def update_state(self, obj):
        return NotImplementedError("Erroneous call to Action.update_state()")

class ModelAction(Action):
    def __init__(self, action_func: Callable, sig_dict: dict):
        super().__init__(action_func=action_func, sig_dict=sig_dict)

    def update_state(self, obj):
        #Ensure that input obj has a correct type/format.
        #IF NOT, simply 'return'.
        #NOTE: This can be extended to allow more datastructures.
        if obj is None or not isinstance(obj, (nn.Module, dict)) and not(hasattr(obj, 'model') and isinstance(obj.model, nn.Module)):
            return
        #Ensure that no update is done if the model was not sampled in the previous run.
        if self.action_state is not None and self.params != self._model_loader.keywords:
            print(f"ModelAction: {self.action_func.__name__} was not sampled in the previous HPO run or no HPO run has been executed yet. This call will not change the action state")
            return

        if isinstance(obj, dict):
            self.action_state = obj.copy() #shallow copy it for safety
        else:
            self.action_state = obj.state_dict() if isinstance(obj, nn.Module) else obj.model.state_dict()

        #Manually set 'self.params'. This way, saved action_state -> self.params available!
        self.params = self._model_loader.keywords.copy()

#Test: Action, ModelAction

In [7]:
"""
Parameters used in the testin of class 'ModelAction' defined in this cell
"""
if __name__ == '__main__':
    class ModelDefaultTestAction(nn.Module):
        def __init__(self, rand_number: int):
            super().__init__()
            self.net = nn.Linear(16,10)
            self.rand_number = torch.tensor(rand_number)

        def forward(self, x):
            return self.net(x) + self.rand_number

    class ModelDropoutTestAction(nn.Module):
        def __init__(self, activation: nn.Module, dropout_p: int):
            super().__init__()
            self.net = nn.Sequential(
                nn.Linear(in_features=16,out_features=10),
                activation(),
                nn.Dropout(p=dropout_p),
                nn.Linear(in_features=10,out_features=10)
            )

        def forward(self, x):
            return self.net(x)

    def action_default_test_action(rand_number: int, device: torch.device):
        return ModelDefaultTestAction(rand_number=rand_number).to(device)

    def action_dropout_test_action(activation, dropout_p, device: torch.device):
        #model_dropout := Class header of 'ModelDropoutTestAction'
        return ModelDropoutTestAction(activation=activation, dropout_p=dropout_p).to(device)

class TestModelAction(unittest.TestCase):
    def setUp(self):
        default_sig_dict = {
            'rand_number': Var(var_type='rand_number', source={'params': {'choices': list(range(1, 11))}, 'sample': 'categorical'})
        }
        dropout_sig_dict = {
            'activation': Var(var_type='activation', source={'params': {'choices': [nn.ReLU, nn.PReLU, nn.SiLU]}, 'sample': 'categorical'}),
            'dropout_p': Var(var_type='dropout_p', source={'params': {'low': 0.0, 'high': 1.0}, 'sample': 'float'})
        }

        self.action_sampler = {
            action_default_test_action: ModelAction(action_default_test_action, default_sig_dict),
            action_dropout_test_action: ModelAction(action_dropout_test_action, dropout_sig_dict)
        }

        #Create a trial object for parameterization
        self.trial = optuna.create_study(direction='minimize').ask()

    def test_initialization(self):
        default_action = self.action_sampler[action_default_test_action]
        dropout_action = self.action_sampler[action_dropout_test_action]

        #Test if ModelAction objects are initialized correctly
        self.assertIsInstance(default_action, ModelAction)
        self.assertIsInstance(dropout_action, ModelAction)

        self.assertTrue(default_action.action_func == action_default_test_action)
        self.assertIsInstance(default_action.get_var('rand_number'), Var)
        self.assertTrue(
            default_action.get_var('rand_number').distribution() == list(range(1, 11)),
            msg=f"{default_action.get_var('rand_number').distribution()} != {list(range(1, 11))}"
        )
        self.assertFalse(default_action.params)
        self.assertIsNone(default_action.action_state)
        self.assertIsNone(default_action._model_loader)

        self.assertTrue(dropout_action.action_func == action_dropout_test_action)
        self.assertIsInstance(dropout_action.get_var('activation'), Var)
        self.assertIsInstance(dropout_action.get_var('dropout_p'), Var)
        self.assertTrue(dropout_action.get_var('activation').distribution() == [nn.ReLU, nn.PReLU, nn.SiLU])
        self.assertFalse(dropout_action.params)
        self.assertIsNone(dropout_action.action_state)
        self.assertIsNone(dropout_action._model_loader)

    def test_default_parameterize_with_action_state(self):
        default_action = self.action_sampler[action_default_test_action]
        default_action.action_state = 'dummy'
        default_action.params = {'rand_number': 1}

        #Test parameterization with the default action
        default_action.parameterize(trial=self.trial)

        #Posterior testing
        self.assertIsInstance(default_action, ModelAction)
        self.assertIsNotNone(default_action.params)
        self.assertIsNotNone(default_action.action_state)
        self.assertIsNotNone(default_action._model_loader)

        try:
            model = default_action(device=torch.device('cpu'))
        except Exception as e:
            pass

    def test_default_parameterize(self):
        default_action = self.action_sampler[action_default_test_action]
        #Test parameterization with the default action
        default_action.parameterize(trial=self.trial)
        self.assertTrue(set(default_action.params.keys()), set(['rand_number']))
        self.assertTrue(default_action.params['rand_number'] in list(range(1, 11)))
        self.assertIsNone(default_action.action_state)
        self.assertIsNotNone(default_action._model_loader)
        model = default_action(device=torch.device('cpu'))
        X = torch.randn(size=[10, 16])
        Y = model(X)
        self.assertEqual(Y.shape, torch.Size([10, 10]))

        self.assertIsNotNone(default_action.params)
        self.assertIsNone(default_action.action_state)
        self.assertIsNotNone(default_action._model_loader)

    def test_dropout_parameterize_with_action_state(self):
        dropout_action = self.action_sampler[action_dropout_test_action]
        dropout_action.params = {
            'activation': nn.ReLU,
            'dropout_p': 0.5
        }
        model = ModelDropoutTestAction(activation=nn.ReLU, dropout_p=0.5).to(torch.device('cpu'))
        dropout_action.action_state = model.state_dict()

        #Test parameterization with the default action
        dropout_action.parameterize(trial=self.trial)

        #Posterior testing
        self.assertIsInstance(dropout_action, ModelAction)
        self.assertIsNotNone(dropout_action.params)
        self.assertIsNotNone(dropout_action.action_state)
        self.assertIsNotNone(dropout_action._model_loader)

        try:
            model = dropout_action(device=torch.device('cpu'))
            self.assertTrue(model(torch.randn(size=[10, 16])).shape, torch.Size([10, 10]))
        except Exception as e:
            pass

        #Final testing
        self.assertIsNotNone(dropout_action.params)
        self.assertIsNotNone(dropout_action.action_state)
        self.assertIsNotNone(dropout_action._model_loader)

    def test_dropout_parameterize(self):
        dropout_action = self.action_sampler[action_dropout_test_action]

        #Prior Testing
        self.assertIsInstance(dropout_action, ModelAction)
        self.assertTrue(dropout_action.params == {})
        self.assertIsNone(dropout_action.action_state)
        self.assertIsNone(dropout_action._model_loader)

        #Test parameterization with the default action
        dropout_action.parameterize(trial=self.trial)

        #Posterior Testing
        self.assertTrue(set(dropout_action.params.keys()), set(['activation', 'dropout_p']))
        self.assertTrue(dropout_action.params['activation'] in [nn.ReLU, nn.PReLU, nn.SiLU])
        self.assertTrue(0.0 <= dropout_action.params['dropout_p'] <= 1.0)
        self.assertIsNone(dropout_action.action_state)
        self.assertIsNotNone(dropout_action._model_loader)

        #Model loading & testing
        model = dropout_action(device=torch.device('cpu'))
        self.assertEqual(model(torch.randn(size=[10, 16])).shape, torch.Size([10, 10]))

        #Final testing
        self.assertIsNotNone(dropout_action.params)
        self.assertIsNone(dropout_action.action_state)
        self.assertIsNotNone(dropout_action._model_loader)

    def test_update_sig_vars(self):
        #Preparation
        default_action = self.action_sampler[action_default_test_action]
        dropout_action = self.action_sampler[action_dropout_test_action]

        default_new_sig_params = {
            'rand_number': [2]
        }
        dropout_new_sig_params = {
            'activation': [1, 2],
            'dropout_p': [0.45, 0.5, 0.72]
        }

        #TEST: default_action FOR: sig_dict
        default_action.update_sig_vars(update_dict=default_new_sig_params)
        self.assertIsInstance(default_action.get_var('rand_number'), Var)
        self.assertTrue(default_action.get_var('rand_number').distribution() == [3])
        self.assertTrue(default_action.get_var('rand_number').distribution(indices=True) == [2])

        #TEST: dropout_action FOR: sig_dict
        dropout_action.update_sig_vars(update_dict=dropout_new_sig_params)
        self.assertIsInstance(dropout_action.get_var('activation'), Var)
        self.assertTrue(dropout_action.get_var('activation').distribution() == [nn.PReLU, nn.SiLU],
                        msg=f"{dropout_action.get_var('activation').distribution()} != {[nn.PReLU, nn.SiLU]}")
        self.assertTrue(dropout_action.get_var('activation').distribution(True) == [1, 2],
                        msg=f"{dropout_action.get_var('activation').distribution(True)} != {[1, 2]}")
        self.assertIsInstance(dropout_action.get_var('dropout_p'), Var)
        dropout_p_var = dropout_action.get_var('dropout_p')
        self.assertTrue(dropout_p_var.distribution()['low'] == 0.45 and dropout_p_var.distribution()['high'] == 0.72)

    #Additional method to test update_state if needed
    def test_update_state(self):
        for i in range(5):
            default_action = self.action_sampler[action_default_test_action]
            dropout_action = self.action_sampler[action_dropout_test_action]

            if not i:
                self.assertTrue(default_action.params == {})
                self.assertIsNone(default_action.action_state)
                self.assertIsNone(default_action._model_loader)

                self.assertTrue(dropout_action.params == {})
                self.assertIsNone(dropout_action.action_state)
                self.assertIsNone(dropout_action._model_loader)
            else:
                self.assertFalse(default_action.params == {})
                self.assertIsNotNone(default_action.action_state)
                self.assertIsNotNone(default_action._model_loader)

                self.assertFalse(dropout_action.params == {})
                self.assertIsNotNone(dropout_action.action_state)
                self.assertIsNotNone(dropout_action._model_loader)

            default_action.parameterize(trial=self.trial)
            dropout_action.parameterize(trial=self.trial)

            default_model = default_action(device=torch.device('cpu'))
            dropout_model = dropout_action(device=torch.device('cpu'))

            self.assertEqual(default_model(torch.randn(size=[10, 16])).shape, torch.Size([10, 10]))
            self.assertEqual(dropout_model(torch.randn(size=[10, 16])).shape, torch.Size([10, 10]))

            default_action.update_state(obj=default_model)
            dropout_action.update_state(obj=dropout_model)

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

[I 2024-10-13 21:16:19,355] A new study created in memory with name: no-name-11234915-b8d5-4044-93d2-1b37223b25ea
.[I 2024-10-13 21:16:19,488] A new study created in memory with name: no-name-d2c25189-0e2e-4be6-a4d1-874456c06734
.[I 2024-10-13 21:16:19,493] A new study created in memory with name: no-name-836d6a09-2b6a-48f4-b30f-77de70fa75da
.[I 2024-10-13 21:16:19,533] A new study created in memory with name: no-name-3f154197-becd-41de-bc29-c7b0e93c622c
.[I 2024-10-13 21:16:19,556] A new study created in memory with name: no-name-9ad5292e-4102-4b76-ba3f-4cdbc89f8efa
.[I 2024-10-13 21:16:19,559] A new study created in memory with name: no-name-72a05e97-6e93-4351-9c35-fac42f5281f5
.[I 2024-10-13 21:16:19,563] A new study created in memory with name: no-name-0b040069-4c56-42af-af9d-ab2602134375
...........[I 2024-10-13 21:16:19,581] A new study created in memory with name: no-name-2fdc6735-9da4-4a04-b898-06c79eb32152
..[I 2024-10-13 21:16:19,584] A new study created in memory with name: 

#Class: ObjectiveExecutor

In [8]:
class ObjectiveExecutor:
    def __init__(self, trial, combined_sampler: CombinedSampler, device: torch.device, default_dict: dict = {}):
        #Instance of ObjectiveExecutor must have an attribute 'execute_objective' set BEFOREHAND
        if not hasattr(self, 'execute_objective'):
            raise RuntimeError("Class ObjectiveExecutor must have an attribute 'execute_objective' set before any instance of it can be initialized. This is an internal issue.")

        #Perform general-error checking
        self._validate_inputs(trial=trial, combined_sampler=combined_sampler, default_dict=default_dict)

        #Initialize all the member variables.
        self.trial = trial
        self.default_dict = default_dict if self.trial is None else {}
        self.combined_sampler = combined_sampler if self.trial is not None else None
        self.sampled_var_types = set()
        self.device = device if device else torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    def _validate_inputs(self, trial: Trial, combined_sampler: CombinedSampler, default_dict: dict):
        #trial type check:
        if trial is not None and not isinstance(trial, (Trial, FrozenTrial)):
            raise ValueError(f"Input 'trial' must either be 'None' or an instance in (optuna::Trial, optuna::FrozenTrial) but found: {trial}")

        #combined_sampler type check:
        if combined_sampler is not None and not isinstance(combined_sampler, CombinedSampler):
            raise ValueError(f"Input 'combined_sampler' must either be None or of type CombinedSampler but found: {combined_sampler}")

        #Make sure there's something to be sampled:
        if not default_dict and not combined_sampler:
            raise ValueError(f"Both default_dict and combined_sampler are empty. Nothing to be sampled.")

        #Check for internal consistency.
        assert (trial is None and default_dict) or trial is not None and combined_sampler, f"trial: {trial}, combined_sampler: {combined_sampler}, default_dict: {default_dict}"

    def sample(self, var_type: str, default_val = None):
        #No double sampling allowed
        if var_type in self.sampled_var_types:
            raise ValueError(f"Variable '{var_type}' has already been sampled. Variables can only be sampled once per trial.")

        sampled_val = None
        if self.trial is None:
            sampled_val = self.default_dict.get(var_type, default_val)
        else:
            sampled_val = self.combined_sampler.sample(var_type, self.trial, default_val)

        #Add var_type to the set of all sampled var_types.
        self.sampled_var_types.add(var_type)

        return sampled_val

#Class ActionVar

In [9]:
class ActionVar(Var):
    """
    ActionVar is a subclass of Var that stores Action/ModelAction objects in its `_frozen_dist`.
    This enables automated sampling of actions and their parameters during HPO.
    """
    def __init__(self, var_type: str, source: dict, belongs_to: str=None):
        super().__init__(var_type=var_type, source=source, belongs_to=belongs_to)

    def is_fixed(self):
        """
        ActionVar is fixed if:
        - Only one sampleable value in the configuration space.
        - All signature variables of the ActionVar are fixed.
        """
        if len(self._dist) > 1:
            return False
        singleton_action = self._frozen_dist[self._dist[0]]
        return all(var.is_fixed() for var in singleton_action.values())

    def get_action(self, action_idx_or_action_func):
        if isinstance(action_idx_or_action_func, int):
            return self.convert_idx_to_val(action_idx_or_action_func)
        action_list = [action for action in self._frozen_dist if action.action_func == action_idx_or_action_func]
        if not action_list:
            raise KeyError(f"No action found with action_func: {action_idx_or_action_func} in ActionVar: {self.var_type}.")
        return action_list[0]

    def get_var(self, action_idx_or_action_func, var_type):
        action = self.get_action(action_idx_or_action_func)
        return action.get_var(var_type)

    #Overwrites Var.sample
    def sample(self, trial):
        #Ensure: trial is an instance of Trial, FrozenTrial.
        if not isinstance(trial, (Trial, FrozenTrial)):
            raise ValueError(f"Input 'trial' must be of type in (optuna::Trial, optuna::FrozenTrial, Mock) but found: {trial}")
        if isinstance(trial, Mock) and not(isinstance(trial, FrozenTrial) and hasattr(trial, 'params')):
            msg = "mock trial does not have attribute 'params'" if isinstance(trial, FrozenTrial) else "mock trial does not have 'spec' set to optuna.trial.FrozenTrial."
            raise ValueError(f"Input 'trial' is an instance of unittest.mock.Mock but {msg}")

        #This will sample the right 'Action' object from the distribution.
        action = super().sample(trial)
        assert isinstance(action, Action), f"type(action): {type(action)} != Action"

        #Parameterize the drawn Action object with the same trial.
        action.parameterize(trial=trial)
        assert action.is_loadable(), f"action object: {action} is not loadable even though it was parameterized"

        #Return the fully parameterized Action object.
        return action

    def get_loadable_action(self, action_identifier, source) -> Action:
        """
        Retrieves a fully parameterized Action object.

        Args:
            action_identifier: An instance of Action, a Callable action function, or an integer index.
            source: A dictionary or optuna Trial/FrozenTrial containing parameter mappings.

        Returns:
            A fully parameterized Action object.
        """
        if not isinstance(source, (Trial, FrozenTrial, dict)):
            raise ValueError(f"Input 'source' must be of type dict, Trial, or FrozenTrial, but found: {type(source)}")
        if not (isinstance(action_identifier, (int, Action)) or callable(action_identifier)):
            raise TypeError(f"Input 'action_identifier' should be an int, Action, or Callable, but found: {type(action_identifier)}")

        #Get the action based on the identifier
        if isinstance(action_identifier, Action):
            action = action_identifier
        else:
            action = self.get_action(action_identifier)

        #If the action has a saved state, return it
        if action.action_state:
            return action

        #Extract parameters from the source
        source = source if isinstance(source, dict) else source.params
        params = {
            var_type: val for (var_type, belongs_to), val in source.items()
            if var_type in action.keys() and belongs_to == self.var_type
        }

        #Check for missing parameters
        missing_params = set(action.keys()) - set(params.keys())
        if missing_params:
            raise KeyError(f"Action '{action.action_func.__name__}' requires missing keys in 'source': {missing_params}. \nNOTE: 'source': {source}, 'params': {params}")

        #Convert indices to actual values
        for var_type, var in action.items():
            val = params[var_type]
            if var.contain_idx(val):
                val = var.convert_idx_to_val(val)
            params[var_type] = val

        #Update the action's model loader
        action._model_loader = partial(action.action_func, **params)

        return action

    def update_actions(self, update_action_idxs: list, list_params: list, update_state_window: int, device: torch.device):
        """
        Updates actions based on recent sampling results.

        Args:
            update_action_idxs: List of unique indices representing actions to update.
            list_params: List of parameter dictionaries from recent optuna trials.
                         Every 'params' in the list_params is guaranteed to have (self.var_type,'None') as its key and therefore,
                         will have all the (self.keys(),self.var_type) as keys as well.
            update_state_window: Number of top actions to update states for.
            device: Torch device to use for model actions.
        """
        update_state_window = min(update_state_window, len(update_action_idxs))
        has_meaningful_state = isinstance(self.distribution()[0], ModelAction)
        for count,idx in enumerate(update_action_idxs):
            action = self.get_action(action_idx_or_action_func=idx)

            #Update signature vars of the retrieved action.
            update_dict = {
                var_type:[params[(var_type, self.var_type)] for params in list_params if (var_type, self.var_type) in params.keys()] for var_type in action.keys()
            }
            action.update_sig_vars(update_dict=update_dict)

            if not has_meaningful_state or count >= update_state_window:
                continue

            #Ensure that there's at least one params in 'list_params' that sampled idx.
            matching_params =  [params for params in list_params if params[(self.var_type,'None')]==idx]
            if not matching_params:
                raise ValueError(f"index: {idx} in 'update_action_idxs' does NOT have any matching params in 'list_params'. Make sure to pass in 'list_params' with at least one params that contain: ({self.var_type},'None')->{idx}")

            #Find the best parameters for this action
            best_params = matching_params[0]

            #Get a fully loadable action
            action = self.get_loadable_action(idx,best_params)

            #Prepare parameters for objective execution
            default_dict = {var_type:val for (var_type,belongs_to),val in best_params.items() if belongs_to == 'None'}
            default_dict[self.var_type] = action

            #Prepare and execute ObjectiveExecutor. Make sure to SHOW the logging info from trainer.
            _, model_container = ObjectiveExecutor(
                trial=None,
                combined_sampler=None,
                device=device,
                default_dict=default_dict
            ).execute_objective(turn_off_training_log=False)

            #Update the action's state with the model_container.
            action.update_state(obj=model_container)

#Test: ActionVar

In [10]:
if __name__ == '__main__':
    class UserObjectiveActionVar:
        def objective(self, turn_off_training_log: bool):
            model = self.default_dict['model_action'](device=self.device)

            for param in model.parameters():
                param.data.fill_(0)

            model_container = Mock(spec=Learner)
            model_container.model = model

            return (0, model)

    #Sample model classes
    class ModelDefaultActionVar(nn.Module):
        def __init__(self, rand_number: int):
            super().__init__()
            self.net = nn.Linear(16, 10)
            self.rand_number = torch.tensor(rand_number)

        def forward(self, x):
            return self.net(x) + self.rand_number

    class ModelDropoutActionVar(nn.Module):
        def __init__(self, activation: nn.Module, dropout_p: float):
            super().__init__()
            self.activation = activation
            self.dropout_p = dropout_p
            self.net = nn.Sequential(
                nn.Linear(16, 10),
                activation(),
                nn.Dropout(p=dropout_p),
                nn.Linear(10, 10)
            )

        def forward(self, x):
            return self.net(x)

    #model_action functions
    def model_action_default_actionvar(rand_number: int, device: torch.device):
        return ModelDefaultActionVar(rand_number=rand_number).to(device)

    def model_action_dropout_actionvar(activation, dropout_p, device: torch.device):
        return ModelDropoutActionVar(activation=activation, dropout_p=dropout_p).to(device)

    #dls_action functions
    def dls_action_default_actionvar(batch_size: int, device: torch.device):
        return {'batch_size': batch_size, 'device': device}

    def dls_action_normalize_actionvar(batch_size: int, normalize: bool):
        return {'batch_size': batch_size, 'normalize': normalize}

    def dls_action_activation_actionvar(batch_size: int, normalize: bool, activation: nn.Module):
        return {'batch_size': batch_size, 'normalize': normalize, 'activation': activation}

class TestActionVar(unittest.TestCase):
    def setUp(self):
        #Define Var objects for parameters of 'model_actions'.
        rand_number_var = Var(
            var_type='rand_number',
            source={'params': {'choices': list(range(1, 11))}, 'sample': 'categorical'},
            belongs_to='model_action'
        )
        activation_var = Var(
            var_type='activation',
            source={'params': {'choices': [nn.ReLU, nn.PReLU, nn.SiLU]}, 'sample': 'categorical'},
            belongs_to='model_action'
        )
        dropout_p_var = Var(
            var_type='dropout_p',
            source={'params': {'low': 0.0, 'high': 1.0}, 'sample': 'float'},
            belongs_to='model_action'
        )
        #Define Var objects for parameters of 'dls_actions'.
        batch_size_var_default = Var(
            var_type='batch_size',
            source={'params': {'low': 16, 'high': 64}, 'sample': 'int'},
            belongs_to='dls_action'
        )
        batch_size_var_normalize = Var(
            var_type='batch_size',
            source={'params': {'low': 16, 'high': 64}, 'sample': 'int'},
            belongs_to='dls_action'
        )
        batch_size_var_activation = Var(
            var_type='batch_size',
            source={'params': {'low': 16, 'high': 64}, 'sample': 'int'},
            belongs_to='dls_action'
        )
        normalize_var_normalize = Var(
            var_type='normalize',
            source={'params': {'choices': [True, False]}, 'sample': 'categorical'},
            belongs_to='dls_action'
        )
        normalize_var_activation = Var(
            var_type='normalize',
            source={'params': {'choices': [True, False]}, 'sample': 'categorical'},
            belongs_to='dls_action'
        )
        activation_var_activation = Var(
            var_type='activation',
            source={'params': {'choices': [nn.ReLU, nn.SiLU]}, 'sample': 'categorical'},
            belongs_to='dls_action'
        )

        #Create ModelAction objects
        self.action_default = ModelAction(
            action_func=model_action_default_actionvar,
            sig_dict={'rand_number': rand_number_var}
        )
        self.action_dropout = ModelAction(
            action_func=model_action_dropout_actionvar,
            sig_dict={'activation': activation_var, 'dropout_p': dropout_p_var}
        )
        #Create an ActionVar ModelAction(s).
        self.model_action_var = ActionVar(
            var_type='model_action',
            source={
                'params': {'choices': [self.action_default, self.action_dropout]},
                'sample': 'categorical'
            }
        )

        #Create Action objects for non-stateful actions
        self.dls_action_default_actionvar = Action(
            action_func=dls_action_default_actionvar,
            sig_dict={'batch_size': batch_size_var_default}
        )
        self.dls_action_normalize_actionvar = Action(
            action_func=dls_action_normalize_actionvar,
            sig_dict={'batch_size': batch_size_var_normalize, 'normalize': normalize_var_normalize}
        )
        self.dls_action_activation_actionvar = Action(
            action_func=dls_action_activation_actionvar,
            sig_dict={'batch_size': batch_size_var_activation, 'normalize': normalize_var_activation, 'activation': activation_var_activation}
        )
        #Create an ActionVar for these Actions
        self.dls_action_var = ActionVar(
            var_type='dls_action',
            source={
                'params': {'choices': [self.dls_action_default_actionvar, self.dls_action_normalize_actionvar, self.dls_action_activation_actionvar]},
                'sample': 'categorical'
            }
        )

        #Set ObjectiveExecutor's .execute_objective manually.
        user_objective = UserObjectiveActionVar()
        setattr(ObjectiveExecutor, 'execute_objective', user_objective.objective.__func__)


    def test_model_action_var_init(self):
        #Test initialization of the model_action_var
        self.assertEqual(self.model_action_var.var_type, 'model_action')
        self.assertEqual(self.model_action_var.sample_method, 'categorical')
        self.assertEqual(len(self.model_action_var._frozen_dist), 2)
        self.assertIsInstance(self.model_action_var._frozen_dist[0], ModelAction)
        self.assertIsInstance(self.model_action_var._frozen_dist[1], ModelAction)

    def test_dls_action_var_init(self):
        #Test initialization of the dls_action_var
        self.assertEqual(self.dls_action_var.var_type, 'dls_action')
        self.assertEqual(self.dls_action_var.sample_method, 'categorical')
        self.assertEqual(len(self.dls_action_var._frozen_dist), 3)
        self.assertIsInstance(self.dls_action_var._frozen_dist[0], Action)
        self.assertIsInstance(self.dls_action_var._frozen_dist[1], Action)
        self.assertIsInstance(self.dls_action_var._frozen_dist[2], Action)

    def test_model_is_fixed(self):
        #Initially, the ActionVar is not fixed
        self.assertFalse(self.model_action_var.is_fixed())

        #Fix all variables in the first action
        for var in self.model_action_var._frozen_dist[0].values():
            var._dist = [0]

        #Update the distribution to only include the first action
        self.model_action_var._dist = [0]
        self.assertTrue(self.model_action_var.is_fixed())

    def test_dls_is_fixed(self):
        #Initially, the ActionVar is not fixed
        self.assertFalse(self.dls_action_var.is_fixed())

        #Fix all variables in the first action
        for var in self.dls_action_var._frozen_dist[0].values():
            if var.sample_method == 'categorical':
                var._dist = [0]
            else:
                var._dist['low'] = var._dist['high']

        #Update the distribution to only include the first action
        self.dls_action_var._dist = [0]
        try:
            self.assertTrue(self.dls_action_var.is_fixed())
        except Exception as e:
            raise RuntimeError(f"Distribution: [{self.dls_action_var.distribution()[0].action_func}], ")

    def test_model_get_action(self):
        #Get action by index
        action = self.model_action_var.get_action(0)
        self.assertEqual(action, self.action_default)

        #Get action by function
        action = self.model_action_var.get_action(model_action_dropout_actionvar)
        self.assertEqual(action, self.action_dropout)

        #Test for KeyError when action not found
        with self.assertRaises(KeyError):
            self.model_action_var.get_action(lambda x: x)
        with self.assertRaises(KeyError):
            self.model_action_var.get_action(2)

    def test_dls_get_action(self):
        #Get action by index
        action = self.dls_action_var.get_action(0)
        self.assertEqual(action, self.dls_action_default_actionvar)

        #Get action by function
        action = self.dls_action_var.get_action(dls_action_normalize_actionvar)
        self.assertEqual(action, self.dls_action_normalize_actionvar)

        #Test for KeyError when action not found
        with self.assertRaises(KeyError):
            self.dls_action_var.get_action(lambda x: x)

    def test_model_get_var(self):
        #Retrieve 'rand_number'
        var = self.model_action_var.get_var(0, 'rand_number')
        self.assertEqual(var.var_type, 'rand_number')
        var = self.model_action_var.get_var(model_action_default_actionvar, 'rand_number')
        self.assertEqual(var.var_type, 'rand_number')

        #Retrieve 'dropout_p'
        var = self.model_action_var.get_var(1, 'dropout_p')
        self.assertEqual(var.var_type, 'dropout_p')
        var = self.model_action_var.get_var(model_action_dropout_actionvar, 'dropout_p')
        self.assertEqual(var.var_type, 'dropout_p')

        #Retrieve 'activation'
        var = self.model_action_var.get_var(1, 'activation')
        self.assertEqual(var.var_type, 'activation')
        var = self.model_action_var.get_var(model_action_dropout_actionvar, 'activation')
        self.assertEqual(var.var_type, 'activation')

    def test_dls_get_var(self):
        #Retrieve 'batch_size' from all actions
        for idx in range(3):
            var = self.dls_action_var.get_var(idx, 'batch_size')
            self.assertEqual(var.var_type, 'batch_size')

        #'normalize' exists in actions at dls_action_normalize_actionvar, dls_action_dropout.
        var = self.dls_action_var.get_var(1, 'normalize')
        self.assertEqual(var.var_type, 'normalize')

        var = self.dls_action_var.get_var(2, 'normalize')
        self.assertEqual(var.var_type, 'normalize')

        #'activation' exists only in action at dls_action_activation_actionvar.
        var = self.dls_action_var.get_var(2, 'activation')
        self.assertEqual(var.var_type, 'activation')

        #Test KeyError for specification of wrong sig_var.
        with self.assertRaises(KeyError):
            self.dls_action_var.get_var(0, 'normalize')
        with self.assertRaises(KeyError):
            self.dls_action_var.get_var(1, 'activation')

    def test_get_loadable_action_model_action_var(self):
        #Create a FrozenTrial with fixed parameters
        frozen_trial = optuna.trial.FrozenTrial(
            number=0,  #You can set any trial number.
            trial_id=0,  #You can set any trial id.
            value=None,  #ObjectiveExecutor value (set if you have an objective function)
            datetime_start=None,  # Start time of the trial
            datetime_complete=None,  # Completion time of the trial
            params={
                ('rand_number', 'model_action'): 5,  # Index 5 <-> Real value 6.
                ('activation', 'model_action'): 0,   # Index of nn.ReLU
                ('dropout_p', 'model_action'): 0.5,
                ('model_action', 'None'): 1,          # Index of action_dropout
                ('batch_size', 'dls_action'): 1,
                ('normalize', 'dls_action'): 0,
                ('activation', 'dls_action'): 1
            },
            distributions={
                ('rand_number', 'model_action'): optuna.distributions.CategoricalDistribution(choices=list(range(1, 11))),
                ('activation', 'model_action'): optuna.distributions.CategoricalDistribution(choices=[0, 1, 2]),
                ('dropout_p', 'model_action'): optuna.distributions.FloatDistribution(low=0.0, high=1.0),
                ('model_action', 'None'): optuna.distributions.CategoricalDistribution(choices=[0, 1]),
                ('batch_size', 'dls_action'): optuna.distributions.CategoricalDistribution(choices=[0, 1]),
                ('normalize', 'dls_action'): optuna.distributions.CategoricalDistribution(choices=[0, 1]),
                ('activation', 'dls_action'): optuna.distributions.CategoricalDistribution(choices=[0, 1])
            },
            user_attrs={},  # Optional: any user-defined attributes
            system_attrs={},  # Optional: system-specific attributes
            intermediate_values={},  # Optional: store intermediate objective values
            state=optuna.trial.TrialState.COMPLETE,  # State of the trial
            values=None,  # Optional: list of objective values for multi-objective optimization
        )

        #Test action(model_action_default_actionvar) retrieved from .get_loadable_action
        action = self.model_action_var.get_loadable_action(0, frozen_trial)
        self.assertIsNotNone(action._model_loader)
        self.assertEqual(action._model_loader.keywords['rand_number'], 6)
        action_two = self.model_action_var.get_loadable_action(model_action_default_actionvar, frozen_trial)
        self.assertEqual(action, action_two)
        action_three = self.model_action_var.get_loadable_action(self.model_action_var.get_action(0), frozen_trial)
        self.assertEqual(action, action_three)
        #Test model created by action(model_action_default_actionvar)
        model = action(device=None)
        self.assertIsInstance(model, nn.Module)
        self.assertEqual(model.rand_number.item(), 6)
        Y = model(torch.randn(32, 16))
        self.assertEqual(Y.shape, (32, 10))

        #Test action(model_action_dropout_actionvar) retrieved from .get_loadable_action
        action = self.model_action_var.get_loadable_action(1, frozen_trial)
        self.assertIsNotNone(action._model_loader)
        self.assertEqual(action._model_loader.keywords['activation'], nn.ReLU)
        self.assertEqual(action._model_loader.keywords['dropout_p'], 0.5)
        action_two = self.model_action_var.get_loadable_action(model_action_dropout_actionvar, frozen_trial)
        self.assertEqual(action, action_two)
        action_three = self.model_action_var.get_loadable_action(self.model_action_var.get_action(1), frozen_trial)
        self.assertEqual(action, action_three)

        #Test model created by action(model_action_dropout_actionvar)
        model = action(device=None)
        self.assertIsInstance(model, nn.Module)
        self.assertEqual(model.activation, nn.ReLU)
        self.assertEqual(model.dropout_p, 0.5)
        Y = model(torch.randn(32, 16))
        self.assertEqual(Y.shape, (32, 10))

    def test_update_actions_model_action_var(self):
        #Prepare list_params
        list_params = [
            #rand_number: 4
            {('model_action', 'None'): 0, ('rand_number', 'model_action'): 3},
            #activation: nn.SiLU, dropout_p: 0.25.
            {('model_action', 'None'): 1, ('activation', 'model_action'): 2, ('dropout_p', 'model_action'): 0.25},
            #rand_number: 1
            {('model_action', 'None'): 0, ('rand_number', 'model_action'): 0},
            #rand_number: 1
            {('model_action', 'None'): 0, ('rand_number', 'model_action'): 0},
            #activation: nn.PReLU, dropout_p: 0.112.
            {('model_action', 'None'): 1, ('activation', 'model_action'): 1, ('dropout_p', 'model_action'): 0.112},
            #activation: nn.ReLU, dropout_p: 0.998.
            {('model_action', 'None'): 1, ('activation', 'model_action'): 0, ('dropout_p', 'model_action'): 0.998},
        ]

        for i in range(3):
            #Update actions
            self.model_action_var.update_actions([0, 1], list_params, update_state_window=i, device=torch.device('cpu'))

            #Verify updated configuration spaces.
            self.assertEqual(self.model_action_var.distribution(), [self.action_default, self.action_dropout])
            self.assertEqual(self.model_action_var.get_var(0, 'rand_number').distribution(), [1,1,4])
            self.assertEqual(self.model_action_var.get_var(1, 'activation').distribution(), [nn.ReLU,nn.PReLU,nn.SiLU])
            self.assertEqual(self.model_action_var.get_var(1, 'dropout_p').distribution(), {'low':0.112,'high':0.998})

            #action_default <- Action(model_action_default_actionvar)
            action_default = self.model_action_var.get_action(0)
            #action_dropout <- Action(model_action_dropout_actionvar)
            action_dropout = self.model_action_var.get_action(1)

            #Test the saved action states
            if i==0:
                self.assertIsNone(action_default.action_state)
                self.assertIsNone(action_dropout.action_state)
            elif i==1:
                self.assertIsNotNone(action_default.action_state)
                self.assertIsNone(action_dropout.action_state)
            else:
                self.assertIsNotNone(action_default.action_state)
                self.assertIsNotNone(action_dropout.action_state)
            action_default.action_state = None
            action_dropout.action_state = None

    def test_error_handling_in_get_loadable_action(self):
        #Test with missing parameters
        params = {('model_action', 'None'): 0}
        with self.assertRaises(KeyError):
            self.model_action_var.get_loadable_action(0, params)

    def test_error_handling_in_update_actions(self):
        #Prepare list_params with missing keys
        list_params = [
            {('model_action', 'None'): 1, ('dropout_p', 'model_action'): 0.112}
        ]
        #Should raise KeyError in .get_action() part of called INSIDE the .update_actions()
        with self.assertRaises(KeyError):
            self.model_action_var.update_actions([1], list_params, update_state_window=10, device=torch.device('cpu'))

        #Prepare list_params that only sampled model_action->action_dropout.
        list_params = [
            {('model_action', 'None'): 1, ('dropout_p', 'model_action'): 0.112, ('activation', 'model_action'): 0},
            {('model_action', 'None'): 1, ('dropout_p', 'model_action'): 1.0, ('activation', 'model_action'): 2},
        ]
        #Pass an unmatching update_indices == [0,1]. This should raise a ValueError.
        with self.assertRaises(ValueError):
            self.model_action_var.update_actions([0,1], list_params, update_state_window=10, device=torch.device('cpu'))

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

............[I 2024-10-13 21:16:39,301] A new study created in memory with name: no-name-aaf3e785-3e01-4371-886b-92bfeaea0356
.[I 2024-10-13 21:16:39,305] A new study created in memory with name: no-name-3050c0e3-1992-49f4-ae84-b22aade931b6
.[I 2024-10-13 21:16:39,308] A new study created in memory with name: no-name-999c4c11-890c-431c-b46b-4cc6c1b53047
.[I 2024-10-13 21:16:39,313] A new study created in memory with name: no-name-4e2a16b3-5679-4554-97e4-f50fa563543f
.[I 2024-10-13 21:16:39,317] A new study created in memory with name: no-name-99e8ae18-d7de-4538-bf2e-f9b38affb1be
.[I 2024-10-13 21:16:39,319] A new study created in memory with name: no-name-3edc0ce7-3821-4a2a-906b-10b6da1a3c9a
.[I 2024-10-13 21:16:39,323] A new study created in memory with name: no-name-c1fcfadc-4e52-444e-b7ae-6388b7b2d0e1
...........[I 2024-10-13 21:16:39,342] A new study created in memory with name: no-name-e93cc271-f7f9-43aa-aedd-1787bafa5c6d
..[I 2024-10-13 21:16:39,345] A new study created in memory

#Class: ConfigSpace

In [11]:
class ConfigSpace:
    """
    ConfigSpace is the user interface for building a properly-formatted configuration space, to be used by the AutoHPO process.
    (1)Allows the user to add,modify,retrieve configuration hyperparameters(whose syntax is identical to that of optuna.sample*)
    (2)Allows the user to retrieve Var/ActionVar object(s).
    """
    def __init__(self, init_config=None):
        """
        Initialize ConfigSpace.

        Parameters:
        config_space(optional): Another ConfigSpace
        """
        if init_config is not None and not isinstance(init_config, (dict, ConfigSpace)):
            raise ValueError(f"Input 'init_config' must either be None or of type in (dict, ConfigSpace) but found type(init_config): {type(init_config)}")

        self._config = {}
        if isinstance(init_config, ConfigSpace):
            self._config = init_config._config.copy()
            self.verify_syntax(False) #verify syntax. Don't raise error(s) for missing sig parameters.
            return
        #IF init_config is a dictionary, then shallow-copy it.
        init_config = {} if init_config is None else init_config.copy()

        #Now, we're guaranteed that 'init_config' is a dictionary, and self._config = {} for now.
        #(0)Tools for construction
        init_keys = set(init_config.keys())
        action_keys = {key for key, source in init_config.items() if source.get('action', False)}
        non_action_keys = {key for key in init_keys - action_keys if key[1] is None}
        sig_keys = init_keys - action_keys - non_action_keys

        #CHECK for invalid_sample method.
        invalid_sample_methods = {var_type:source for var_type, source in init_config.items() if source['sample'] not in ('categorical','int','float')}
        if invalid_sample_methods:
            raise ValueError(f"All values in 'init_config' must have 'sample' set to one of ('categorical','int','float') but found: {invalid_sample_methods}")

        #CHECK format of source(s).
        invalid_source_format = {
            var_type:source for var_type, source in init_config.items()
            if not isinstance(source, dict) or not set(source.keys()).issubset({'params','sample', 'action', 'save_state'})
        }
        if invalid_source_format:
            raise ValueError(f"All values in 'init_config' must be a dictionary with keys ['params', 'sample', 'action', 'save_state'(optional)] but following found: {invalid_source_format}")

        #(2)Transfer the info from the dictionary to ConfigSpace.
        #Firstly, define all the action parameters
        for key in action_keys:
            source = init_config[key]
            params = source['params']
            sample_method = source['sample']
            save_state = source.get('save_state', False)
            if sample_method!='categorical':
                raise ValueError(f"init_config[{key}]['sample'] must be set to 'categorical' but found: {sample_method}")
            action_var_type, belongs_to = key
            self.set_categorical(
                var_type=action_var_type,
                belongs_to=belongs_to,
                action=True,
                save_state=save_state,
                **params
            )
        ordered_keys = [key for key in non_action_keys] + [key for key in sig_keys]
        #Secondly, define all the non-action parameters and sig parameters, IN ORDER.
        for key in ordered_keys:
            source = init_config[key]
            sample_method = source['sample']
            params = source['params']
            var_type, belongs_to = key
            cls = self.set_int if sample_method == 'int' else self.set_float \
                               if sample_method == 'float' else partial(self.set_categorical, action=False)
            cls(var_type=var_type, belongs_to=belongs_to, **params)

        #(3)Perform post verification
        self.verify_syntax(True)

    def __len__(self):
        return len(self._config)

    def all_pairs(self):
        return set(self._config.keys())

    def verify_syntax(self, missing_sig_var_types_raise_error: bool):
        """
        Verify the hyperparameter configuration.

        Raises ValueError or SyntaxError depending on the point of failure.
        """
        #First and foremost, ensure that there's no overlap between non-action_var_types AND action_var_types
        overlaps = self.non_action_var_types() & self.action_var_types()
        if overlaps:
            raise ValueError(f"Non-action hyperparameters and action hyperparameters overlap: {overlaps}")
        #More checks can be added, if necessary.
        self._verify_missing_sig_var_types(missing_sig_var_types_raise_error)

    def _verify_missing_sig_var_types(self, missing_sig_var_types_raise_error: bool):
        msg = ''
        for action_var_type in self.action_var_types():
            all_potential_sig_var_types = self.all_potential_sig_var_types(action_var_type)
            missing_var_types = all_potential_sig_var_types - self.sig_var_types(action_var_type)
            #If missing_var_types exist, then add their info to the message.
            if missing_var_types:
                msg += f'Missing signature parameters in {action_var_type}: {missing_var_types} \n'

        if len(msg) and missing_sig_var_types_raise_error:
            raise ValueError(msg)
        if len(msg): #IF there's message to print, print it.
            print(msg)
        else:
            print("All good\n")

    def var_types(self) -> set:
        return {var_type for var_type, belongs_to in self._config.keys() if belongs_to is None}

    def action_var_types(self, save_state: bool = None) -> set:
        if save_state is None:
            return {var_type for (var_type, _), source in self._config.items() if source.get('action',False)}
        return {
            var_type for (var_type, _), source in self._config.items()
            if source.get('action', False) and source.get('save_state',False) == save_state
        }

    def non_action_var_types(self) -> set:
        return self.var_types() - self.action_var_types()

    def sig_var_types(self, action_var_type: str = None) -> set:
        if action_var_type is None:
            return {var_type for (var_type, belongs_to) in self._config.keys() if belongs_to is not None}
        if action_var_type not in self.action_var_types():
            raise KeyError(f"Input 'action_var_type': {action_var_type} is not in the set of registered action hyperparameters: {self.action_var_types()}")
        #Return all var_types whose belongs_to == action_var_type.
        return {var_type for (var_type, belongs_to) in self._config.keys() if belongs_to == action_var_type}

    def all_potential_sig_var_types(self, action_var_type: str) -> set:
        """
        Returns all potential signature parameters that belong to the given action_var_type.
        """
        if action_var_type not in self.action_var_types():
            raise KeyError(f"Input 'action_var_type': {action_var_type} is not in the set of registered action hyperparameters: {self.action_var_types()}")
        action_source = self.get_source(action_var_type)
        return reduce(set.union, [set(signature(func).parameters.keys()) - {'device'} for func in action_source['params']['choices']], set())

    def get_source(self, var_type: str, belongs_to: str = None) -> dict:
        config_keys = self._config.keys()
        var_types = self.var_types()
        action_var_types = self.action_var_types()
        sig_var_types = self.sig_var_types()
        if var_type not in var_types and belongs_to is None:
            raise KeyError(f"Input 'var_type': {var_type} is not in the set of registered hyperparameters: {var_types}")
        if var_type in action_var_types and belongs_to is not None:
            raise KeyError(f"Input 'var_type': {var_type} refers to an action parameter but 'belongs_to': {belongs_to} is not None")
        if belongs_to is not None and belongs_to not in action_var_types:
            raise KeyError(f"Input 'belongs_to': {belongs_to} is not in the set of registered action hyperparameters: {action_var_types}")
        if var_type in sig_var_types and (var_type, belongs_to) not in config_keys:
            all_potential_sig_var_types = self.all_potential_sig_var_types(belongs_to)
            msg = f"({var_type, belongs_to}) refers to a potential parameter in {belongs_to} which hasn't been defined yet" if var_type in all_potential_sig_var_types \
                  else f"({var_type}) does not refer to any acceptable signature parameter of {belongs_to}: {all_potential_sig_var_types}"
            raise KeyError(msg)
        if (var_type, belongs_to) not in config_keys:
            raise KeyError(f"Requested ({var_type}, {belongs_to}) does not exist in the set of all pairs stored in ConfigSpace: {config_keys}")

        return self._config[(var_type, belongs_to)]

    def get_sources(self, list_of_keys) -> list:
        """
        Get the configurations for a list of hyperparameters.

        Parameters:
        - list_of_keys (list or set): List of (parameter, belongs_to) tuples.
        """
        if not isinstance(list_of_keys, (list, set, tuple)):
            raise ValueError(f"Input 'list_of_keys' must be a list,set or a tuple but found: {type(list_of_keys)}")
        if any(not isinstance(key, tuple) for key in list_of_keys):
            raise KeyError(f"Non-tuple elements found in input 'list_of_keys': {list_of_keys}")
        return [self.get_source(*key) for key in list_of_keys]

    def get_var(self, var_type: str, belongs_to: str = None) -> Var:
        """
        Get the configuration for a specific hyperparameter.

        Parameters:
        - var_type (str): The hyperparameter name.
        - belongs_to (str, optional): The action hyperparameter this hyperparameter belongs to.
        """
        source = self.get_source(var_type, belongs_to)
        if not source.get('action', False):
            return Var(var_type, source, belongs_to)

        #Assemble 'sig_dict' to initialize Action/ModelAction objects
        action_var_type = var_type
        #Ensure that we can assemble the ActionVar, by checking if we have all the required signature parameters
        all_potential_sig_var_types = self.all_potential_sig_var_types(action_var_type)
        missing_var_types = all_potential_sig_var_types - self.sig_var_types(action_var_type)
        if missing_var_types:
            raise ValueError(f"Requested action hyperparameter: {action_var_type} cannot be assembled due to missing hyperparameters: {missing_var_types}")
        choices = source['params']['choices']
        cls = ModelAction if source.get('save_state', False) else Action
        action_choices = [
            cls(func, {var_type: Var(var_type, self.get_source(var_type, action_var_type), action_var_type) for var_type in set(signature(func).parameters.keys()) - {'device'}})
            for func in choices
        ]
        action_source = {
            'params': {'choices': action_choices},
            'sample': 'categorical'
        }
        return ActionVar(action_var_type, action_source)

    def get_action_vars(self, action_var_types=None):
        self.verify_syntax(True)
        if isinstance(action_var_types, str):
            action_var_types = [action_var_types]
        action_var_types = self.action_var_types() if action_var_types is None else set(action_var_types)
        return [self.get_var(action_var_type) for action_var_type in action_var_types]

    def get_non_action_vars(self, non_action_var_types=None):
        self.verify_syntax(True)
        if isinstance(non_action_var_types, str):
            non_action_var_types = [non_action_var_types]
        non_action_var_types = self.non_action_var_types() if non_action_var_types is None else set(non_action_var_types)
        return [self.get_var(non_action_var_type) for non_action_var_type in non_action_var_types]

    def get_sig_vars(self, list_keys: list=None):
        if list_keys is not None and not isinstance(list_keys, list):
            raise ValueError(f"Input 'list_keys' must be None or a list but found: {list_keys}")
        invaid_keys = set() if list_keys is None else {key not in self._config.keys() for key in list_keys}
        if invaid_keys:
            raise KeyError(f"Non-existent keys found in input 'list_keys': {list_keys}")

        if list_keys is None:
            return [self.get_var(*key) for key in self._config.keys() if key[1] is not None]
        return [self.get_var(*key) for key in list_keys]

    def get_vars(self, list_keys: list=None):
        if list_keys is None:
            return self.get_action_vars() + self.get_non_action_vars() + self.get_sig_vars()
        action_var_keys = [key for key in list_keys if key[1] is None and key[0] in self.action_var_types()]
        non_action_var_keys = [key for key in list_keys if key[1] is None and key[0] in self.non_action_var_types()]
        sig_var_keys = [key for key in list_keys if key not in action_var_keys+non_action_var_keys]
        invalid_keys = set(sig_var_keys) - self._config.keys()
        if invalid_keys:
            raise KeyError(f"Non-existent keys found in input 'list_keys': {invalid_keys}")
        action_var_types = list(map(lambda x:x[0], action_var_keys))
        non_action_var_types = list(map(lambda x:x[0], non_action_var_keys))
        return self.get_action_vars(action_var_types) + self.get_non_action_vars(non_action_var_types) + self.get_sig_vars(list_keys)

    def set_float(self, var_type: str, low: float, high: float, step: float = None, log: bool = False, belongs_to: str = None):
        """
        Define a float hyperparameter.

        Parameters:
        - var_type (str): The name of the hyperparameter.
        - low (float): The lower bound.
        - high (float): The upper bound.
        - step (float, optional): Step size.
        - log (bool, optional): Whether to sample logarithmically.
        - belongs_to (str, optional): The action hyperparameter this hyperparameter belongs to.
        """
        #ValueError(s).
        if any([
            not isinstance(var_type, str),
            not (belongs_to is None or isinstance(belongs_to, str)),
            not isinstance(low, (float, int)),
            not isinstance(high, (float, int)),
            not (step is None or isinstance(step, (float, int))),
            not isinstance(log, bool)
        ]):
            types = [type(x) for x in [var_type, str(belongs_to), low, high, step, log]]
            raise ValueError(f"Input var_type, belongs_to, low, high, step, log should be of types str, str/None, float/int, float/int, float/int or None, bool respectively but found {types}")
        if low > high:
            raise ValueError(f"low: {low} > high: {high}")

        #KeyError(s).
        if any('>' in x for x in [var_type, str(belongs_to)]):
            raise KeyError(f"Input (var_type: {var_type}, belongs_to: {belongs_to}) contains at least one of the following characters ['>']")
        if var_type in self.action_var_types():
            raise KeyError(f"Input (var_type: {var_type}, belongs_to: {belongs_to}) to .set_float() can only define a non-action parameter but var_type: {var_type} collides with the var_type of a registered action parameter. Namespace of action parameters and namespace of non-action parameters must be separated.")
        if belongs_to is not None and belongs_to not in self.action_var_types():
            raise KeyError(f"'belongs_to': {belongs_to} is not in the set of registered action hyperparameters: {self.action_var_types()}")
        potential_sig_var_types = self.all_potential_sig_var_types(belongs_to) if belongs_to is not None else set()
        if belongs_to is not None and var_type not in potential_sig_var_types:
            raise KeyError(f"Input (var_type: {var_type}, belongs_to: {belongs_to}) cannot define a signature parameter since var_type: {var_type} does not belong to the {belongs_to}'s signature parameters: {potential_sig_var_types}")

        params = {'low': low, 'high': high, 'log': log}
        if step is not None:
            params['step'] = step

        self._config[(var_type, belongs_to)] = {
            'params': params,
            'sample': 'float'
        }

        self.verify_syntax(False)

    def set_int(self, var_type: str, low: int, high: int, step: int = 1, log: bool = False, belongs_to: str = None):
        """
        Define an integer hyperparameter.

        Parameters:
        - var_type (str): The name of the hyperparameter.
        - low (int): The lower bound.
        - high (int): The upper bound.
        - step (int, optional): Step size.
        - log (bool, optional): Whether to sample logarithmically.
        - belongs_to (str, optional): The action hyperparameter this hyperparameter belongs to.
        """
        #ValueError(s).
        if any([
            not isinstance(var_type, str),
            not (belongs_to is None or isinstance(belongs_to, str)),
            not isinstance(low, int),
            not isinstance(high, int),
            not (step is None or isinstance(step, int)),
            not isinstance(log, bool)
        ]):
            types = [type(x) for x in [var_type, str(belongs_to), low, high, step, log]]
            raise ValueError(f"Input var_type, belongs_to, low, high, step, log should be of types str, str/None, float/int, float/int, float/int or None, bool respectively but found {types}")
        if low > high:
            raise ValueError(f"low: {low} > high: {high}")

        #KeyError(s).
        if any('>' in x for x in [var_type, str(belongs_to)]):
            raise KeyError(f"Input (var_type: {var_type}, belongs_to: {belongs_to}) contains at least one of the following characters ['>']")
        if var_type in self.action_var_types():
            raise KeyError(f"Input (var_type: {var_type}, belongs_to: {belongs_to}) to .set_int() can only define a non-action parameter but var_type: {var_type} collides with the var_type of a registered action parameter. Namespace of action parameters and namespace of non-action parameters must be separated.")
        if belongs_to is not None and belongs_to not in self.action_var_types():
            raise KeyError(f"'belongs_to': {belongs_to} is not in the set of registered action hyperparameters: {self.action_var_types()}")
        potential_sig_var_types = self.all_potential_sig_var_types(belongs_to) if belongs_to is not None else set()
        if belongs_to is not None and var_type not in potential_sig_var_types:
            raise KeyError(f"Input (var_type: {var_type}, belongs_to: {belongs_to}) cannot define a signature parameter since var_type: {var_type} does not belong to the {belongs_to}'s signature parameters: {potential_sig_var_types}")

        params = {'low': low, 'high': high, 'log': log, 'step': step}

        self._config[(var_type, belongs_to)] = {
            'params': params,
            'sample': 'int'
        }

        self.verify_syntax(False)

    def set_categorical(self, var_type: str, choices: list, belongs_to: str = None, action: bool = False, save_state: bool = False):
        """
        Define a categorical hyperparameter.

        Parameters:
        - var_type (str): The name of the hyperparameter.
        - choices (list): List of possible values.
        - belongs_to (str, optional): The action hyperparameter this hyperparameter belongs to.
        - action (bool, optional): Whether this hyperparameter is an action hyperparameter.
        - save_state (bool, optional): Whether to save the state of the hyperparameter.
        """
        #'save_state' is ONLY meaningful when action=True.
        save_state = save_state if action else False

        #ValueError(s).
        if any([
            not isinstance(var_type, str),
            not (belongs_to is None or isinstance(belongs_to, str)),
            not isinstance(action, bool),
            not isinstance(save_state, bool)
        ]):
            types = [type(x) for x in [var_type, str(belongs_to), action, save_state]]
            raise ValueError(f"Input var_type, belongs_to, action, save_state should be of types str, str/None, bool, bool respectively but found: {types}")
        if not isinstance(choices, list):
            raise ValueError(f"Input 'choices' must be a list but found: {type(choices)}")
        if action and any(not callable(choice) for choice in choices):
            raise ValueError(f"Non-callable elements found in input choices: {choices}")

        #KeyError(s).
        if any('>' in x for x in [var_type, str(belongs_to)]):
            raise KeyError(f"Input (var_type: {var_type}, belongs_to: {belongs_to}) contains at least one of the following characters ['>']")
        if action and belongs_to is not None:
            raise KeyError(f"Input 'belongs_to' must be None if 'action' is True but found: {belongs_to}")
        if belongs_to is not None and belongs_to not in self.action_var_types():
            raise KeyError(f"'belongs_to': {belongs_to} is not in the set of registered action hyperparameters: {self.action_var_types()}")
        source = self._config.get((var_type, belongs_to), None)
        if belongs_to is None and source is not None and action != source['action']:
            target, opposite = ('action','non-action') if action else ('non-action', 'action')
            raise KeyError(f"Input (var_type: {var_type}, belongs_to: {belongs_to}) defines a {target} parameter but var_type: {var_type} collides with the var_type of a registered {opposite} parameter. Overlapping names between {target} parameters and {opposite} parameters is not allowed.")
        potential_sig_var_types = self.all_potential_sig_var_types(belongs_to) if belongs_to is not None else set()
        if belongs_to is not None and var_type not in potential_sig_var_types:
            raise KeyError(f"Input (var_type: {var_type}, belongs_to: {belongs_to}) cannot define a signature parameter since var_type: {var_type} does not belong to the {belongs_to}'s signature: {potential_sig_var_types}")

        if not action or source is None:
            self._config[(var_type, belongs_to)] = {
                'params': {'choices': choices},
                'sample': 'categorical',
                'action': action,
                'save_state': save_state
            }
        else:
            action_var_type = var_type
            #Overriding the existing (action_var_type,belongs_to) has been requested:
            #WARNING: save_state will simply be overwritten.
            #STEP1: POP all the related parameters
            popped = self.pop(action_var_type)
            #STEP2: Now, given that we're guaranteed we DO NOT have (var_type+'--action',None) in self._config,
            #       we do self.set_categorical(var_type,..rest)
            self.set_categorical(var_type=action_var_type, choices=choices, action=action, save_state=save_state)
            #STEP3: Last but not least, selectively 'inherit' the information by looking at the potential sig_var_types needed for the new action parameter
            all_potential_sig_var_keys = {
                (sig_var_type, action_var_type) for sig_var_type in self.all_potential_sig_var_types(action_var_type)
            }
            inherit_sig_var_keys = all_potential_sig_var_keys & set(popped.keys())
            #Inherit via internal method, which enables double-checking.
            for key in inherit_sig_var_keys:
                source = popped[key]
                sig_var_type, _ = key
                initializer = {'var_type': sig_var_type, 'belongs_to': action_var_type} | source['params']
                cls = self.set_float if source['sample']=='float' else \
                      self.set_int if source['sample']=='int' else partial(self.set_categorical, action=False)
                cls(**initializer)

        self.verify_syntax(False)

    def pop(self, var_type: str, belongs_to: str=None):
        """
        Delete a hyperparameter and return its source(s). Multiple sources may be returned if a deleted hyperparameter is an action parameter.

        Parameters:
        - var_type (str): The name of the hyperparameter.
        - belongs_to (str, optional): The action hyperparameter this hyperparameter belongs to.

        Returns:
        Dictionary(or dictionaries if what you deleted is an action_var_type) of mapping (var_type->source) for all deleted var_types.

        NOTE: If you delete an Action parameter, Then ALL of its signature parameters will also be deleted.
        """
        #If you can successfully retrieve a source from the provided pair, it means we have a valid pair
        source = self.get_source(var_type, belongs_to)
        popped = dict()
        sig_var_types = self.sig_var_types(var_type) if source.get('action', False) else set()
        if sig_var_types:
            #Note that we avoid recursion by directly popping both sig_var_types, and the action_var_type.
            popped |= {(sig_var_type, var_type):self._config.pop((sig_var_type, var_type)) for sig_var_type in sig_var_types}
            return {(var_type,belongs_to):self._config.pop((var_type, belongs_to))} | popped

        popped = {(var_type, belongs_to): self._config.pop((var_type, belongs_to))}
        #If what was popped was a signature var_type, then run self.verify_syntax(False) to print out message about what the owner action_var_type will now be missing.
        if belongs_to is not None:
            self.verify_syntax(False)
        #Return the resultant value.
        return popped

#Test: ConfigSpace

In [12]:
if __name__ == '__main__':
    def dummy_action1(param1, param2):
        pass

    def dummy_action2(param3, param4, device=None):
        pass

    def dummy_action3(param5, param6):
        pass

    def dummy_action_evens(param2, param4, param6, device=None):
        pass

    def dummy_action_odds(param1, param3, param5, device=None):
        pass

    def dummy_action_combined(param1, param2, param3, param4, param5, param6):
        pass

    def non_callable():
        pass

class TestConfigSpace(unittest.TestCase):
    def setUp(self):
        self.cs = ConfigSpace()

    def test_set_float(self):
        self.cs.set_float('learning_rate', low=0.001, high=0.01)
        expected = {
            ('learning_rate', None): {
                'params': {'low': 0.001, 'high': 0.01, 'log': False},
                'sample': 'float'
            }
        }
        self.assertEqual(self.cs._config, expected)

        self.assertEqual(
            self.cs.get_source('learning_rate'),
            expected[('learning_rate', None)]
        )

    def test_set_int(self):
        self.cs.set_int('num_layers', low=1, high=5)
        expected = {
            ('num_layers', None): {
                'params': {'low': 1, 'high': 5, 'log': False, 'step': 1},
                'sample': 'int'
            }
        }
        self.assertEqual(self.cs._config, expected)

        self.assertEqual(
            self.cs.get_source('num_layers'),
            expected[('num_layers', None)]
        )

    def test_set_categorical(self):
        self.cs.set_categorical('activation', choices=[nn.ReLU, nn.Tanh, nn.Sigmoid])
        expected = {
            ('activation', None): {
                'params': {'choices': [nn.ReLU, nn.Tanh, nn.Sigmoid]},
                'sample': 'categorical',
                'action': False,
                'save_state': False
            }
        }
        self.assertEqual(
            self.cs._config,
            expected,
            msg = f"First: {self.cs._config} != \nexpected: {expected[('activation', None)]}"
        )

        self.assertEqual(
            self.cs.get_source('activation'),
            expected[('activation', None)],
            msg = f"First: {self.cs.get_source('activation')} != \nexpected: {expected[('activation', None)]}"
        )

    def test_set_action(self):
        self.cs.set_categorical('optimizer', choices=[dummy_action1, dummy_action2], action=True)
        expected = {
            ('optimizer', None): {
                'params': {'choices': [dummy_action1, dummy_action2]},
                'sample': 'categorical',
                'action': True,      # Include the 'action' key
                'save_state': False
            }
        }
        self.assertEqual(self.cs._config, expected)

        self.assertEqual(
            self.cs.get_source('optimizer'),
            expected[('optimizer', None)]
        )

        for i in range(1,5):
            self.cs.set_float(f'param{i}', low=0.0, high=1.0, belongs_to='optimizer')
        sig_dict_dummy_action1 = {f'param{i}': Var(f'param{i}', {'params': {'low': 0.0, 'high': 1.0, 'log': False}, 'sample': 'float'}, 'optimizer') for i in range(1,3)}
        sig_dict_dummy_action2 = {f'param{i}': Var(f'param{i}', {'params': {'low': 0.0, 'high': 1.0, 'log': False}, 'sample': 'float'}, 'optimizer') for i in range(3,5)}
        self.assertEqual(
            self.cs.get_var('optimizer'),
            ActionVar(
                'optimizer',
                 { 'params': {'choices': [Action(dummy_action1, sig_dict_dummy_action1), Action(dummy_action2, sig_dict_dummy_action2)]},
                   'sample': 'categorical'}
            )
        )

        self.assertEqual(
            self.cs.action_var_types(True),
            set(),
            msg=f"First: {self.cs.action_var_types(True)} != \nexpected: {set()}"
        )
        self.assertEqual(
            self.cs.action_var_types(False),
            {'optimizer'},
            msg=f"First: {self.cs.action_var_types(False)} != \nexpected: {set(['optimizer'])}"
        )
        self.assertEqual(
            self.cs.action_var_types(),
            {'optimizer'},
            msg=f"First: {self.cs.action_var_types()} != \nexpected: {set(['optimizer'])}"
        )

        #DO: Another version; where the only difference is save_state == True
        #The should override the old one. Additionally, note that ALL sig parameters SHOULD be directly transferred
        self.cs.set_categorical('optimizer', choices=[dummy_action1, dummy_action2], action=True, save_state=True)
        expected = {
            ('optimizer', None): {
                'params': {'choices': [dummy_action1, dummy_action2]},
                'sample': 'categorical',
                'action': True,
                'save_state': True
            }
        }
        self.assertEqual(
            self.cs.get_source('optimizer'),
            expected[('optimizer', None)]
        )
        #CHECK that all the sig_var_types have been transferred correctly.
        self.assertEqual(
            self.cs.sig_var_types('optimizer'),
            {'param1', 'param2', 'param3', 'param4'},
            msg=f"First: {self.cs.sig_var_types('optimizer')} != \nexpected: {set(['param1', 'param2', 'param3', 'param4'])}"
        )

        self.assertEqual(
            self.cs.get_var('optimizer'),
            ActionVar(
                'optimizer',
                 { 'params': {'choices': [ModelAction(dummy_action1, sig_dict_dummy_action1), ModelAction(dummy_action2, sig_dict_dummy_action2)]},
                   'sample': 'categorical'}
            )
        )

        self.assertEqual(
            self.cs.action_var_types(True),
            {'optimizer'}
        )
        self.assertEqual(
            self.cs.action_var_types(False),
            set()
        )
        self.assertEqual(
            self.cs.action_var_types(),
            {'optimizer'}
        )

    def test_categorization(self):
        #action_var_types
        self.cs.set_categorical('optimizer', choices=[dummy_action1, dummy_action2], action=True)

        #non_action_var_types
        self.cs.set_int('num_layers', low=1, high=5)
        self.cs.set_float('learning_rate', low=0.001, high=0.01)
        self.cs.set_categorical('activation', choices=[nn.ReLU, nn.Tanh, nn.SiLU])

        #sig_var_types
        self.cs.set_float('param1', low=0.0, high=1.0, belongs_to='optimizer')
        self.cs.set_float('param2', low=0.0, high=1.0, belongs_to='optimizer')
        self.cs.set_float('param3', low=0.0, high=1.0, belongs_to='optimizer')
        self.cs.set_float('param4', low=0.0, high=1.0, belongs_to='optimizer')

        self.assertEqual(
            self.cs.action_var_types(),
            self.cs.action_var_types(False)
        )
        self.assertEqual(
            self.cs.non_action_var_types(),
            {'num_layers', 'learning_rate', 'activation'}
        )
        self.assertEqual(
            self.cs.sig_var_types('optimizer'),
            {'param1', 'param2', 'param3', 'param4'}
        )

        with self.assertRaises(KeyError):
            self.cs.set_float('param5', low=0.0, high=1.0, belongs_to='non_existent_action')
        with self.assertRaises(KeyError):
            #Should raise KeyError since 'non_existent_param' does NOT exist in signature(s) of ActionVar category 'optimizer'
            self.cs.set_int('non_existent_param', low=0, high=1, belongs_to='optimizer')

        #Get rid of the 'param1' from 'optimizer'
        popped = self.cs.pop('param1', 'optimizer')
        self.assertEqual(
            popped,
            {
                ('param1', 'optimizer'): {
                    'params': {'low': 0.0, 'high': 1.0, 'log': False},
                    'sample': 'float'
                }
            }
        )

        #Verify that the action parameter requires signature parameters now.(The missing one; param1)
        with self.assertRaises(ValueError):
            self.cs.verify_syntax(True)

    def test_add_signature_parameters(self):
        self.cs.set_categorical('optimizer', choices=[dummy_action1, dummy_action2], action=True, save_state=True)
        self.cs.set_float('param1', low=0.0, high=1.0, belongs_to='optimizer')
        self.cs.set_float('param2', low=0.0, high=1.0, belongs_to='optimizer')
        self.cs.set_float('param3', low=0.0, high=1.0, belongs_to='optimizer')
        self.cs.set_float('param4', low=0.0, high=1.0, belongs_to='optimizer')
        self.assertEqual(
            self.cs.action_var_types(True),
            self.cs.action_var_types()
        )

        #Now verify should pass
        self.cs.verify_syntax(False)

        #Delete the 'optimizer'. This should delete all the other 'param1',...,'param4' as well.
        popped = self.cs.pop('optimizer')
        expected = {
                ('optimizer', None): {
                    'params': {'choices': [dummy_action1, dummy_action2]},
                    'sample': 'categorical',
                    'action': True,
                    'save_state': True
                },
                ('param1', 'optimizer'): {
                    'params': {'low': 0.0, 'high': 1.0, 'log': False},
                    'sample': 'float'
                },
                ('param2', 'optimizer'): {
                    'params': {'low': 0.0, 'high': 1.0, 'log': False},
                    'sample': 'float'
                },
                ('param3', 'optimizer'): {
                    'params': {'low': 0.0, 'high': 1.0, 'log': False},
                    'sample': 'float'
                },
                ('param4', 'optimizer'): {
                    'params': {'low': 0.0, 'high': 1.0, 'log': False},
                    'sample': 'float'
                }
        }
        self.assertEqual(
            popped,
            expected,
            msg = f"popped: {popped} != \nexpected: {expected}"
        )

    def test_invalid_var_type_name(self):
        with self.assertRaises(KeyError):
            self.cs.set_float('learning>rate', low=0.001, high=0.01)

    def test_invalid_belongs_to(self):
        with self.assertRaises(KeyError):
            self.cs.set_float('param', low=0.0, high=1.0, belongs_to='non_existent_action')

    def test_non_callable_action_choice(self):
        with self.assertRaises(ValueError): #action=True means that all inputs in 'choices' must be callable.
            self.cs.set_categorical('optimizer', choices=[1,2], action=True)

    def test_set_categorical_with_invalid_choices(self):
        with self.assertRaises(ValueError):
            self.cs.set_categorical('optimizer', choices='not_a_list')

    def test_set_float_with_invalid_types(self):
        with self.assertRaises(ValueError):
            self.cs.set_float('learning_rate', low='0.001', high=0.01)

    def test_set_int_with_invalid_types(self):
        with self.assertRaises(ValueError):
            self.cs.set_int('num_layers', low=1, high='5')

    def test_get_non_action_var(self):
        self.cs.set_float('learning_rate', low=0.001, high=0.01)
        var = self.cs.get_var('learning_rate')
        self.assertIsInstance(var, Var)
        expected_var = Var('learning_rate', {'params': {'low': 0.001, 'high': 0.01, 'log': False}, 'sample': 'float'}, None)
        self.assertEqual(var, expected_var)

    def test_get_action_var(self):
        self.cs.set_categorical('optimizer', choices=[dummy_action1], action=True)
        self.cs.set_float('param1', low=0.0, high=1.0, belongs_to='optimizer')
        self.cs.set_float('param2', low=0.0, high=1.0, belongs_to='optimizer')
        action_var = self.cs.get_var('optimizer')
        self.assertIsInstance(action_var, ActionVar)
        expected_param1_var = Var('param1', {'params': {'low': 0.0, 'high': 1.0, 'log': False}, 'sample': 'float'}, 'optimizer')
        expected_param2_var = Var('param2', {'params': {'low': 0.0, 'high': 1.0, 'log': False}, 'sample': 'float'}, 'optimizer')
        expected_action_var = ActionVar(
            'optimizer',
             {
                 'params': {'choices': [Action(dummy_action1, {expected_param1_var.var_type: expected_param1_var, expected_param2_var.var_type: expected_param2_var})]},
                 'sample': 'categorical'
             }
        )
        self.assertEqual(
            action_var,
            expected_action_var,
            msg=f"action_var: {action_var} != \nexpected_action_var: {expected_action_var}"
        )

    def test_get_action_vars(self):
        self.cs.set_categorical('optimizer', choices=[dummy_action1], action=True)
        self.cs.set_float('param1', low=0.0, high=1.0, belongs_to='optimizer')
        #Attempt to retrieve optimizer w/o filling in all the required signature parameters
        with self.assertRaises(ValueError): #Will internally run self._verify(), NOT self._verify(True,True)
            self.cs.get_action_vars()

        self.cs.set_float('param2', low=0.0, high=1.0, belongs_to='optimizer')
        self.cs.verify_syntax(False)

        #This time, you'll be missing param1
        self.cs.pop('param1', 'optimizer')
        with self.assertRaises(ValueError):
            self.cs.get_action_vars()

        #Add it again.
        self.cs.set_float('param1', low=0.0, high=1.0, belongs_to='optimizer')
        action_vars = self.cs.get_action_vars() #Should work this time.

        #Verify the number of action_vars.
        self.assertEqual(len(action_vars), 1)

        #Verify that it's as we expected.
        action_var = action_vars[0]
        self.assertIsInstance(action_var, ActionVar)
        expected_param1_var = Var('param1', {'params': {'low': 0.0, 'high': 1.0, 'log': False}, 'sample': 'float'}, 'optimizer')
        expected_param2_var = Var('param2', {'params': {'low': 0.0, 'high': 1.0, 'log': False}, 'sample': 'float'}, 'optimizer')
        expected_action_var = ActionVar(
            'optimizer',
             {
                 'params': {'choices': [Action(dummy_action1, {expected_param1_var.var_type: expected_param1_var, expected_param2_var.var_type: expected_param2_var})]},
                 'sample': 'categorical'
             }
        )
        self.assertEqual(
            action_var,
            expected_action_var,
            msg=f"action_var: {action_var} != \nexpected_action_var: {expected_action_var}"
        )

    def test_get_non_action_vars(self):
        self.cs.set_float('learning_rate', low=0.001, high=0.01)
        self.cs.set_int('num_layers', low=1, high=5)
        vars = self.cs.get_non_action_vars()
        self.assertEqual(len(vars), 2)

        #Check that created Var objects are as we expected.
        expected_lr_var = Var('learning_rate', {'params': {'low': 0.001, 'high': 0.01, 'log': False}, 'sample': 'float'}, None)
        expected_num_layers_var = Var('num_layers', {'params': {'low': 1, 'high': 5, 'log': False, 'step': 1}, 'sample': 'int'}, None)
        expected_dict = {expected_lr_var.var_type: expected_lr_var, expected_num_layers_var.var_type: expected_num_layers_var}
        for var in vars:
            var_type = var.var_type
            expected = expected_dict[var_type]
            self.assertEqual(
                var,
                expected,
                msg=f"var_type: {var_type} maps to --> var: {var} != /nexpected: {expected}"
            )
        #Do the same, but for signature parameters. Don't forget to add the 'optimizer' first.
        self.cs.set_categorical('optimizer', choices=[dummy_action1], action=True)
        self.cs.set_float('param1', low=0.0, high=1.0, belongs_to='optimizer')

        with self.assertRaises(KeyError): #'param4' not in optimizer signatures
            self.cs.set_float('param4', low=0.0, high=1.0, belongs_to='optimizer')

        self.cs.set_float('param2', low=0.0, high=1.0, belongs_to='optimizer')
        sig_vars = self.cs.get_sig_vars()
        self.assertEqual(len(sig_vars), 2)
        expected_param1_var = Var('param1', {'params': {'low': 0.0, 'high': 1.0, 'log': False}, 'sample': 'float'}, 'optimizer')
        expected_param2_var = Var('param2', {'params': {'low': 0.0, 'high': 1.0, 'log': False}, 'sample': 'float'}, 'optimizer')
        expected_dict = {expected_param1_var.var_type:expected_param1_var,
                         expected_param2_var.var_type:expected_param2_var}
        for sig_var in sig_vars:
            sig_var_type = sig_var.var_type
            expected = expected_dict[sig_var_type]
            self.assertEqual(
                sig_var,
                expected,
                msg=f"sig_var_type: {sig_var_type} maps to --> sig_var: {sig_var} != /nexpected: {expected}"
            )

    def test_post_set_verify(self):
        #Just print what you need out.
        self.cs.set_categorical('optimizer', choices=[dummy_action1], action=True)

        #Should NOT raise KeyError, overlap between non-action_var_types and sig_var_types is ALLOWED.
        self.cs.set_float('param1', low=0.0, high=1.0)
        self.cs.set_float('param1', low=0.0, high=1.0, belongs_to='optimizer')
        self.cs.set_float('param2', low=0.0, high=1.0, belongs_to='optimizer')
        self.cs.verify_syntax(False)

        #Check sig_var_types
        sig_var_types = self.cs.sig_var_types()
        self.assertEqual(
            sig_var_types,
            {'param1', 'param2'}
        )

        #Check for action_var_types
        action_var_types = self.cs.action_var_types()
        self.assertEqual(
            action_var_types,
            {'optimizer'}
        )

        #Check for non_action_var_types
        non_action_var_types = self.cs.non_action_var_types()
        self.assertEqual(
            non_action_var_types,
            {'param1'}
        )

    def test_duplicate_hyperparameter(self):
        self.cs.set_float('learning_rate', low=0.001, high=0.01)
        self.cs.set_float('learning_rate', low=0.0001, high=0.1) #overriding is ok!
        self.assertTrue(True)

    def test_sig_var_types_method(self):
        self.cs.set_categorical('optimizer', choices=[dummy_action1], action=True)
        self.cs.set_float('param1', low=0.0, high=1.0, belongs_to='optimizer')
        self.cs.set_float('param2', low=0.0, high=1.0, belongs_to='optimizer')
        self.cs.set_categorical('optimizer2', choices=[dummy_action2], action=True)

        with self.assertRaises(KeyError):
            self.cs.set_float('param3', low=0.0, high=1.0, belongs_to='optimizer') #ONLY param1,param2 allowed
        with self.assertRaises(KeyError):
            self.cs.set_float('param4', low=0.0, high=1.0, belongs_to='optimizer') #ONLY param1,param2 allowed

        self.cs.set_float('param3', low=0.0, high=1.0, belongs_to='optimizer2')
        self.cs.set_float('param4', low=0.0, high=1.0, belongs_to='optimizer2')

        sig_var_types = self.cs.sig_var_types('optimizer')
        self.assertEqual(sig_var_types, {'param1', 'param2'})

        sig_var_types = self.cs.sig_var_types('optimizer2')
        self.assertEqual(sig_var_types, {'param3', 'param4'})

        sig_var_types = self.cs.sig_var_types()
        self.assertEqual(sig_var_types, {'param1', 'param2','param3', 'param4'})

    def test_get_source_invalid_var_type(self):
        with self.assertRaises(KeyError):
            self.cs.get_source('nonexistent_var_type')

    def test_get_source_invalid_belongs_to(self):
        self.cs.set_categorical('optimizer', choices=[dummy_action1], action=True)
        with self.assertRaises(KeyError):
            self.cs.get_source('param1', belongs_to='invalid_action')

    def test_set_action_with_belongs_to(self):
        self.cs.set_categorical('optimizer', choices=[dummy_action1], action=True)
        with self.assertRaises(KeyError):
            self.cs.set_categorical('optimizer2', choices=[dummy_action2], action=True, belongs_to='optimizer')

    def test_set_float_invalid_var_type_name(self):
        with self.assertRaises(KeyError):
            self.cs.set_float('learning>rate', low=0.001, high=0.01)

    def test_set_int_invalid_belongs_to_name(self):
        with self.assertRaises(KeyError):
            self.cs.set_int('num_layers', low=1, high=5, belongs_to='invalid>action')

    def test_set_categorical_invalid_action_choice(self):
        with self.assertRaises(ValueError):
            self.cs.set_categorical('optimizer', choices=[123], action=True)

    def test_set_float_invalid_range(self):
        with self.assertRaises(ValueError):
            self.cs.set_float('learning_rate', low=0.1, high=0.001)

    def test_get_var_invalid_var_type(self):
        with self.assertRaises(KeyError):
            self.cs.get_var('nonexistent_var_type')

    def test_get_var_invalid_belongs_to(self):
        with self.assertRaises(KeyError):
            self.cs.get_var('param1', belongs_to='nonexistent_action')

    def test_set_categorical_with_invalid_action_flag(self):
        with self.assertRaises(ValueError):
            self.cs.set_categorical('optimizer', choices=[dummy_action1], action='not_bool')

    def test_set_float_with_invalid_step(self):
        with self.assertRaises(ValueError):
            self.cs.set_float('learning_rate', low=0.001, high=0.01, step='not_float')

    def test_set_int_with_invalid_step(self):
        with self.assertRaises(ValueError):
            self.cs.set_int('num_layers', low=1, high=5, step='not_int')

    def test_set_categorical_with_invalid_choices_type(self):
        with self.assertRaises(ValueError):
            self.cs.set_categorical('activation', choices='not_a_list')

    def test_action_param_with_invalid_choices(self):
        with self.assertRaises(ValueError):
            self.cs.set_categorical('optimizer', choices=[123, 'string'], action=True)

    def test_verify_missing_signature_parameters(self):
        self.cs.set_categorical('optimizer', choices=[dummy_action1], action=True)
        #Missing signature parameters
        with self.assertRaises(ValueError):
            self.cs.verify_syntax(True)

    def test_sig_var_types_invalid_action_var_type(self):
        with self.assertRaises(KeyError):
            self.cs.sig_var_types('nonexistent_action')

    def test_get_var_missing_signature_parameters(self):
        self.cs.set_categorical('optimizer', choices=[dummy_action1], action=True)
        with self.assertRaises(ValueError):
            self.cs.get_var('optimizer')

    def test_get_sources_with_invalid_input(self):
        with self.assertRaises(ValueError):
            self.cs.get_sources('not_a_list')

    def test_get_sources_with_non_tuple_elements(self):
        with self.assertRaises(KeyError):
            self.cs.get_sources(['not_a_tuple'])

    def test_set_float_with_invalid_log_type(self):
        with self.assertRaises(ValueError):
            self.cs.set_float('learning_rate', low=0.001, high=0.01, log='not_bool')

    def test_set_int_with_invalid_log_type(self):
        with self.assertRaises(ValueError):
            self.cs.set_int('num_layers', low=1, high=5, log='not_bool')

    def test_set_float_with_invalid_param_types(self):
        with self.assertRaises(ValueError):
            self.cs.set_float(123, low=0.001, high=0.01)

    def test_set_int_with_invalid_param_types(self):
        with self.assertRaises(ValueError):
            self.cs.set_int(123, low=1, high=5)

    def test_set_categorical_with_invalid_param_types(self):
        with self.assertRaises(ValueError):
            self.cs.set_categorical(123, choices=[1, 2, 3])

    def test_set_float_with_belongs_to_none_when_required(self):
        self.cs.set_categorical('optimizer', choices=[dummy_action1], action=True)
        #Overlap is allowed between sig_var_types and the rest of the spaces
        #(It only needs to be prevented between action, non-action name spaces)
        self.cs.set_float('param1', low=0.0, high=1.0, belongs_to=None)

    def test_set_float_with_belongs_to_not_none_when_not_required(self):
        with self.assertRaises(KeyError):
            self.cs.set_float('learning_rate', low=0.001, high=0.01, belongs_to='optimizer')

    def test_set_categorical_action_with_belongs_to_not_none(self):
        with self.assertRaises(KeyError):
            self.cs.set_categorical('optimizer', choices=[dummy_action1], action=True, belongs_to='something')
        self.cs.set_categorical('optimizer', choices=[dummy_action1], action=True)
        #Should raise KeyError because 'some_action' is not a registered action
        with self.assertRaises(KeyError):
            self.cs.set_categorical('optimizer2', choices=['adam', 'sgd'], belongs_to='some_action')

    def test_get_var_with_invalid_belongs_to(self):
        self.cs.set_float('learning_rate', low=0.001, high=0.01)
        with self.assertRaises(KeyError):
            self.cs.get_var('learning_rate', belongs_to='some_action')

    def test_get_action_vars_with_invalid_input(self):
        with self.assertRaises(KeyError):
            self.cs.get_action_vars(action_var_types='nonexistent_action')

    def test_get_non_action_vars_with_invalid_input(self):
        with self.assertRaises(KeyError):
            self.cs.get_non_action_vars(non_action_var_types='nonexistent_var')

    #****STILL UNDER CONSTRUCTION*****
    def test_overriding_action_parameter(self):
        #Add 2,6th params to 'optimizer_evens'
        self.cs.set_categorical('optimizer_evens', choices=[dummy_action_evens], action=True)
        self.cs.set_float('param2', low=0.0, high=1.0, belongs_to='optimizer_evens')
        with self.assertRaises(KeyError): #since 1 is an odd number
            self.cs.set_float('param1', low=0.0, high=1.0, belongs_to='optimizer_evens')
        self.cs.set_float('param6', low=0.0, high=1.0, belongs_to='optimizer_evens')
        self.assertEqual(
            self.cs.sig_var_types('optimizer_evens'),
            {'param2', 'param6'}
        )
        self.assertEqual(
            self.cs.sig_var_types(),
            {'param2', 'param6'}
        )

        #Override 'optimizer_evens', with dummy_action_odds.
        #Note that this will cause ALL the sig_var_types to be deleted, with none inherited resultantly.
        self.cs.set_categorical('optimizer_evens', choices=[dummy_action_odds], action=True)
        self.assertEqual(
            self.cs.sig_var_types('optimizer_evens'),
            set() #since none of the potential_sig_var_types(={'param1','param3','param5'}) were in the 'cs'.
        )
        self.assertEqual(
            self.cs.sig_var_types(),
            set()
        )

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

..............................................................[I 2024-10-13 21:16:49,609] A new study created in memory with name: no-name-0eb7c602-dbd8-4676-afc8-5a8cdd2e7a8c
.[I 2024-10-13 21:16:49,615] A new study created in memory with name: no-name-6a980606-8b9a-4ec1-8cda-8ef8edcaffa0
.[I 2024-10-13 21:16:49,618] A new study created in memory with name: no-name-8ccbb54c-f6d4-4d4d-9a14-b0ed66e5a6d9
.[I 2024-10-13 21:16:49,624] A new study created in memory with name: no-name-345235a4-0f24-4efc-a899-43b2d8efabfc
.[I 2024-10-13 21:16:49,629] A new study created in memory with name: no-name-982abb23-3d90-4287-ad6b-725cccf0f50b
.[I 2024-10-13 21:16:49,631] A new study created in memory with name: no-name-2f58bec4-0268-4caf-bf68-b84090169328
.[I 2024-10-13 21:16:49,634] A new study created in memory with name: no-name-0d3f8754-b170-4e2b-bac9-87b8d7ee944b
...........[I 2024-10-13 21:16:49,654] A new study created in memory with name: no-name-9b19e5be-5dfd-4620-a8dc-23e7e8b51689
..[I 2024

All good

All good

Missing signature parameters in optimizer: {'param3', 'param1', 'param4', 'param2'} 

Missing signature parameters in optimizer: {'param3', 'param4', 'param2'} 

Missing signature parameters in optimizer: {'param3', 'param4'} 

Missing signature parameters in optimizer: {'param4'} 

All good

All good

All good

Missing signature parameters in optimizer: {'param3', 'param1', 'param4', 'param2'} 

Missing signature parameters in optimizer: {'param3', 'param1', 'param4', 'param2'} 

Missing signature parameters in optimizer: {'param3', 'param1', 'param4', 'param2'} 

Missing signature parameters in optimizer: {'param3', 'param1', 'param4', 'param2'} 

Missing signature parameters in optimizer: {'param3', 'param4', 'param2'} 

Missing signature parameters in optimizer: {'param3', 'param4'} 

Missing signature parameters in optimizer: {'param4'} 

All good

Missing signature parameters in optimizer: {'param1'} 

All good

All good

All good

All good

Missing signature 

#Class: SampleCallVisitor, Objective, UserObjective

In [13]:
#Define a custom AST NodeVisitor to collect var_types
class SampleCallVisitor(ast.NodeVisitor):
    def __init__(self):
        self.user_requests = set()
        self.user_requests_with_defaults = set()

    def visit_Call(self, node):
        if isinstance(node.func, ast.Attribute) and node.func.attr == 'sample' and \
           isinstance(node.func.value, ast.Attribute) and node.func.value.attr == 'self':
            target = node.func.value.value
            msg = target.id if isinstance(target, ast.Name) else target.attr
            raise ValueError(f"Following code sippet '...{target}.self.sample' is not allowed. 'self' keyword cannot be used for attribute access")

        if isinstance(node.func, ast.Attribute) and node.func.attr == 'sample' and \
           isinstance(node.func.value, ast.Name) and node.func.value.id == 'self':
            if not node.args:
                raise ValueError(f"Call self.sample with no argument detected. self.sample must recieve a string argument 'var_type' and an optional literal argument 'default_val'.")
            if len(node.args) > 2:
                raise ValueError(f"Call self.sample with more than 2 arguments detected. self.sample must recieve a string argument 'var_type' and an optional literal argument 'default_val'.")

            var_type_node, default_val = node.args[0], None if len(node.args)==1 else node.args[1]
            if not isinstance(var_type_node, ast.Constant) or not isinstance(var_type_node.value, str):
                raise ValueError(f"Call self.sample with non-string literal argument: '{var_type_node}' detected. self.sample must recieve a string argument 'var_type' and an optional literal argument 'default_val'.")

            #We only check the accuracy of the node IFF 'default_val' is None or literal(ast.Constant)
            #The rest of the cases will be checked during runtime.
            if default_val is None or isinstance(default_val, ast.Constant):
                default_val = default_val.value if default_val else None
                cls = self.user_requests if default_val is None else self.user_requests_with_defaults
                cls.add((var_type_node.value, default_val))

        #Continue walking the tree
        self.generic_visit(node)

In [14]:
logging.basicConfig(level=logging.DEBUG)

class Objective(ABC):
    """
    class Objective.
    User should subclass it and overwrite its 'objective'.
    .static_verification(ConfigSpace) <- .verify_objective
    .dynamic_verification(ConfigSpace) <- New function, utilizes context manager & Mock & MagicMock to verify objective
                                          dynamically.
                                          (You don't need to be granular in regards to errors in this case.
                                           This is to catch errors in the objective function itself.)
    """
    def __init__(self):
        #Variables that will be used in the objective.
        #NOTE: They are identified by necessary variables in class 'ObjectiveExecutor'.
        self.init_config = None
        self.trial = None
        self.device = None

    def verify_objective_static(self, init_config: ConfigSpace, missing_var_types_raise_error):
        """
        Verify the objective function on corresponding ConfigSpace statically.

        Returns:
        (calls_with_missing_var_types, calls_with_potentially_missing_var_types)
        """
        if not isinstance(init_config, ConfigSpace):
            raise ValueError(f"Input 'init_config' should be an instance of ConfigSpace but found type(init_config): {type(init_config)}")

        try:
            source = inspect.getsource(self.objective)
        except OSError as e:
            raise RuntimeError("Unable to retrieve source code of '.objective'"
                                "Ensure that the method is defined in a way that allows source code inspection.") from e

        #Dedent the source code
        source = textwrap.dedent(source)
        #Parse the source code into an AST
        tree = ast.parse(source)
        #Create an instance of the visitor and walk the tree
        visitor = SampleCallVisitor()
        visitor.visit(tree)

        config_var_types, sig_var_types = init_config.var_types(), init_config.sig_var_types()
        #Extract only the user requests that have either None or an instance of ast.Constant
        user_requests = visitor.user_requests
        user_requests_with_defaults = visitor.user_requests_with_defaults
        total_requests = user_requests | user_requests_with_defaults

        #Catch any illegal request(s) to signature parameters.
        #NOTE: Overlap between config_var_types and sig_var_types IS allowed,
        #      and we only check incidents for sig_var_types - config_var_types
        illegal_sig_var_types = sig_var_types - config_var_types
        illegal_sig_calls = set(filter(
            lambda req: req[0] in illegal_sig_var_types,
            total_requests
        ))
        if illegal_sig_calls:
            formatted = [f'self.sample({var_type},{default_val})' for var_type,default_val in illegal_sig_calls]
            raise SyntaxError(f"Following calls in '.objective' are illegal due to var_type referring to a signature parameter: {formatted}")

        #raise error iff missing_var_types_raise_error==True.
        calls_with_missing_var_types = {req for req in user_requests if req[0] not in config_var_types}
        formatted = [f'self.sample({req[0]},{req[1]})' for req in calls_with_missing_var_types]
        msg = f"Following calls in '.objective' are illegal due to var_type referring to a parameter that does not exist in the configuration space\n: {formatted}\n"
        if calls_with_missing_var_types and missing_var_types_raise_error:
            raise ValueError(msg)
        if calls_with_missing_var_types:
            print(msg)

        #This MUST be printed out to let the user know, even though it won't raise any error(s).
        calls_with_potentially_missing_var_types = {
            req for req in user_requests_with_defaults
            if req[0] not in config_var_types
        }
        formatted = [f'self.sample({req[0]},{req[1]})' for req in calls_with_potentially_missing_var_types]
        print(f"Following calls in '.objective' are referring to var_type(s) not registered in ConfigSpace and will simply return respective default_val(s)\n: {formatted}.\n")

        #Catch any (var_type, default_val) such that var_type exists in config_var_types BUT default_val is a value NOT allowed in the configuration space of 'var_type'.
        existent_requests_with_defaults = {req for req in user_requests_with_defaults if req[0] in config_var_types}
        req_to_sources = {req: init_config.get_source(req[0]) for req in existent_requests_with_defaults}
        requests_with_illegal_defaults = list()
        for (var_type,default_val),source in req_to_sources.items():
            params, sample_method = source['params'], source['sample']
            if any([
                sample_method=='float' and not(isinstance(default_val, (float,int)) and  params['low']<=default_val<=params['high']),
                sample_method=='int' and not(isinstance(default_val, int) and params['low']<=default_val<=params['high']),
                sample_method=='categorical' and default_val not in params['choices']
            ]):
                requests_with_illegal_defaults.append((var_type,default_val,params))
        if requests_with_illegal_defaults:
            formatted = [
                f'self.sample({v},{d}), where {d} not in {p}'
                for v,d,p in requests_with_illegal_defaults
            ]
            formatted = reduce(lambda s1,s2: s1+'\n'+s2, formatted)
            raise ValueError(f"Following calls in '.objective' are illegal due to following reason(s)\n: {formatted}")

    def verify_objective_dynamic(self, init_config: ConfigSpace, device: torch.device = None, turn_off_training_log: bool = False):
        #Static verification is necesssary before running dynamic check.
        try:
            self.verify_objective_static(init_config, True)
        except Exception as e:
            raise RuntimeError(f"Dynamic verification cannot be done due to the following error on static check:\n {e}")

        #Initialize member variables
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') if device is None else device
        self.init_config = init_config
        #Create a Mock FrozenTrial with necessary attributes.
        self.trial = Mock(spec=FrozenTrial)
        self.trial.params = dict() #Set .params to be an empty dictionary on purpose.

        #This .sample will be used to execute self.objective
        def sample(var_type, default_val=None):
            #When sample requested, find & load the corresponding Var object.
            var = self.init_config.get_var(var_type)

            assert isinstance(var, (Var, ActionVar)), f"type(var): {type(var)}"

            #Sample from var and return the result.
            return var.sample(trial=self.trial)

        #Use unittest.mock.patch.object to temporarily create a Mock(name=sample).side_effect = sample.
        with unittest.mock.patch.object(self, 'sample', side_effect=sample, create=True):
            try:
                self.objective(turn_off_training_log=turn_off_training_log)
            except Exception as e:
                raise RuntimeError(f"Dynamic verification failed: {e}") from e
            else:
                print("Dynamic verification passed successfully\n.")

    @abstractmethod
    def objective(self, turn_off_training_log):
        return NotImplementedError(".objective not implemented")

class UserObjective(Objective):
    def objective(self, turn_off_training_log):
        #dls_action <- Action object encapsulating DataLoaders construction(and involved parameters)
        dls_action = self.sample('dls_action')
        #dls <- Executed dls_action(device=self.device)
        dls = dls_action(device=self.device)
        #model_action <- ModelAction object encapsulating model construction(and involved parameters).
        model_action = self.sample('model_action')
        #model_action_func <- model_action's action_func
        model_action_func = model_action.action_func
        #model_action's saves state <- model_action.action_state.
        model_action_state = model_action.action_state
        #model <- Model yielded from action.
        model = model_action(device=self.device)
        #loss_func <- Sampled loss function.
        loss_func = self.sample('loss_func')
        #If the loss_func is a subclass of nn.Module, Then initialize.
        if isinstance(loss_func, type) and issubclass(loss_func, nn.Module):
            loss_func = loss_func()

        #model_container <- Fast.AI's model_container object constructed from it.
        model_container = Learner(
            dls=dls,
            model=model,
            loss_func=loss_func,
            #***opt_func=self.sample('opt_func'), NOT IMPLEMENTED YET!!!***
            metrics=self.sample('metric')
        )

        #Interaction with ModelAction.action_state during runtime to make a decision on 'freeze'.
        if model_action_state is None and self.sample('freeze', False):
            model_container.freeze()
        else: #IF (action_state is not None) OR (action_state is None but sampled 'freeze' states False, THEN .unfreeze().
            model_container.unfreeze()

        #Barebone cbs list:
        cbs = []
        #Fill the list:
        if self.sample('gradient_clip',False): #Apply Gradient clipping, if exists and is True.
            cbs.append(GradientClip(max_norm=self.sample('max_norm', 0.1))) #If exists, then 'max_norm' MUST exist, hence no safety guarantee.
        if self.trial is None:
            cbs.append(EarlyStoppingCallback(monitor='valid_loss',
                                            min_delta=self.sample('min_delta', 0.1),
                                            patience=self.sample('patience', 15)))
        elif isinstance(self.trial, Trial): #IFF when it's a proper trial, do you prune it.
            cbs.append(FastAIPruningCallback(trial=self.trial, monitor='valid_loss'))

        #Make the action and return it.
        trainer = None
        if self.sample('one_cycle', True):
            lr_low = self.sample('lr_low', 9e-4)
            lr_high = self.sample('lr_high', lr_low)
            trainer = partial(model_container.fit_one_cycle,
                            n_epoch=self.sample('n_epoch', 10),
                            lr_max=slice(lr_low, lr_high),
                            wd=self.sample('wd', 0.01), pct_start=self.sample('pct_start',0.3),
                            cbs=cbs)
        else:
            trainer = partial(model_container.fit,
                            n_epoch=self.sample('n_epoch', 10),
                            lr=self.sample('lr', 1e-3),
                            wd=self.sample('wd', 0.01),
                            cbs=cbs)

        if turn_off_training_log:
            #Train the model, without bar, logging.
            with model_container.no_bar(), model_container.no_logging():
                    trainer()
        else:
            trainer()

        #Return (objective loss, model container)
        return (model_container.validate()[0], model_container)

#Test: SampleCallVisitor, Objective, UserObjective

In [15]:
if __name__ == '__main__':
    from torch.utils.data import Dataset, DataLoader
    class UserObjectiveForTestingObjective(UserObjective):
        def __init__(self):
            super().__init__()

    class PolynomialDataset(Dataset):
        def __init__(self, coeffs: torch.tensor, input_min: float, input_max: float, input_size: int):
            self.coeffs = coeffs.clone().detach()
            self.inputs = torch.empty(size=(input_size,)).uniform_(input_min, input_max)
            self.outputs = self._f(X=self.inputs)

        def __getitem__(self, idx):
            return self.inputs[idx], self.outputs[idx]

        def __len__(self):
            return self.inputs.size(0)

        def _f(self, X):
            powers = torch.arange(self.coeffs.size(0)).unsqueeze(0)
            Y = self.inputs.unsqueeze(-1)**powers
            return torch.matmul(Y, self.coeffs).squeeze(-1)

    degree = 6
    coeffs_min, coeffs_max = -10.0, 10.0
    coeffs = torch.empty(size=(degree+1,)).uniform_(coeffs_min, coeffs_max)

    train_dset = PolynomialDataset(coeffs=coeffs, input_min=-100.0, input_max=100.0, input_size=8000)
    valid_dset = PolynomialDataset(coeffs=coeffs, input_min=-300.12, input_max=500.78, input_size=2000)

    class ModelDefault(nn.Module):
        def __init__(self, activation: nn.Module, preprocess: bool, start_neurons: int):
            super().__init__()
            self.preprocess = preprocess
            if self.preprocess:
                start_neurons = max(start_neurons, degree+1)
            self.net = nn.Sequential(
                nn.Linear(in_features=degree+1 if self.preprocess else 1, out_features=start_neurons),
                nn.BatchNorm1d(num_features=start_neurons),
                activation(),
                nn.Linear(in_features=start_neurons, out_features=start_neurons*2),
                nn.BatchNorm1d(num_features=start_neurons*2),
                activation(),
                nn.Linear(in_features=start_neurons*2, out_features=1)
            )

        def _preprocess(self, X):
            N = X.size(0)
            X = X.unsqueeze(-1)**torch.arange(degree+1).unsqueeze(0)
            return X

        def forward(self, X):
            if X.dim() != 1:
                X = X.reshape(-1)

            if self.preprocess:
                X = self._preprocess(X)
            else:
                X = X.unsqueeze(-1)
            return self.net(X)

    class ModelDropout(nn.Module):
        def __init__(self, activation: nn.Module, dropout_p: float, preprocess: bool, start_neurons: int):
            super().__init__()
            self.preprocess = preprocess
            if self.preprocess:
                start_neurons = max(start_neurons, degree+1)
            self.net = nn.Sequential(
                nn.Linear(in_features=degree+1 if self.preprocess else 1, out_features=start_neurons),
                nn.BatchNorm1d(num_features=start_neurons),
                activation(),
                nn.Dropout(p=dropout_p),
                nn.Linear(in_features=start_neurons, out_features=start_neurons*2),
                nn.BatchNorm1d(num_features=start_neurons*2),
                activation(),
                nn.Linear(in_features=start_neurons*2, out_features=1)
            )

        def _preprocess(self, X):
            N = X.size(0)
            X = X.unsqueeze(-1)**torch.arange(degree+1).unsqueeze(0)
            return X

        def forward(self, X):
            if X.dim() != 1:
                X = X.reshape(-1)

            if self.preprocess:
                X = self._preprocess(X)
            else:
                X = X.unsqueeze(-1)
            return self.net(X)

    def model_action_default(activation: nn.Module, preprocess: bool, start_neurons: int, device: torch.device):
        if start_neurons < 1:
            raise ValueError(f"start_neurons MUST be a positive integer but found start_neurons: {start_neurons}")
        return ModelDefault(activation=activation,
                            preprocess=preprocess,
                            start_neurons=start_neurons).to(device)

    def model_action_dropout(activation: nn.Module, dropout_p: float, preprocess: bool, start_neurons: int, device: torch.device):
        if start_neurons < 1:
            raise ValueError(f"start_neurons MUST be a positive integer but found start_neurons: {start_neurons}")
        if dropout_p < 0.0 or dropout_p >= 1.0:
            raise ValueError(f"0.0 <= dropout_p < 1.0 required but found dropout_p: {dropout_p}")
        return ModelDropout(activation=activation,
                            dropout_p=dropout_p,
                            preprocess=preprocess,
                            start_neurons=start_neurons).to(device)

    def dls_action_default(batch_size: int, device=None):
        train_dataloader = DataLoader(train_dset, batch_size=batch_size, shuffle=True)
        valid_dataloader = DataLoader(valid_dset, batch_size=batch_size, shuffle=True)
        dls = DataLoaders(train_dataloader, valid_dataloader)
        return dls

    def custom_mse(Y_hat, Y):
        Y_hat = Y_hat.reshape(-1)
        Y = Y.reshape(-1)
        return nn.MSELoss()(Y_hat, Y)

class TestUserObjective(unittest.TestCase):
    def setUp(self):
        self.INIT_DICT = {
            ('dls_action', None): {
                'params': {'choices': [dls_action_default]},
                'sample': 'categorical',
                'action': True
            },
            ('freeze', None): {
                'params': {'choices': [True, False]},
                'sample': 'categorical'
            },
            ('lr_low', None): {
                'params': {'low': 1e-7, 'high': 9e-4, 'log': True},
                'sample': 'float'
            },
            ('lr_high', None): {
                'params': {'low': 9e-4, 'high': 1e-1, 'log': True},
                'sample': 'float'
            },
            ('lr', None): {
                'params': {'low': 1e-6, 'high': 1e-1, 'log': True},
                'sample': 'float'
            },
            ('wd', None): {
                'params': {'choices': [1e-4, 1e-3, 1e-2, 1e-1]},
                'sample': 'categorical'
            },
            ('gradient_clip', None): {
                'params': {'choices': [True, False]},
                'sample': 'categorical'
            },
            ('max_norm', None): {
                'params': {'low': 0.0, 'high': 15.0, 'log': False},
                'sample': 'float'
            },
            ('one_cycle', None): {
                'params': {'choices': [True, False]},
                'sample': 'categorical'
            },
            ('pct_start', None): {
                'params': {'low': 0.10, 'high': 0.95, 'log': False, 'step': 0.05},
                'sample': 'float'
            },
            ('n_epoch', None): {
                'params': {'choices': [5, 10, 15]},
                'sample': 'categorical'
            },
            ('model_action', None): {
                'params': {'choices': [model_action_default, model_action_dropout]},
                'sample': 'categorical',
                'action': True,
                'save_state': True
            },
            ('activation', 'model_action'): {
                'params': {'choices': [nn.ReLU, nn.PReLU, nn.SiLU]},
                'sample': 'categorical'
            },
            ('loss_func', None): {
                'params': {'choices': [custom_mse]},
                'sample': 'categorical'
            },
            ('metric', None): {
                'params': {'choices': [custom_mse]},
                'sample': 'categorical'
            },
            ('preprocess', 'model_action'): {
                'params': {'choices': [True, False]},
                'sample': 'categorical'
            },
            ('start_neurons', 'model_action'): {
                'params': {'choices': [1, 2, 4, 8, 16, 32]},
                'sample': 'categorical'
            },
            ('dropout_p', 'model_action'): {
                'params': {'low': 0.01, 'high': 0.95, 'log': False, 'step': 0.05},
                'sample': 'float'
            },
            ('batch_size', 'dls_action'): {
                'params': {'choices': [2, 4, 8, 16, 32, 64]},
                'sample': 'categorical'
            }
        }
        #ConfigSpace, is always to be tested WITH Objective, UserObjective.
        self.cs = ConfigSpace(init_config=self.INIT_DICT)
        self.objective = UserObjectiveForTestingObjective()
        self.trial = optuna.create_study(direction='minimize').ask()
        self.device = torch.device('cpu')

    def test_verify_on(self):
        #Should pass w/o any trouble.
        self.objective.verify_objective_static(self.cs, True)

        self.objective.verify_objective_dynamic(self.cs)

        self.assertTrue(True)

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

..............................................................[I 2024-10-13 21:16:58,363] A new study created in memory with name: no-name-6dc39dfe-4ca6-462d-b5a9-12b99007fd3b
.[I 2024-10-13 21:16:58,368] A new study created in memory with name: no-name-9ae456cb-ef5d-4619-ab56-0fb3c18e283f
.[I 2024-10-13 21:16:58,371] A new study created in memory with name: no-name-bd19ab5d-2693-4862-a351-834ab92f040d
.[I 2024-10-13 21:16:58,376] A new study created in memory with name: no-name-8216b98d-f9f6-4c9a-a5e7-6e1eb7c3726a
.[I 2024-10-13 21:16:58,381] A new study created in memory with name: no-name-0c575802-5bb7-4dc4-a27f-1ece179b2fd9
.[I 2024-10-13 21:16:58,383] A new study created in memory with name: no-name-80363a3a-70a1-4280-b188-8d72d4f4809a
.[I 2024-10-13 21:16:58,387] A new study created in memory with name: no-name-0ac82962-cfe3-45a1-a386-e70bb3962d30
.[I 2024-10-13 21:16:58,401] A new study created in memory with name: no-name-156dff5a-8067-4855-9ffc-5aea51eae5b3


All good

All good

Missing signature parameters in optimizer: {'param3', 'param1', 'param4', 'param2'} 

Missing signature parameters in optimizer: {'param3', 'param4', 'param2'} 

Missing signature parameters in optimizer: {'param3', 'param4'} 

Missing signature parameters in optimizer: {'param4'} 

All good

All good

All good

Missing signature parameters in optimizer: {'param3', 'param1', 'param4', 'param2'} 

Missing signature parameters in optimizer: {'param3', 'param1', 'param4', 'param2'} 

Missing signature parameters in optimizer: {'param3', 'param1', 'param4', 'param2'} 

Missing signature parameters in optimizer: {'param3', 'param1', 'param4', 'param2'} 

Missing signature parameters in optimizer: {'param3', 'param4', 'param2'} 

Missing signature parameters in optimizer: {'param3', 'param4'} 

Missing signature parameters in optimizer: {'param4'} 

All good

Missing signature parameters in optimizer: {'param1'} 

All good

All good

All good

All good

Missing signature 

epoch,train_loss,valid_loss,custom_mse,time
0,4161867640567127370366976.000000,681844297896722503585284000579584.000000,681844297896722503585284000579584.000000,00:00
1,4096315117888567416717312.000000,681843988411712682240215275798528.000000,681843988411712682240215275798528.000000,00:00
2,4387252265502719885705216.000000,681843446812945494886345007431680.000000,681843446812945494886345007431680.000000,00:00
3,4487623882311535073165312.000000,681842982585430762868741920260096.000000,681842982585430762868741920260096.000000,00:00
4,4361360242582635468029952.000000,681842982585430762868741920260096.000000,681842982585430762868741920260096.000000,00:00


...........[I 2024-10-13 21:17:03,137] A new study created in memory with name: no-name-1ed78e1a-707b-4881-9696-356338792c61
..[I 2024-10-13 21:17:03,147] A new study created in memory with name: no-name-1993d275-6bfb-44e3-861d-655a307ca80f
.....
----------------------------------------------------------------------
Ran 87 tests in 4.911s

OK


Dynamic verification passed successfully
.


#Class: ActionSampler

In [16]:
class ActionSampler:
    def __init__(self, INIT_ACTION: ConfigSpace):
        if not isinstance(INIT_ACTION, ConfigSpace):
            raise ValueError(f"Input 'INIT_ACTION' must be an instance of ConfigSpace but found: {type(INIT_ACTION)}")

        self.action_vars = {var.var_type: var for var in INIT_ACTION.get_action_vars()}
        self.model_var_types = INIT_ACTION.action_var_types(save_state=True)

    def __len__(self):
        return len(self.action_vars)

    def __eq__(self, other: ActionSampler):
        return all([
            isinstance(other, ActionSampler),
            self.action_vars == other.action_vars,
            self.model_var_types == other.model_var_types
        ])

    def __str__(self):
        output = "**********ActionSampler**********\n"
        output += "".join([f"{action_var}\n" for action_var in self.action_vars.values()])
        return output

    def get_var(self, var_type: str, action_idx_or_action_func=None, sig_var_type: str=None) -> ActionVar:
        if var_type not in self.keys():
            raise KeyError(f"ActionSampler cannot retrieve Requested ActionVar: {var_type}, since it does NOT belong to: {self.keys()}")
        if action_idx_or_action_func != sig_var_type and (action_idx_or_action_func is None or sig_var_type is None):
            raise KeyError(f"Either both action_idx_or_action_func,sig_var_type should be None or both of them must be properly defined but found {(action_idx_or_action_func,sig_var_type)}")
        if sig_var_type is None:
            return self.action_vars[var_type]
        return self.get_action(var_type, action_idx_or_action_func).get_var(sig_var_type)

    def get_action(self, action_var_type: str, action_idx_or_action_func) -> Action:
        action_var = self.get_var(action_var_type)
        assert isinstance(action_var, ActionVar), f"type(action_var): {type(action_var)} != ActionVar"
        return action_var.get_action(action_idx_or_action_func)

    def is_fixed(self):
        return all(action_var.is_fixed() for action_var in self.action_vars.values())

    def keys(self):
        return self.action_vars.keys()

    def values(self):
        return self.action_vars.values()

    def items(self):
        return self.action_vars.items()

    def sample(self, action_var_type: str, trial: Trial):
        if trial is None:
            raise ValueError("Input 'trial' cannot be None when sampling is requested through ActionSampler.")
        if not isinstance(trial, (Trial, FrozenTrial)):
            raise ValueError(f"Input 'trial' should be an object optuna::Trial or optuna::FrozenTrial but found: {type(trial)}")

        action_var = self.get_var(action_var_type)
        assert isinstance(action_var, ActionVar), f"type(action_var): {type(action_var)} != ActionVar"

        return action_var.sample(trial)

    def get_evaluated_params(self, source, exclude):
        if not isinstance(source, (Trial, FrozenTrial, dict)):
            raise ValueError(f"Input 'source' must be of type in (optuna::Trial, optuna::FrozenTrial, dict) but found: {source}")
        if not isinstance(exclude, (list, tuple, str)):
            raise ValueError(f"Input 'exclude' must be one of (list, tuple, str) but found type(exclude): {type(exclude)}")

        source = source if isinstance(source, dict) else source.params
        params = dict()

        if isinstance(exclude, (list, tuple)):
            exclude_set = set(exclude)
        else:
            exclude_set = {exclude}
        action_var_types = set(self.keys()) - exclude_set

        action_vars_to_assemble = {
            key: val for key, val in source.items()
            if key[0] in action_var_types and key[1] == 'None'
        }

        for key, val in action_vars_to_assemble.items():
            action_var = self.get_var(key[0])
            params[key] = action_var.get_loadable_action(val, source)

        return params

    def update(self, list_params: list, update_state_window: int, device: torch.device):
        #IF action_var_type == 'model_action'
        for action_var_type, action_var in self.items():
            relevant_params = [params for params in list_params if (action_var_type, 'None') in params]
            if not relevant_params:
                continue

            action_idxs = [params[(action_var_type,'None')] for params in relevant_params]
            action_var.update(vals=action_idxs)

            unique_action_idxs = list(dict.fromkeys(action_idxs).keys())

            if action_var_type in self.model_var_types:
                relevant_params = [
                    params|self.get_evaluated_params(source=params,exclude=[action_var_type])
                    for params in relevant_params
                ]

            action_var.update_actions(
                update_action_idxs=unique_action_idxs,
                list_params=relevant_params,
                update_state_window=update_state_window,
                device=device
            )

#Test: ActionSampler

In [17]:
if __name__ == '__main__':
    class UserObjectiveActionSampler:
        def objective(self, turn_off_training_log: bool):
            model = self.default_dict['model_action'](device=self.device)

            for param in model.parameters():
                param.data.fill_(0)

            model_container = Mock(spec=Learner)
            model_container.model = model

            return (0, model)

    #Sample model classes
    class ModelDefaultActionSampler(nn.Module):
        def __init__(self, rand_number: int):
            super().__init__()
            self.net = nn.Linear(16, 10)
            self.rand_number = torch.tensor(rand_number)

        def forward(self, x):
            return self.net(x) + self.rand_number

    class ModelDropoutActionSampler(nn.Module):
        def __init__(self, activation: nn.Module, dropout_p: float):
            super().__init__()
            self.activation = activation
            self.dropout_p = dropout_p
            self.net = nn.Sequential(
                nn.Linear(16, 10),
                activation(),
                nn.Dropout(p=dropout_p),
                nn.Linear(10, 10)
            )

        def forward(self, x):
            return self.net(x)

    #Action functions
    def model_action_default_action_sampler(rand_number: int, device: torch.device):
        return ModelDefaultActionSampler(rand_number=rand_number).to(device)

    def model_action_dropout_action_sampler(activation, dropout_p, device: torch.device):
        return ModelDropoutActionSampler(activation=activation, dropout_p=dropout_p).to(device)

    def dls_action_default_action_sampler(batch_size: int, device: torch.device):
        return {'batch_size': batch_size, 'device': device}

    def dls_action_normalize_action_sampler(batch_size: int, normalize: bool):
        return {'batch_size': batch_size, 'normalize': normalize}

    def dls_action_activation_action_sampler(batch_size: int, normalize: bool, activation: nn.Module):
        return {'batch_size': batch_size, 'normalize': normalize, 'activation': activation}

class TestActionSampler(unittest.TestCase):
    def setUp(self):
        #Initialize ConfigSpace
        self.cs = ConfigSpace()
        self.user_objective = UserObjectiveActionSampler()
        setattr(ObjectiveExecutor, 'execute_objective', self.user_objective.objective.__func__)

        #Define action hyperparameters
        self.cs.set_categorical(
            var_type='model_action',
            choices=[model_action_default_action_sampler, model_action_dropout_action_sampler],
            action=True,
            save_state=True
        )
        self.cs.set_categorical(
            var_type='dls_action',
            choices=[dls_action_default_action_sampler, dls_action_normalize_action_sampler, dls_action_activation_action_sampler],
            action=True
        )

        #Define signature parameters for model_action
        self.cs.set_int(
            var_type='rand_number',
            low=1,
            high=10,
            belongs_to='model_action'
        )
        self.cs.set_categorical(
            var_type='activation',
            choices=[nn.ReLU, nn.PReLU, nn.SiLU],
            belongs_to='model_action'
        )
        self.cs.set_float(
            var_type='dropout_p',
            low=0.0,
            high=1.0,
            belongs_to='model_action'
        )

        # Define signature parameters for dls_action
        self.cs.set_int(
            var_type='batch_size',
            low=16,
            high=64,
            belongs_to='dls_action'
        )
        self.cs.set_categorical(
            var_type='normalize',
            choices=[True, False],
            belongs_to='dls_action'
        )
        self.cs.set_categorical(
            var_type='activation',
            choices=[nn.ReLU, nn.SiLU],
            belongs_to='dls_action'
        )

        #Initialize ActionSampler with ConfigSpace
        self.action_sampler = ActionSampler(self.cs)

    def test_initialization(self):
        #Test that ActionSampler initializes correctly
        self.assertEqual(len(self.action_sampler), 2)
        self.assertIn('model_action', self.action_sampler.keys())
        self.assertIn('dls_action', self.action_sampler.keys())
        self.assertIn('model_action', self.action_sampler.model_var_types)
        self.assertNotIn('dls_action', self.action_sampler.model_var_types)
        self.assertEqual(len(self.action_sampler), 2)
        self.assertEqual(
            self.action_sampler.keys(),
            {'model_action', 'dls_action'}
        )
        self.assertEqual(
            self.action_sampler.model_var_types,
            {'model_action'}
        )

    def test_sample_actions(self):
        #Test sampling model_action
        trial = optuna.create_study(direction='minimize').ask()
        model_action = self.action_sampler.sample('model_action', trial)
        self.assertIsInstance(model_action, ModelAction)
        self.assertIsNotNone(model_action.params)
        self.assertIsNotNone(model_action._model_loader)

        #Test sampling dls_action
        trial = optuna.create_study(direction='minimize').ask()
        dls_action = self.action_sampler.sample('dls_action', trial)
        self.assertIsInstance(dls_action, Action)
        self.assertIsNotNone(dls_action.params)
        self.assertIsNotNone(dls_action._model_loader)

    def test_get_evaluated_params(self):
        #Create a source with both model_action and dls_action parameters
        source_params = {
            ('model_action', 'None'): 0,
            ('rand_number', 'model_action'): 5,
            ('dls_action', 'None'): 2,
            ('batch_size', 'dls_action'): 32,
            ('normalize', 'dls_action'): 1, #False
            ('activation', 'dls_action'): 0  # nn.ReLU
        }
        source_params_two = {
            ('model_action', 'None'): 1,
            ('activation', 'model_action'): 2, #nn.SiLU
            ('dropout_p', 'model_action'): 0.5,
            ('dls_action', 'None'): 1,
            ('batch_size', 'dls_action'): 48,
            ('normalize', 'dls_action'): 0, #True
        }
        list_params = [source_params, source_params_two]

        for count,source_params in enumerate(list_params):
            #Exclude 'model_action' and get evaluated params
            evaluated_params = self.action_sampler.get_evaluated_params(source_params, exclude='model_action')
            self.assertIn(('dls_action', 'None'), evaluated_params, msg=f"action_sampler.var_types: {self.action_sampler.keys()}")
            dls_action = evaluated_params[('dls_action', 'None')]
            self.assertIsInstance(dls_action, Action)
            self.assertNotIn(('model_action', 'None'), evaluated_params)
            if count==0:
                d = {'batch_size': 32, 'normalize': False, 'activation': nn.ReLU}
                self.assertEqual(
                    dls_action(device=None),
                    d,
                    msg=f"dls_action(): {dls_action(device=None)} != \nExpected: {d}"
                )
            else:
                d = {'batch_size': 48, 'normalize': True}
                self.assertEqual(
                    dls_action(device=None),
                    d,
                    msg=f"dls_action(): {dls_action(device=None)} != \nExpected: {d}"
                )

            #Exclude 'dls_action' and get evaluated params
            evaluated_params = self.action_sampler.get_evaluated_params(
                source_params,
                exclude='dls_action'
            )
            self.assertIn(('model_action', 'None'), evaluated_params)
            model_action = evaluated_params[('model_action', 'None')]
            self.assertIsInstance(model_action, ModelAction)
            self.assertNotIn(('dls_action', 'None'), evaluated_params)
            if count==0:
                d = {'rand_number': 5}
                self.assertEqual(
                    model_action._model_loader.keywords,
                    d,
                    msg=f"model_action._model_loader.keywords: {model_action._model_loader.keywords} != \nExpected: {d}"
                )
            else:
                d = {'dropout_p': 0.5, 'activation':nn.SiLU}
                self.assertEqual(
                    model_action._model_loader.keywords,
                    d,
                    msg = f"model_action._model_loader.keywords: {model_action._model_loader.keywords} != \nExpected: {d}"
                )

    def test_update_method(self):
        #Prepare list_params with mixed model_action and dls_action parameters
        list_params = [
            # Entry 1:
            {
                ('model_action', 'None'): 0,  # model_action_default_action_sampler
                ('rand_number', 'model_action'): 5,

                ('dls_action', 'None'): 1,  # dls_action_normalize_action_sampler
                ('batch_size', 'dls_action'): 32,
                ('normalize', 'dls_action'): 1,  # False

                ('lr', 'None'): 0.01  # non_action_var_type
            },
            # Entry 2:
            {
                ('model_action', 'None'): 1,  # model_action_dropout_action_sampler
                ('activation', 'model_action'): 1,  # nn.PReLU
                ('dropout_p', 'model_action'): 0.5,

                ('dls_action', 'None'): 2,  # dls_action_activation_action_sampler
                ('batch_size', 'dls_action'): 64,
                ('normalize', 'dls_action'): 1,  # False
                ('activation', 'dls_action'): 1,  # nn.SiLU

                ('lr', 'None'): 0.001  # non_action_var_type
            },
            # Entry 3:
            {
                ('model_action', 'None'): 1,  # model_action_dropout_action_sampler
                ('activation', 'model_action'): 0,  # nn.ReLU
                ('dropout_p', 'model_action'): 0.23,

                ('dls_action', 'None'): 0,  # dls_action_default_action_sampler
                ('batch_size', 'dls_action'): 32,

                ('lr', 'None'): 0.988  # non_action_var_type
            },
            # Entry 4:
            {
                ('dls_action', 'None'): 2,  # dls_action_activation_action_sampler
                ('batch_size', 'dls_action'): 64,
                ('normalize', 'dls_action'): 1,  # False
                ('activation', 'dls_action'): 0  # nn.ReLU
            },
            # Entry 5:
            {
                ('model_action', 'None'): 0,  # model_action_default_action_sampler
                ('rand_number', 'model_action'): 8,
                ('lr', 'None'): 0.02
            }
        ]

        #Call update with update_state_window=1
        self.action_sampler.update(list_params, update_state_window=1, device=torch.device('cpu'))

        #Verify that the actions have been updated correctly
        #For model_action
        model_action_var = self.action_sampler.get_var('model_action')
        self.assertEqual(model_action_var.distribution(indices=True), [0,0,1,1])

        #For dls_action
        dls_action_var = self.action_sampler.get_var('dls_action')
        self.assertEqual(dls_action_var.distribution(indices=True), [0,1,2,2])

        #Determine which model action should have updated state
        #The unique action indices in the order they appear are [0, 1]
        #Since update_state_window=1, only the first action (index 0) should have its state updated

        model_action_default_action_sampler = model_action_var.get_action(0)
        model_action_dropout_action_sampler = model_action_var.get_action(1)

        #Verify that only the first model action has updated action_state
        self.assertIsNotNone(model_action_default_action_sampler.action_state)
        self.assertIsNone(model_action_dropout_action_sampler.action_state)

        #Verify that dls actions do not have action_state updated
        for idx in [0, 1, 2]:
            dls_action = dls_action_var.get_action(idx)
            self.assertIsInstance(dls_action, Action)
            self.assertIsNone(dls_action.action_state)

        #Additionally, we can verify the updated variables
        #For model_action_default_action_sampler
        rand_number_var = model_action_var.get_var(0, 'rand_number')
        self.assertEqual(rand_number_var.distribution(), {'low':5, 'high':8, 'step':1, 'log':False})

        #For model_action_dropout_action_sampler
        activation_var = model_action_var.get_var(1, 'activation')
        self.assertEqual(activation_var.distribution(), [nn.ReLU, nn.PReLU])

        dropout_p_var = model_action_var.get_var(1, 'dropout_p')
        self.assertEqual(dropout_p_var._dist['low'], 0.23)
        self.assertEqual(dropout_p_var._dist['high'], 0.5)

        for i in range(0,3):
            batch_size_var = dls_action_var.get_var(i, 'batch_size')
            self.assertEqual(batch_size_var._dist['low'], 32)
            self.assertEqual(batch_size_var._dist['high'], 64)

        for i in range(1,3):
            normalize_var = dls_action_var.get_var(i, 'normalize')
            self.assertEqual(normalize_var.distribution(), [False])

        for i in range(2,3):
            activation_var = dls_action_var.get_var(i, 'activation')
            self.assertEqual(activation_var.distribution(), [nn.ReLU, nn.SiLU])

    def test_update_dls_action(self):
        #Prepare list_params with only dls_action parameters
        list_params = [
            # Entry 1: dls_action_default_action_sampler, batch_size=32
            {
                ('dls_action', 'None'): 0,
                ('batch_size', 'dls_action'): 32
            },
            # Entry 2: dls_action_normalize_action_sampler, batch_size=48, normalize=True
            {
                ('dls_action', 'None'): 1,
                ('batch_size', 'dls_action'): 48,
                ('normalize', 'dls_action'): 0  # True
            },
            # Entry 3: dls_action_activation_action_sampler, batch_size=64, normalize=False, activation=nn.SiLU
            {
                ('dls_action', 'None'): 2,
                ('batch_size', 'dls_action'): 64,
                ('normalize', 'dls_action'): 1,  # False
                ('activation', 'dls_action'): 1  # nn.SiLU
            }
        ]

        #Call update with update_state_window=1 (should be ignored for dls_action)
        self.action_sampler.update(list_params, update_state_window=1, device=torch.device('cpu'))

        #Verify that the dls actions have updated configuration spaces
        dls_action_var = self.action_sampler.get_var('dls_action')

        batch_size_var = dls_action_var.get_var(0, 'batch_size')
        self.assertEqual(batch_size_var._dist['low'], 32)
        self.assertEqual(batch_size_var._dist['high'], 64)

        normalize_var = dls_action_var.get_var(1, 'normalize')
        self.assertEqual(normalize_var.distribution(), [True, False])

        activation_var = dls_action_var.get_var(2, 'activation')
        self.assertEqual(activation_var.distribution(), [nn.SiLU])

        # Verify that action_state is not updated for dls actions
        for idx in [0, 1, 2]:
            action = dls_action_var.get_action(idx)
            self.assertIsNone(action.action_state)

    def test_update_model_action(self):
        # Prepare list_params with only model_action parameters
        list_params = [
            # Entry 1: model_action_default_action_sampler, rand_number=5
            {
                ('model_action', 'None'): 0,
                ('rand_number', 'model_action'): 5,
                ('lr', 'None'): 0.01
            },
            # Entry 2: model_action_dropout_action_sampler, activation=nn.SiLU, dropout_p=0.3
            {
                ('model_action', 'None'): 1,
                ('activation', 'model_action'): 2,  # nn.SiLU
                ('dropout_p', 'model_action'): 0.3,
                ('lr', 'None'): 0.001
            },
            # Entry 3: model_action_default_action_sampler, rand_number=8
            {
                ('model_action', 'None'): 0,
                ('rand_number', 'model_action'): 8,
                ('lr', 'None'): 0.02
            }
        ]

        # Call update with update_state_window=2
        self.action_sampler.update(list_params, update_state_window=2, device=torch.device('cpu'))

        # Verify that the model actions have updated configuration spaces
        model_action_var = self.action_sampler.get_var('model_action')

        rand_number_var = model_action_var.get_var(0, 'rand_number')
        self.assertEqual(rand_number_var.distribution(), {'low': 5, 'high': 8, 'log': False, 'step': 1})

        activation_var = model_action_var.get_var(1, 'activation')
        self.assertEqual(activation_var.distribution(), [nn.SiLU])

        dropout_p_var = model_action_var.get_var(1, 'dropout_p')
        self.assertEqual(dropout_p_var._dist['low'], 0.3)
        self.assertEqual(dropout_p_var._dist['high'], 0.3)

        # Verify that action_state is updated for model actions
        model_action_default_action_sampler = model_action_var.get_action(0)
        model_action_dropout_action_sampler = model_action_var.get_action(1)

        self.assertIsNotNone(model_action_default_action_sampler.action_state)
        self.assertIsNotNone(model_action_dropout_action_sampler.action_state)

    def test_update_with_mixed_actions(self):
        # Prepare list_params with both model_action and dls_action parameters
        list_params = [
            # Entry 1: model_action_default_action_sampler, rand_number=5
            {
                ('model_action', 'None'): 0,
                ('rand_number', 'model_action'): 5,
                ('lr', 'None'): 0.01,
                ('dls_action', 'None'): 1,
                ('batch_size', 'dls_action'): 48,
                ('normalize', 'dls_action'): 0  # True
            },
            # Entry 2: model_action_dropout_action_sampler, activation=nn.SiLU, dropout_p=0.3
            {
                ('model_action', 'None'): 1,
                ('activation', 'model_action'): 2,  # nn.SiLU
                ('dropout_p', 'model_action'): 0.3,
                ('lr', 'None'): 0.001,
                ('dls_action', 'None'): 2,
                ('batch_size', 'dls_action'): 64,
                ('normalize', 'dls_action'): 1,  # False
                ('activation', 'dls_action'): 1  # nn.SiLU
            }
        ]

        # Call update with update_state_window=1
        self.action_sampler.update(list_params, update_state_window=1, device=torch.device('cpu'))

        # Verify that the model actions have updated states
        model_action_var = self.action_sampler.get_var('model_action')
        model_action_default_action_sampler = model_action_var.get_action(0)
        model_action_dropout_action_sampler = model_action_var.get_action(1)

        self.assertIsNotNone(model_action_default_action_sampler.action_state)
        self.assertIsNone(model_action_dropout_action_sampler.action_state)  # Since update_state_window=1

        # Verify that dls actions do not have action_state updated
        dls_action_var = self.action_sampler.get_var('dls_action')
        for idx in [0, 1, 2]:
            action = dls_action_var.get_action(idx)
            self.assertIsNone(action.action_state)

        #Verify that get_evaluated_params correctly assembles actions
        #When updating model_action, dls_action should be prepared
        #This is tested by checking if the dls_action is included in the parameters passed to ObjectiveExecutor
        #(In a real implementation, the ObjectiveExecutor function would use these actions)

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

..[I 2024-10-13 21:17:03,224] A new study created in memory with name: no-name-5c403c0a-d254-4fef-8144-0364f250b54e
[I 2024-10-13 21:17:03,226] A new study created in memory with name: no-name-2b062e7a-9c25-4992-966e-e4562883db42
.......................................

All good

Missing signature parameters in model_action: {'dropout_p', 'rand_number', 'activation'} 

Missing signature parameters in model_action: {'dropout_p', 'rand_number', 'activation'} 
Missing signature parameters in dls_action: {'normalize', 'activation', 'batch_size'} 

Missing signature parameters in model_action: {'dropout_p', 'activation'} 
Missing signature parameters in dls_action: {'normalize', 'activation', 'batch_size'} 

Missing signature parameters in model_action: {'dropout_p'} 
Missing signature parameters in dls_action: {'normalize', 'activation', 'batch_size'} 

Missing signature parameters in dls_action: {'normalize', 'activation', 'batch_size'} 

Missing signature parameters in dls_action: {'normalize', 'activation'} 

Missing signature parameters in dls_action: {'activation'} 

All good

All good

All good

Missing signature parameters in model_action: {'dropout_p', 'rand_number', 'activation'} 

Missing signature parameters in model_action: {'dropout_p', 'rand

............................[I 2024-10-13 21:17:03,373] A new study created in memory with name: no-name-9db64d54-f7ab-4ffc-a15c-2d2fe22b06a2
.[I 2024-10-13 21:17:03,384] A new study created in memory with name: no-name-1d6d2fed-b54b-4d06-8381-b5244799fef0
.[I 2024-10-13 21:17:03,387] A new study created in memory with name: no-name-affe642d-5c61-48e6-b301-25cedcad7663
.[I 2024-10-13 21:17:03,392] A new study created in memory with name: no-name-bd49a946-6142-4aef-b6de-df21c737f7a2
.[I 2024-10-13 21:17:03,396] A new study created in memory with name: no-name-2fe2b674-aea5-44b1-9a92-35d9c33defe1
.[I 2024-10-13 21:17:03,398] A new study created in memory with name: no-name-b8ae1a8a-982b-412f-8005-c0e7d1123cbf
.[I 2024-10-13 21:17:03,403] A new study created in memory with name: no-name-efc14f5d-e96e-446b-92d2-d178fa7d123e
.[I 2024-10-13 21:17:03,414] A new study created in memory with name: no-name-1b5b0c67-064c-4ec7-b76d-ac8e5b3e6573


All good

Missing signature parameters in optimizer: {'param1', 'param2'} 

Missing signature parameters in optimizer: {'param1', 'param2'} 

Missing signature parameters in optimizer: {'param2'} 

All good

All good

All good

Missing signature parameters in optimizer: {'param3', 'param1', 'param4', 'param2'} 

Missing signature parameters in optimizer: {'param3', 'param4', 'param2'} 

Missing signature parameters in optimizer: {'param3', 'param4'} 

Missing signature parameters in optimizer: {'param4'} 

All good

Missing signature parameters in optimizer: {'param3', 'param1', 'param4', 'param2'} 

Missing signature parameters in optimizer: {'param3', 'param1', 'param4'} 

Missing signature parameters in optimizer: {'param3', 'param4'} 

Missing signature parameters in optimizer: {'param3'} 

All good

All good

All good

Missing signature parameters in optimizer: {'param1', 'param2'} 

All good

All good

All good

Missing signature parameters in optimizer: {'param1', 'param2'} 

Al

epoch,train_loss,valid_loss,custom_mse,time
0,4487048574480736256524288.000000,681844297896722503585284000579584.000000,681844297896722503585284000579584.000000,00:01
1,4445150542312194982281216.000000,681844297896722503585284000579584.000000,681844297896722503585284000579584.000000,00:01
2,4172877464475370455564288.000000,681844375267974958921551181774848.000000,681844375267974958921551181774848.000000,00:01
3,4229637807759798594174976.000000,681844530010479869594085544165376.000000,681844530010479869594085544165376.000000,00:01
4,4338472156642804190150656.000000,681844375267974958921551181774848.000000,681844375267974958921551181774848.000000,00:01
5,4396682298719275438833664.000000,681844375267974958921551181774848.000000,681844375267974958921551181774848.000000,00:01
6,4856138812127673061474304.000000,681844375267974958921551181774848.000000,681844375267974958921551181774848.000000,00:01
7,4381833246200691553206272.000000,681844375267974958921551181774848.000000,681844375267974958921551181774848.000000,00:01
8,4695164452889951266144256.000000,681844452639227414257818362970112.000000,681844452639227414257818362970112.000000,00:01
9,4129932291350270012555264.000000,681844452639227414257818362970112.000000,681844452639227414257818362970112.000000,00:01


...........[I 2024-10-13 21:17:17,422] A new study created in memory with name: no-name-237a7eeb-e1f6-4770-ab93-bbb7024f50be
..[I 2024-10-13 21:17:17,426] A new study created in memory with name: no-name-ab5bcb54-6a74-432d-8b5a-e3a8870f6a1a
.....
----------------------------------------------------------------------
Ran 94 tests in 14.222s

OK


Dynamic verification passed successfully
.


#Class: PrimitiveSampler

In [18]:
class PrimitiveSampler:
    def __init__(self, INIT_PRIM: ConfigSpace):
        if not isinstance(INIT_PRIM, ConfigSpace):
            raise ValueError(f"Input 'INIT_PRIM' must be an instance of ConfigSpace but found type(INIT_PRIM): {type(INIT_PRIM)}")
        self.primitive_vars = {var.var_type: var for var in INIT_PRIM.get_non_action_vars()}

    def __len__(self):
        return len(self.primitive_vars)

    def __eq__(self, other: PrimitiveSampler):
        return isinstance(other, PrimitiveSampler) and self.primitive_vars == other.vars

    def __str__(self):
        output = "**********PrimitiveSampler**********\n"
        output += "".join([f"{var}\n" for var in self.primitive_vars.values()])
        return output

    def get_var(self, var_type):
        if var_type not in self.keys():
            raise KeyError(f"Requested 'var_type': {var_type} is not in PrimitiveSampler.keys(): {self.keys()}.")
        return self.primitive_vars[var_type]

    def keys(self):
        return self.primitive_vars.keys()

    def values(self):
        return self.primitive_vars.values()

    def items(self):
        return self.primitive_vars.items()

    def is_fixed(self):
        return all(var.is_fixed() for var in self.values())

    def get_evaluated_params(self, source):
        if not isinstance(source, (Trial, FrozenTrial, dict)):
            raise ValueError(f"Input 'source' must be of type in (optuna::Trial, optuna::FrozenTrial, dict) but found: {source}")

        source = source if isinstance(source, dict) else source.params

        params = {
            (var_type,belongs_to):val for (var_type,belongs_to),val in source.items()
            if var_type in self.keys() and belongs_to=='None'
        }

        for key,val in params.items():
            var = self.get_var(key[0])
            if var.contain_idx(val):
                val = var.convert_idx_to_val(val)
            params[key] = val

        return params

    def sample(self, var_type: str, trial):
        if trial is None:
            raise ValueError("Input 'trial' cannot be 'None'. Pass in an Trial or FrozenTrial instance.")
        if not isinstance(trial, (Trial, FrozenTrial)):
            raise ValueError(f"Input 'trial' should be an instance of Trial or FrozenTrial but found: {trial}")
        if var_type not in self.keys():
            raise KeyError(f"Requested var_type: {var_type} does not exist in the PrimitiveSampler.")

        #sample value from the variable of type 'var_type'.
        sampled_val = self.get_var(var_type).sample(trial)

        return sampled_val

    def update_vars(self, list_params: list):
        #Update variables based on the list of parameters
        for var_type, var in self.items():
            var.update(vals=[params[(var_type, 'None')] for params in list_params if (var_type, 'None') in params.keys()])

#Class: CombinedSampler

In [19]:
class CombinedSampler:
    def __init__(self, INIT_CONFIG: ConfigSpace):
        if not isinstance(INIT_CONFIG, ConfigSpace):
            raise ValueError(f"Input 'INIT_CONFIG' must be of type in ConfigSpace but found type(INIT_CONFIG): {type(INIT_CONFIG)}")
        self.action_sampler = ActionSampler(INIT_CONFIG)
        self.primitive_sampler = PrimitiveSampler(INIT_CONFIG)
        self._sig_var_types = INIT_CONFIG.sig_var_types()

    def __str__(self):
        output = "CombinedSampler:\n"
        output += f"{self.action_sampler}\n"
        return output + f"{self.primitive_sampler}\n"

    def is_fixed(self):
        return self.primitive_sampler.is_fixed() and self.action_sampler.is_fixed()

    def get_var(self, var_type: str, *args, **kwargs) -> Var:
        if var_type in self.non_action_var_types():
            return self.primitive_sampler.get_var(var_type)
        return self.action_sampler.get_var(var_type, *args, **kwargs)

    def get_action(self, action_var_type: str, action_idx_or_action_func) -> Action:
        return self.action_sampler.get_action(action_var_type, action_idx_or_action_func)

    def non_action_var_types(self):
        return set(self.primitive_sampler.keys())

    def action_var_types(self):
        return set(self.action_sampler.keys())

    def model_action_var_types(self):
        return self.action_sampler.model_var_types

    def sig_var_types(self, action_var_type: str=None, action_idx_or_action_func=None):
        if action_var_type is None and action_idx_or_action_func is None:
            return self._sig_var_types
        elif action_var_type is None or action_idx_or_action_func is None:
            raise KeyError("Either 'action_var_type' and 'action_idx_or_action_func' must be provided or both must be None.")
        return set(self.get_action(action_var_type, action_idx_or_action_func).keys())

    def var_types(self):
        return set(self.primitive_sampler.keys()) | set(self.action_sampler.keys())

    def contain_idx(self, *args, idx):
        return self.get_var(*args).contain_idx(idx)

    def convert_idx_to_val(self, *args, idx):
        return self.get_var(*args).convert_idx_to_val(idx)

    def contain_val(self, *args, val):
        return self.get_var(*args).contain_val(val)

    def convert_val_to_idx(self, *args, val):
        return self.get_var(*args).convert_val_to_idx(val)

    def sample(self, var_type: str, trial: Trial, default_val = None):
        """
        If trial == Trial, Then expected behavior
        IF trial == None, Then default_val will be returned.
        If trial == FrozenTrial, Then special retrieval.
        """
        if var_type in self._sig_var_types:
            raise KeyError(f"Requested var_type: {var_type} is a signature var_type, which is not allowed to be sampled directly.")
        if var_type not in self.var_types():
            raise KeyError(f"Requested var_type: {var_type} does not exist in the CombinedSampler.")
        var = self.get_var(var_type)
        valid_trial = isinstance(trial, (Trial, FrozenTrial))

        #Any 'default_val' must be in the current distribution of the var_type
        if not valid_trial and not var.contain_val(default_val):
            raise ValueError(f"Var of type: {var_type} cannot produce the given 'default_val': {default_val} since its current distribution is: {var.distribution()}")
        if not valid_trial:
            #Return 'default_val' if 'trial' is invalid but 'default_val' is valid
            return default_val

        sample_from = self.action_sampler if var_type in self.action_var_types() else self.primitive_sampler
        return sample_from.sample(var_type, trial)

    def update(self, top_k_trials: list, update_state_window: int, device: torch.device):
        #Empty list for top_k_trials is not allowed. (Auto-HPO should have been finished before any empty-update is justifiable).
        if not len(top_k_trials):
            raise ValueError("Input 'top_k_trials' cannot be empty")

        #Convert trial params to the required format
        list_params = [{tuple(key.split('>')): val for key, val in trial.params.items()} for trial in top_k_trials]

        #Update primitive_sampler
        self.primitive_sampler.update_vars(list_params=list_params)

        #Update list_params by evaluating non-action sampled idx(s) to value(s).
        list_params = [params|self.primitive_sampler.get_evaluated_params(source=params) for params in list_params]

        #Update action_sampler
        self.action_sampler.update(
            list_params=list_params,
            update_state_window=update_state_window,
            device=device
        )

#Test: CombinedSampler



In [20]:
if __name__ == '__main__':
    class UserObjectiveCombinedSampler:
        def objective(self, turn_off_training_log: bool):
            model = self.default_dict['model_action'](device=self.device)

            for param in model.parameters():
                param.data.fill_(0)

            model_container = Mock(spec=Learner)
            model_container.model = model

            return (0, model)

    #Sample model classes
    class ModelDefaultCombinedSampler(nn.Module):
        def __init__(self, rand_number: int):
            super().__init__()
            self.net = nn.Linear(16, 10)
            self.rand_number = torch.tensor(rand_number)

        def forward(self, x):
            return self.net(x) + self.rand_number

    class ModelDropoutCombinedSampler(nn.Module):
        def __init__(self, activation: nn.Module, dropout_p: float):
            super().__init__()
            self.activation = activation
            self.dropout_p = dropout_p
            self.net = nn.Sequential(
                nn.Linear(16, 10),
                activation(),
                nn.Dropout(p=dropout_p),
                nn.Linear(10, 10)
            )

        def forward(self, x):
            return self.net(x)

    class ModelDropoutCombinedSamplerVariant(nn.Module):
        def __init__(self, rand_number: int, activation: nn.Module, dropout_p: int):
            super().__init__()
            self.rand_number = rand_number
            self.net = nn.Sequential(
                nn.Linear(in_features=16, out_features=10),
                activation(),
                nn.Dropout(p=dropout_p),
                nn.Linear(in_features=10, out_features=10)
            )

        def forward(self, x):
            return self.net(x)

    #Action functions
    def model_action_default_combined_sampler(rand_number: int, device: torch.device):
        return ModelDefaultCombinedSampler(rand_number=rand_number).to(device)

    def model_action_dropout_combined_sampler(activation, dropout_p, device: torch.device):
        return ModelDropoutCombinedSampler(activation=activation, dropout_p=dropout_p).to(device)

    def model_action_dropout_combined_sampler_variant(rand_number: int, activation, dropout_p, device: torch.device):
        return ModelDropoutCombinedSamplerVariant(rand_number=rand_number, activation=activation, dropout_p=dropout_p).to(device)

    def dls_action_default_combined_sampler(batch_size: int, device: torch.device):
        return {'batch_size': batch_size, 'device': device}

    def dls_action_normalize_combined_sampler(batch_size: int, normalize: bool):
        return {'batch_size': batch_size, 'normalize': normalize}

    def dls_action_activation_combined_sampler(batch_size: int, normalize: bool, activation: nn.Module):
        return {'batch_size': batch_size, 'normalize': normalize, 'activation': activation}

class TestCombinedSampler(unittest.TestCase):
    def setUp(self):
        #Initialize ConfigSpace
        self.cs = ConfigSpace()
        self.user_objective = UserObjectiveCombinedSampler()
        setattr(ObjectiveExecutor, 'execute_objective', self.user_objective.objective.__func__)

        #Define action hyperparameters
        self.cs.set_categorical(
            var_type='model_action',
            choices=[model_action_default_combined_sampler, model_action_dropout_combined_sampler, model_action_dropout_combined_sampler_variant],
            action=True,
            save_state=True
        )
        self.cs.set_categorical(
            var_type='dls_action',
            choices=[dls_action_default_combined_sampler, dls_action_normalize_combined_sampler, dls_action_activation_combined_sampler],
            action=True
        )

        # Define signature parameters for 'model_action'
        self.cs.set_categorical(
            var_type='rand_number',
            choices=list(range(1, 11)),
            belongs_to='model_action'
        )
        self.cs.set_categorical(
            var_type='activation',
            choices=[nn.ReLU, nn.PReLU, nn.SiLU],
            belongs_to='model_action'
        )
        self.cs.set_float(
            var_type='dropout_p',
            low=0.0,
            high=1.0,
            belongs_to='model_action'
        )

        #Define signature parameters for 'dls_action'
        self.cs.set_int(
            var_type='batch_size',
            low=16,
            high=64,
            belongs_to='dls_action'
        )
        self.cs.set_categorical(
            var_type='normalize',
            choices=[True, False],
            belongs_to='dls_action'
        )
        self.cs.set_categorical(
            var_type='activation',
            choices=[nn.ReLU, nn.SiLU],
            belongs_to='dls_action'
        )

        #Define non-action variables
        self.cs.set_float(
            var_type='lr',
            low=0.0001,
            high=0.01,
            log=True
        )
        self.cs.set_float(
            var_type='wd',
            low=0.0001,
            high=0.1,
            log=True
        )

        #Initialize CombinedSampler with ConfigSpace
        self.combined_sampler = CombinedSampler(self.cs)

    def create_mock_trial(self, params):
        trial = Mock(spec=optuna.trial.FrozenTrial)
        trial.params = params
        return trial

    def test_initialization(self):
        #Test that CombinedSampler initializes correctly
        self.assertIsInstance(self.combined_sampler.action_sampler, ActionSampler)
        self.assertIsInstance(self.combined_sampler.primitive_sampler, PrimitiveSampler)

        #Check var_types retrieval/categorization.
        self.assertEqual(self.combined_sampler.action_var_types(), {'model_action', 'dls_action'})
        self.assertEqual(self.combined_sampler.non_action_var_types(), {'lr', 'wd'})
        self.assertEqual(self.combined_sampler.sig_var_types(), {'rand_number', 'batch_size', 'normalize', 'activation', 'dropout_p'})
        self.assertEqual(self.combined_sampler.var_types(), {'model_action', 'dls_action', 'lr', 'wd'})

    def test_sample_non_action_vars(self):
        trial = optuna.create_study(direction='minimize').ask()
        lr = self.combined_sampler.sample('lr', trial)
        wd = self.combined_sampler.sample('wd', trial)
        #Assert that the sampled values are within the expected ranges
        self.assertTrue(0.0001 <= lr <= 0.01)
        self.assertTrue(0.0001 <= wd <= 0.1)

    def test_sample_model_actions(self):
        trial = optuna.create_study(direction='minimize').ask()
        model_action = self.combined_sampler.sample('model_action', trial)
        self.assertIsInstance(model_action, ModelAction)
        self.assertIsNotNone(model_action.params)
        self.assertIsNotNone(model_action._model_loader)
        self.assertEqual(
            model_action.params,
            model_action._model_loader.keywords,
            msg=f"model_action.params: {model_action.params} != model_action._model_loader.keywores: {model_action._model_loader.keywords }"
        )
        #Now check which action_func was sampled and assert accordingly
        if model_action.action_func == model_action_default_combined_sampler:
            rand_number = model_action.params['rand_number']
            self.assertIn(rand_number, list(range(1, 11)))
        elif model_action.action_func == model_action_dropout_combined_sampler:
            activation = model_action.params['activation']
            dropout_p = model_action.params['dropout_p']
            self.assertIn(activation, [nn.ReLU, nn.PReLU, nn.SiLU])
            self.assertTrue(0.0 <= dropout_p <= 1.0)
        elif model_action.action_func == model_action_dropout_combined_sampler_variant:
            rand_number = model_action.params['rand_number']
            activation = model_action.params['activation']
            dropout_p = model_action.params['dropout_p']
            self.assertIn(rand_number, list(range(1, 11)))
            self.assertIn(activation, [nn.ReLU, nn.PReLU, nn.SiLU])
            self.assertTrue(0.0 <= dropout_p <= 1.0)
        else:
            raise ValueError(f"Unexpected model_action action_func: {model_action.action_func}")

    def test_sample_dls_actions(self):
        trial = optuna.create_study(direction='minimize').ask()
        dls_action = self.combined_sampler.sample('dls_action', trial)
        self.assertIsInstance(dls_action, Action)
        self.assertFalse(isinstance(dls_action, ModelAction))
        self.assertIsNotNone(dls_action.params)
        self.assertIsNotNone(dls_action._model_loader)
        self.assertEqual(
            dls_action.params,
            dls_action._model_loader.keywords
        )
        #Now check which action_func was sampled and assert accordingly
        if dls_action.action_func == dls_action_default_combined_sampler:
            batch_size = dls_action.params['batch_size']
            self.assertTrue(16 <= batch_size <= 64)
        elif dls_action.action_func == dls_action_normalize_combined_sampler:
            batch_size = dls_action.params['batch_size']
            normalize = dls_action.params['normalize']
            self.assertTrue(16 <= batch_size <= 64)
            self.assertIn(normalize, [True, False])
        elif dls_action.action_func == dls_action_activation_combined_sampler:
            batch_size = dls_action.params['batch_size']
            normalize = dls_action.params['normalize']
            activation = dls_action.params['activation']
            self.assertTrue(16 <= batch_size <= 64)
            self.assertIn(normalize, [True, False])
            self.assertIn(activation, [nn.ReLU, nn.SiLU])
        else:
            raise ValueError(f"Unexpected dls_action action_func: {dls_action.action_func}")

    def test_update(self):
        #Prepare mock trials with various combinations
        trial1_params = {
            'model_action>None': 0,  #model_action_default_combined_sampler
            'rand_number>model_action': 5, #6

            'dls_action>None': 1,  #dls_action_normalize_combined_sampler
            'batch_size>dls_action': 32,
            'normalize>dls_action': 0,  #True

            'lr>None': 0.005,
            'wd>None': 0.001
        }
        trial2_params = {
            'model_action>None': 1,  #model_action_dropout_combined_sampler
            'activation>model_action': 2,  #nn.SiLU
            'dropout_p>model_action': 0.3,

            'dls_action>None': 2,  #dls_action_activation_combined_sampler
            'batch_size>dls_action': 48,
            'normalize>dls_action': 1,  #False
            'activation>dls_action': 1,  #nn.SiLU

            'lr>None': 0.002,
            'wd>None': 0.01
        }
        trial3_params = {
            'model_action>None': 2,  #model_action_dropout_combined_sampler_variant
            'rand_number>model_action': 7, #8
            'activation>model_action': 0,  #nn.ReLU
            'dropout_p>model_action': 0.5,

            'dls_action>None': 0,  #dls_action_default_combined_sampler
            'batch_size>dls_action': 64,

            'lr>None': 0.001,
            'wd>None': 0.005
        }
        #Create trials
        trial1 = self.create_mock_trial(trial1_params)
        trial2 = self.create_mock_trial(trial2_params)
        trial3 = self.create_mock_trial(trial3_params)
        top_k_trials = [trial1, trial2, trial3]
        #Call update with update_state_window=2
        self.combined_sampler.update(top_k_trials=top_k_trials, update_state_window=2, device=torch.device('cpu'))
        # Now verify that the variables and actions have been updated correctly
        # For non-action variables
        lr_var = self.combined_sampler.get_var('lr')
        wd_var = self.combined_sampler.get_var('wd')
        #Note that it reflects ALL the 'k' number of params in top_k_trials
        self.assertEqual(lr_var._dist['low'], 0.001)
        self.assertEqual(lr_var._dist['high'], 0.005)
        self.assertEqual(wd_var._dist['low'], 0.001)
        self.assertEqual(wd_var._dist['high'], 0.01)
        #For model_action
        model_action_var = self.combined_sampler.action_sampler.get_var('model_action')
        self.assertEqual(model_action_var.distribution(indices=True), [0, 1, 2])
        #Check action_states
        model_action_default_combined_sampler = model_action_var.get_action(0)
        model_action_dropout_combined_sampler = model_action_var.get_action(1)
        model_action_dropout_combined_sampler_variant = model_action_var.get_action(2)
        self.assertIsNotNone(model_action_default_combined_sampler.action_state)
        self.assertIsNotNone(model_action_dropout_combined_sampler.action_state)
        self.assertIsNone(model_action_dropout_combined_sampler_variant.action_state)
        #For dls_action(action_state should not be updated)
        dls_action_var = self.combined_sampler.action_sampler.get_var('dls_action')
        for idx in [0, 1, 2]:
            action = dls_action_var.get_action(idx)
            self.assertIsNone(action.action_state)

    def test_sampling_after_update(self):
        #Prepare and update as before
        self.test_update()
        self.assertEqual(self.combined_sampler.get_var('model_action').distribution(True), [0,1,2])
        self.assertEqual(self.combined_sampler.get_var('dls_action').distribution(True), [0,1,2])
        #After updating, sample variables and check that distributions are updated
        trial = optuna.create_study(direction='minimize').ask()
        for var_type in self.combined_sampler.var_types():
            val = self.combined_sampler.sample(var_type, trial)
            if var_type=='model_action':
                self.assertIsInstance(val, ModelAction)
                self.assertIn(val.action_func, [model_action_default_combined_sampler, model_action_dropout_combined_sampler, model_action_dropout_combined_sampler_variant])
                self.assertEqual(val(device=None)(torch.randn(16,16)).shape, torch.Size([16,10]))
            elif var_type=='dls_action':
                self.assertIsInstance(val, Action)
                self.assertIn(val.action_func, [dls_action_default_combined_sampler, dls_action_normalize_combined_sampler, dls_action_activation_combined_sampler])
            elif var_type=='lr':
                self.assertTrue(0.001 <= val <= 0.005)
            elif var_type=='wd':
                self.assertTrue(0.001 <= val <= 0.01)

    def test_get_evaluated_params(self):
        #Prepare a source with both action and non-action parameters
        source_params = {
            ('model_action', 'None'): 2,  # model_action_dropout_combined_sampler_variant
            ('rand_number', 'model_action'): 7, #8
            ('activation', 'model_action'): 1,  # nn.PReLU
            ('dropout_p', 'model_action'): 0.4,

            ('dls_action', 'None'): 2,  # dls_action_activation_combined_sampler
            ('batch_size', 'dls_action'): 48,
            ('normalize', 'dls_action'): 1,  # False
            ('activation', 'dls_action'): 0,  # nn.ReLU

            ('lr', 'None'): 0.001,
            ('wd', 'None'): 0.005
        }
        #Get evaluated params from primitive_sampler
        evaluated_params = self.combined_sampler.primitive_sampler.get_evaluated_params(source_params)
        expected_params = {
            ('lr', 'None'): 0.001,
            ('wd', 'None'): 0.005
        }
        self.assertEqual(evaluated_params, expected_params)
        #Get evaluated actions from action_sampler
        evaluated_actions = self.combined_sampler.action_sampler.get_evaluated_params(source_params, exclude=[])
        #Check that both model_action and dls_action are in evaluated_actions
        self.assertIn(('model_action', 'None'), evaluated_actions)
        self.assertIn(('dls_action', 'None'), evaluated_actions)
        model_action = evaluated_actions[('model_action', 'None')]
        dls_action = evaluated_actions[('dls_action', 'None')]
        self.assertIsInstance(model_action, ModelAction)
        self.assertIsInstance(dls_action, Action)
        #Check model_action._model_loader parameters
        model_action_loader_params = model_action._model_loader.keywords
        self.assertEqual(model_action_loader_params['rand_number'], 8)
        self.assertEqual(
            model_action_loader_params['activation'],
            nn.PReLU,
            msg=f"model_action: {model_action.action_func}, model_action_loader_params: {model_action_loader_params}")
        self.assertEqual(model_action_loader_params['dropout_p'], 0.4)
        #Check dls_action parameters
        dls_model_loader_params = dls_action._model_loader.keywords
        self.assertEqual(dls_model_loader_params['batch_size'], 48)
        self.assertEqual(dls_model_loader_params['normalize'], False)
        self.assertEqual(
            dls_model_loader_params['activation'],
            nn.ReLU,
            msg=f"dls_action: {dls_action.action_func}, dls_model_loader_params: {dls_model_loader_params}")

    def test_combination_of_actions(self):
        #Test sampling multiple times to cover combinations
        for _ in range(10):
            trial = optuna.create_study(direction='minimize').ask()
            model_action = self.combined_sampler.sample('model_action', trial)
            dls_action = self.combined_sampler.sample('dls_action', trial)
            lr = self.combined_sampler.sample('lr', trial)
            wd = self.combined_sampler.sample('wd', trial)

            #Check types
            self.assertIsInstance(model_action, ModelAction)
            self.assertIsInstance(dls_action, Action)
            self.assertTrue(0.0001 <= lr <= 0.01)
            self.assertTrue(0.0001 <= wd <= 0.1)

            #Check model_action parameters
            if model_action.action_func == model_action_default_combined_sampler:
                rand_number = model_action.params['rand_number']
                self.assertIn(rand_number, list(range(1, 11)))
            elif model_action.action_func == model_action_dropout_combined_sampler:
                activation = model_action.params['activation']
                dropout_p = model_action.params['dropout_p']
                self.assertIn(activation, [nn.ReLU, nn.PReLU, nn.SiLU])
                self.assertTrue(0.0 <= dropout_p <= 1.0)
            elif model_action.action_func == model_action_dropout_combined_sampler_variant:
                rand_number = model_action.params['rand_number']
                activation = model_action.params['activation']
                dropout_p = model_action.params['dropout_p']
                self.assertIn(rand_number, list(range(1, 11)))
                self.assertIn(activation, [nn.ReLU, nn.PReLU, nn.SiLU])
                self.assertTrue(0.0 <= dropout_p <= 1.0)
            else:
                raise ValueError(f"Unexpected model_action action_func: {model_action.action_func}")

            #Check dls_action parameters similarly
            if dls_action.action_func == dls_action_default_combined_sampler:
                batch_size = dls_action.params['batch_size']
                self.assertTrue(16 <= batch_size <= 64)
            elif dls_action.action_func == dls_action_normalize_combined_sampler:
                batch_size = dls_action.params['batch_size']
                normalize = dls_action.params['normalize']
                self.assertTrue(16 <= batch_size <= 64)
                self.assertIn(normalize, [True, False])
            elif dls_action.action_func == dls_action_activation_combined_sampler:
                batch_size = dls_action.params['batch_size']
                normalize = dls_action.params['normalize']
                activation = dls_action.params['activation']
                self.assertTrue(16 <= batch_size <= 64)
                self.assertIn(normalize, [True, False])
                self.assertIn(activation, [nn.ReLU, nn.SiLU])
            else:
                raise ValueError(f"Unexpected dls_action action_func: {dls_action.action_func}")

    def test_invalid_variable_type(self):
        trial = optuna.create_study(direction='minimize').ask()
        with self.assertRaises(KeyError):
            self.combined_sampler.sample('invalid_var_type', trial)

    def test_invalid_default_value(self):
        with self.assertRaises(ValueError):
            self.combined_sampler.sample(
                var_type='lr',
                trial=None,
                #Throw in a value out of range
                default_val=self.combined_sampler.get_var('lr').distribution()['high']+10.0
            )

    def test_update_with_no_trials(self):
        with self.assertRaises(ValueError):
            self.combined_sampler.update(top_k_trials=[], update_state_window=1, device=torch.device('cpu'))

    def test_update_with_large_window(self):
        #Prepare one trial
        trial_params = {
            'model_action>None': 0,  #model_action_default_combined_sampler
            'rand_number>model_action': 5, #6
            'lr>None': 1e-4,
            'wd>None': 1e-3
        }
        trial = self.create_mock_trial(trial_params)
        #Call update with update_state_window larger than number of trials
        #Ensure that update_state_window does not cause errors
        self.combined_sampler.update(top_k_trials=[trial], update_state_window=5, device=torch.device('cpu'))

        #Check that the (primitive)variables are updated based on the trial
        lr_var = self.combined_sampler.get_var('lr')
        self.assertEqual(lr_var._dist['low'], 1e-4)
        self.assertEqual(lr_var._dist['high'], 1e-4)
        wd_var = self.combined_sampler.get_var('wd')
        self.assertEqual(wd_var._dist['low'], 1e-3)
        self.assertEqual(wd_var._dist['high'], 1e-3)

        #Check that the ModelAction(and its signature Vars) are updated properly
        model_action_var = self.combined_sampler.action_sampler.get_var('model_action')
        self.assertEqual(model_action_var.distribution(indices=True), [0])
        rand_number_var = model_action_var.get_var(model_action_default_combined_sampler,'rand_number')
        self.assertEqual(rand_number_var.distribution(), [6])

    def test_missing_action_variable_in_trial(self):
        trial_params = {
            'lr>None': 0.005,
            'wd>None': 0.001
            # 'model_action-None' is missing
        }
        trial = self.create_mock_trial(trial_params)

        #Call update and expect that it handles missing action vars
        self.combined_sampler.update(top_k_trials=[trial], update_state_window=2, device=torch.device('cpu'))

        #Check that non-action vars are updated
        lr_var = self.combined_sampler.get_var('lr')
        self.assertEqual(lr_var._dist['low'], 0.005)
        self.assertEqual(lr_var._dist['high'], 0.005)
        wd_var = self.combined_sampler.get_var('wd')
        self.assertEqual(wd_var._dist['low'], 0.001)
        self.assertEqual(wd_var._dist['high'], 0.001)

        #Check that action vars are not updated
        model_action_var = self.combined_sampler.action_sampler.get_var('model_action')
        #Since no action indices were provided, the distribution should remain the same
        self.assertEqual(model_action_var.distribution(indices=True), [0, 1, 2])

    def test_trial_with_unknown_variables(self):
        trial_params = {
            'model_action>None': 0,
            'rand_number>model_action': 5,
            'unknown_var>None': 42,
            'lr>None': 0.005,
            'wd>None': 0.001
        }
        trial = self.create_mock_trial(trial_params)

        #Call update and expect that it ignores unknown variables
        self.combined_sampler.update(top_k_trials=[trial], update_state_window=1, device=torch.device('cpu'))

        #Ensure that known variables are updated
        lr_var = self.combined_sampler.get_var('lr')
        self.assertEqual(lr_var._dist['low'], 0.005)
        self.assertEqual(lr_var._dist['high'], 0.005)
        wd_var = self.combined_sampler.get_var('wd')
        self.assertEqual(wd_var._dist['low'], 0.001)
        self.assertEqual(wd_var._dist['high'], 0.001)

        #Check that the ModelAction(and its signature Vars) are updated properly
        model_action_var = self.combined_sampler.action_sampler.get_var('model_action')
        self.assertEqual(model_action_var.distribution(indices=True), [0])
        rand_number_var = model_action_var.get_var(model_action_default_combined_sampler,'rand_number')
        self.assertEqual(rand_number_var.distribution(), [6])

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

..[I 2024-10-13 21:17:17,515] A new study created in memory with name: no-name-6806f627-f3c3-48c7-b438-a7212863b101
[I 2024-10-13 21:17:17,521] A new study created in memory with name: no-name-314a8cdd-edb5-4e85-b41e-38a935c19177
.................[I 2024-10-13 21:17:17,573] A new study created in memory with name: no-name-5f0e885d-02c6-402b-8571-f710ddbad7c8
[I 2024-10-13 21:17:17,576] A new study created in memory with name: no-name-3a603ba4-2a96-44cd-8be1-d05765261c93
[I 2024-10-13 21:17:17,578] A new study created in memory with name: no-name-1f80a813-9ef5-45f0-8390-d7cdb052958f
[I 2024-10-13 21:17:17,580] A new study created in memory with name: no-name-5785c6d7-dc75-427f-9eee-b3d8ce0949d8
[I 2024-10-13 21:17:17,586] A new study created in memory with name: no-name-4c4c9feb-4b3d-486c-b292-044e3365b188
[I 2024-10-13 21:17:17,589] A new study created in memory with name: no-name-d0ec7db4-6362-47bf-bb35-d0455746715c
[I 2024-10-13 21:17:17,591] A new study created in memory with name: 

All good

Missing signature parameters in model_action: {'dropout_p', 'rand_number', 'activation'} 

Missing signature parameters in model_action: {'dropout_p', 'rand_number', 'activation'} 
Missing signature parameters in dls_action: {'normalize', 'activation', 'batch_size'} 

Missing signature parameters in model_action: {'dropout_p', 'activation'} 
Missing signature parameters in dls_action: {'normalize', 'activation', 'batch_size'} 

Missing signature parameters in model_action: {'dropout_p'} 
Missing signature parameters in dls_action: {'normalize', 'activation', 'batch_size'} 

Missing signature parameters in dls_action: {'normalize', 'activation', 'batch_size'} 

Missing signature parameters in dls_action: {'normalize', 'activation'} 

Missing signature parameters in dls_action: {'activation'} 

All good

All good

All good

Missing signature parameters in model_action: {'dropout_p', 'rand_number', 'activation'} 

Missing signature parameters in model_action: {'dropout_p', 'rand

..[I 2024-10-13 21:17:17,619] A new study created in memory with name: no-name-b852a13d-93bb-4c2f-9818-b97193d753f2
..[I 2024-10-13 21:17:17,639] A new study created in memory with name: no-name-1526c7d9-e755-4627-8807-6044da108625
.[I 2024-10-13 21:17:17,647] A new study created in memory with name: no-name-6bd6ac83-c2b3-4d82-b18b-c50b63d414e3
.[I 2024-10-13 21:17:17,652] A new study created in memory with name: no-name-de8d8162-31a3-449f-8088-0f5bae68123f
.[I 2024-10-13 21:17:17,672] A new study created in memory with name: no-name-b05eb09f-bcc3-4221-8085-2b88f9c1f704
........

All good

All good

All good

All good

Missing signature parameters in model_action: {'dropout_p', 'rand_number', 'activation'} 

Missing signature parameters in model_action: {'dropout_p', 'rand_number', 'activation'} 
Missing signature parameters in dls_action: {'normalize', 'activation', 'batch_size'} 

Missing signature parameters in model_action: {'dropout_p', 'activation'} 
Missing signature parameters in dls_action: {'normalize', 'activation', 'batch_size'} 

Missing signature parameters in model_action: {'dropout_p'} 
Missing signature parameters in dls_action: {'normalize', 'activation', 'batch_size'} 

Missing signature parameters in dls_action: {'normalize', 'activation', 'batch_size'} 

Missing signature parameters in dls_action: {'normalize', 'activation'} 

Missing signature parameters in dls_action: {'activation'} 

All good

All good

All good

All good

All good

All good

Missing signature parameters in model_action: {'dropout_p', 'rand_number', 'activation'} 

Missi

...............................................[I 2024-10-13 21:17:17,789] A new study created in memory with name: no-name-1a55c863-ea7d-4ce4-8392-f1aa35b23517
.[I 2024-10-13 21:17:17,802] A new study created in memory with name: no-name-04de6d5a-cd77-4c30-b261-8b2633f453cf
.[I 2024-10-13 21:17:17,805] A new study created in memory with name: no-name-f31f0d65-d46b-40bf-bc0e-be48556f8bc9
.[I 2024-10-13 21:17:17,809] A new study created in memory with name: no-name-f3b17b84-de6f-497d-a270-94af2dfaf99d
.

...

[I 2024-10-13 21:17:17,814] A new study created in memory with name: no-name-54988265-b264-4ba6-b4b3-e6c13ec77ccc
.[I 2024-10-13 21:17:17,818] A new study created in memory with name: no-name-053146ed-2323-4b2d-88d0-618bf1259ad6
.[I 2024-10-13 21:17:17,820] A new study created in memory with name: no-name-b72d0a46-de9b-432e-9255-c51dc300a418
.[I 2024-10-13 21:17:17,836] A new study created in memory with name: no-name-4983576c-fd8e-46d7-8e87-abcc3cf5a7b3


Missing signature parameters in model_action: {'dropout_p', 'activation', 'start_neurons', 'preprocess'} 

Missing signature parameters in model_action: {'dropout_p', 'activation', 'start_neurons', 'preprocess'} 
Missing signature parameters in dls_action: {'batch_size'} 

Missing signature parameters in model_action: {'dropout_p', 'activation', 'start_neurons', 'preprocess'} 
Missing signature parameters in dls_action: {'batch_size'} 

Missing signature parameters in model_action: {'dropout_p', 'activation', 'start_neurons', 'preprocess'} 
Missing signature parameters in dls_action: {'batch_size'} 

Missing signature parameters in model_action: {'dropout_p', 'activation', 'start_neurons', 'preprocess'} 
Missing signature parameters in dls_action: {'batch_size'} 

Missing signature parameters in model_action: {'dropout_p', 'activation', 'start_neurons', 'preprocess'} 
Missing signature parameters in dls_action: {'batch_size'} 

Missing signature parameters in model_action: {'dropout_p'

epoch,train_loss,valid_loss,custom_mse,time
0,4507174548725905680760832.000000,681844375267974958921551181774848.000000,681844375267974958921551181774848.000000,00:00
1,4344166147723681255653376.000000,681844297896722503585284000579584.000000,681844297896722503585284000579584.000000,00:00
2,4400551215058359865573376.000000,681844375267974958921551181774848.000000,681844375267974958921551181774848.000000,00:00
3,4302481406033492248100864.000000,681844375267974958921551181774848.000000,681844375267974958921551181774848.000000,00:00
4,4263848447565997564493824.000000,681844297896722503585284000579584.000000,681844297896722503585284000579584.000000,00:00


...........[I 2024-10-13 21:17:20,107] A new study created in memory with name: no-name-5d45ca8c-dc92-4e65-8271-fedab8584983
..[I 2024-10-13 21:17:20,116] A new study created in memory with name: no-name-2e57fb4d-2a48-4484-b32d-eddc2baee2f2
.....
----------------------------------------------------------------------
Ran 108 tests in 2.623s

OK


Dynamic verification passed successfully
.


#Test: ObjectiveExecutor

In [21]:
if __name__ == '__main__':
    from typing import Dict, Any
    from torch.utils.data import Dataset, DataLoader
    class UserObjectiveForTestingObjectiveExecutor(UserObjective):
        def __init__(self):
            super().__init__()

    class PolynomialDataset(Dataset):
        def __init__(self, coeffs: torch.tensor, input_min: float, input_max: float, input_size: int):
            self.coeffs = coeffs.clone().detach()
            self.inputs = torch.empty(size=(input_size,)).uniform_(input_min, input_max)
            self.outputs = self._f(X=self.inputs)

        def __getitem__(self, idx):
            return self.inputs[idx], self.outputs[idx]

        def __len__(self):
            return self.inputs.size(0)

        def _f(self, X):
            powers = torch.arange(self.coeffs.size(0)).unsqueeze(0)
            Y = self.inputs.unsqueeze(-1)**powers
            return torch.matmul(Y, self.coeffs).squeeze(-1)

    degree = 6
    coeffs_min, coeffs_max = -10.0, 10.0
    coeffs = torch.empty(size=(degree+1,)).uniform_(coeffs_min, coeffs_max)

    train_dset = PolynomialDataset(coeffs=coeffs, input_min=-100.0, input_max=100.0, input_size=8000)
    valid_dset = PolynomialDataset(coeffs=coeffs, input_min=-300.12, input_max=500.78, input_size=2000)

    class ModelDefaultObjectiveExecutor(nn.Module):
        def __init__(self, activation: nn.Module, preprocess: bool, start_neurons: int):
            super().__init__()
            self.preprocess = preprocess
            if self.preprocess:
                start_neurons = max(start_neurons, degree+1)
            self.net = nn.Sequential(
                nn.Linear(in_features=degree+1 if self.preprocess else 1, out_features=start_neurons),
                nn.BatchNorm1d(num_features=start_neurons),
                activation(),
                nn.Linear(in_features=start_neurons, out_features=start_neurons*2),
                nn.BatchNorm1d(num_features=start_neurons*2),
                activation(),
                nn.Linear(in_features=start_neurons*2, out_features=1)
            )

        def _preprocess(self, X):
            N = X.size(0)
            X = X.unsqueeze(-1)**torch.arange(degree+1).unsqueeze(0)
            return X

        def forward(self, X):
            if X.dim() != 1:
                X = X.reshape(-1)

            if self.preprocess:
                X = self._preprocess(X)
            else:
                X = X.unsqueeze(-1)
            return self.net(X)

    class ModelDropoutObjectiveExecutor(nn.Module):
        def __init__(self, activation: nn.Module, dropout_p: float, preprocess: bool, start_neurons: int):
            super().__init__()
            self.preprocess = preprocess
            if self.preprocess:
                start_neurons = max(start_neurons, degree+1)
            self.net = nn.Sequential(
                nn.Linear(in_features=degree+1 if self.preprocess else 1, out_features=start_neurons),
                nn.BatchNorm1d(num_features=start_neurons),
                activation(),
                nn.Dropout(p=dropout_p),
                nn.Linear(in_features=start_neurons, out_features=start_neurons*2),
                nn.BatchNorm1d(num_features=start_neurons*2),
                activation(),
                nn.Linear(in_features=start_neurons*2, out_features=1)
            )

        def _preprocess(self, X):
            N = X.size(0)
            X = X.unsqueeze(-1)**torch.arange(degree+1).unsqueeze(0)
            return X

        def forward(self, X):
            if X.dim() != 1:
                X = X.reshape(-1)

            if self.preprocess:
                X = self._preprocess(X)
            else:
                X = X.unsqueeze(-1)
            return self.net(X)

    def model_action_default_objective_executor(activation: nn.Module, preprocess: bool, start_neurons: int, device: torch.device):
        if start_neurons < 1:
            raise ValueError(f"start_neurons MUST be a positive integer but found start_neurons: {start_neurons}")
        return ModelDefaultObjectiveExecutor(activation=activation,
                            preprocess=preprocess,
                            start_neurons=start_neurons).to(device)

    def model_action_dropout_objective_executor(activation: nn.Module, dropout_p: float, preprocess: bool, start_neurons: int, device: torch.device):
        if start_neurons < 1:
            raise ValueError(f"start_neurons MUST be a positive integer but found start_neurons: {start_neurons}")
        if dropout_p < 0.0 or dropout_p >= 1.0:
            raise ValueError(f"0.0 <= dropout_p < 1.0 required but found dropout_p: {dropout_p}")
        return ModelDropoutObjectiveExecutor(activation=activation,
                            dropout_p=dropout_p,
                            preprocess=preprocess,
                            start_neurons=start_neurons).to(device)

    def dls_action_default_objective_executor(batch_size: int, device=None):
        train_dataloader = DataLoader(train_dset, batch_size=batch_size, shuffle=True)
        valid_dataloader = DataLoader(valid_dset, batch_size=batch_size, shuffle=True)
        dls = DataLoaders(train_dataloader, valid_dataloader)
        return dls

    def custom_mse(Y_hat, Y):
        Y_hat = Y_hat.reshape(-1)
        Y = Y.reshape(-1)
        return nn.MSELoss()(Y_hat, Y)

class TestObjectiveExecutor(unittest.TestCase):
    def setUp(self):
        self.INIT_DICT = {
            ('dls_action', None): {
                'params': {'choices': [dls_action_default_objective_executor]},
                'sample': 'categorical',
                'action': True

            },
            ('freeze', None): {
                'params': {'choices': [True, False]},
                'sample': 'categorical'
            },
            ('lr_low', None): {
                'params': {'low': 1e-7, 'high': 9e-4, 'log': True},
                'sample': 'float'
            },
            ('lr_high', None): {
                'params': {'low': 9e-4, 'high': 1e-1, 'log': True},
                'sample': 'float'
            },
            ('lr', None): {
                'params': {'low': 1e-6, 'high': 1e-1, 'log': True},
                'sample': 'float'
            },
            ('wd', None): {
                'params': {'choices': [1e-4, 1e-3, 1e-2, 1e-1]},
                'sample': 'categorical'
            },
            ('gradient_clip', None): {
                'params': {'choices': [True, False]},
                'sample': 'categorical'
            },
            ('max_norm', None): {
                'params': {'low': 0.0, 'high': 15.0, 'log': False},
                'sample': 'float'
            },
            ('one_cycle', None): {
                'params': {'choices': [True, False]},
                'sample': 'categorical'
            },
            ('pct_start', None): {
                'params': {'low': 0.10, 'high': 0.95, 'log': False, 'step': 0.05},
                'sample': 'float'
            },
            ('n_epoch', None): {
                'params': {'choices': [5, 10, 15]},
                'sample': 'categorical'
            },
            ('model_action', None): {
                'params': {'choices': [model_action_default_objective_executor, model_action_dropout_objective_executor]},
                'sample': 'categorical',
                'action': True,
                'save_state': True
            },
            ('activation', 'model_action'): {
                'params': {'choices': [nn.ReLU, nn.PReLU, nn.SiLU]},
                'sample': 'categorical'
            },
            ('loss_func', None): {
                'params': {'choices': [custom_mse]},
                'sample': 'categorical'
            },
            ('metric', None): {
                'params': {'choices': [custom_mse]},
                'sample': 'categorical'
            },
            ('preprocess', 'model_action'): {
                'params': {'choices': [True, False]},
                'sample': 'categorical'
            },
            ('start_neurons', 'model_action'): {
                'params': {'choices': [1, 2, 4, 8, 16, 32]},
                'sample': 'categorical'
            },
            ('dropout_p', 'model_action'): {
                'params': {'low': 0.01, 'high': 0.95, 'log': False, 'step': 0.05},
                'sample': 'float'
            },
            ('batch_size', 'dls_action'): {
                'params': {'choices': [2, 4, 8, 16, 32, 64]},
                'sample': 'categorical'
            }
        }
        self.cs = ConfigSpace(init_config=self.INIT_DICT)
        self.user_objective = UserObjectiveForTestingObjectiveExecutor()
        #SET: ObjectiveExecutor WHAT: UserObjective.objective.__func__ AS: execute_objective.
        setattr(ObjectiveExecutor, 'execute_objective', self.user_objective.objective.__func__)

        self.trial = optuna.create_study(direction='minimize').ask()
        self.device = torch.device('cpu')

    def test_objective_initialization(self):
        #TEST: Initialziation of ConfigSpace via INIT_DICT
        config_space = ConfigSpace(init_config=self.INIT_DICT)
        self.assertEqual(
            config_space.action_var_types(save_state=True),
            {'model_action'},
            msg=f"{config_space._config}"
        )
        self.assertEqual(
            config_space.action_var_types(False),
            {'dls_action'},
            msg=f"config_space.action_var_types(save_state=False): {config_space.action_var_types(save_state=False)} != \nExpected: {set(['dls_action'])}"
        )
        self.assertEqual(
            config_space.action_var_types(),
            {'model_action', 'dls_action'},
            msg=f"config_space.action_var_types(): {config_space.action_var_types()} != \nExpected: {set(['dls_action', 'model_action'])}"
        )

        combined_sampler = CombinedSampler(config_space)
        self.assertEqual(
            combined_sampler.var_types(),
            {
            'dls_action', 'freeze', 'lr_low', 'lr_high', 'lr', 'wd',
            'gradient_clip', 'max_norm', 'one_cycle', 'pct_start', 'n_epoch',
            'model_action', 'loss_func', 'metric'
            },
            msg=f"combined_sampler.var_types(): {combined_sampler.var_types()}")
        self.assertEqual(
            combined_sampler.action_var_types(),
            {'model_action', 'dls_action'},
            msg=f"combined_sampler.action_var_types(): {combined_sampler.action_var_types()} != \nExpected: {set(['model_action', 'dls_action'])}"
        )
        self.assertEqual(
            combined_sampler.model_action_var_types(),
            {'model_action'},
            msg=f"combined_sampler.model_action_var_types(): {combined_sampler.model_action_var_types()} != \nExpected: {set(['model_action'])}"
        )
        self.assertEqual(
            combined_sampler.sig_var_types(),
             {'activation', 'batch_size', 'preprocess', 'dropout_p', 'start_neurons'},
             msg=f"combined_sampler.sig_var_types(): {combined_sampler.sig_var_types()}"
        )
        sampled_dict = dict()
        for key in combined_sampler.var_types():
            sampled_dict[key] = combined_sampler.sample(key, self.trial, default_val=None)
        executor = ObjectiveExecutor(trial=self.trial, combined_sampler=combined_sampler, device=self.device)
        self.assertEqual(executor.default_dict, {})

    def test_execute_objective_trial(self):
        combined_sampler = CombinedSampler(INIT_CONFIG=self.cs)
        executor = ObjectiveExecutor(trial=self.trial, combined_sampler=combined_sampler, device=self.device, default_dict={})
        objective_loss, model_container = executor.execute_objective(False)
        self.assertIsInstance(model_container, Learner)
        model = model_container.model
        self.assertIsInstance(model, nn.Module)
        input_sample = torch.randn(size=[64, 1])
        output = model(input_sample)
        self.assertEqual(output.shape, torch.Size([64, 1]))
        #Additional tests can be adapted similarly

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

..[I 2024-10-13 21:17:20,179] A new study created in memory with name: no-name-0534b06d-caf8-4a87-a4d7-85a1f4998869
[I 2024-10-13 21:17:20,187] A new study created in memory with name: no-name-71ddcf96-763f-4927-b226-5903573abcf4
.................[I 2024-10-13 21:17:20,242] A new study created in memory with name: no-name-5457afa1-7d58-4560-bc6c-61a02df5d50d
[I 2024-10-13 21:17:20,247] A new study created in memory with name: no-name-c332a383-a60c-42b9-8768-676d3e3f4dd2
[I 2024-10-13 21:17:20,249] A new study created in memory with name: no-name-734a86ad-5018-4600-b793-639794eb2986
[I 2024-10-13 21:17:20,252] A new study created in memory with name: no-name-ea28029d-fe2e-4bf0-aeff-f5f6acbcd90e
[I 2024-10-13 21:17:20,255] A new study created in memory with name: no-name-3c5ba2d3-cb4a-48de-8c35-f884fad46646
[I 2024-10-13 21:17:20,258] A new study created in memory with name: no-name-fdbcb711-dbcc-42a3-8a57-e3b42b93f997
[I 2024-10-13 21:17:20,261] A new study created in memory with name: 

All good

Missing signature parameters in model_action: {'dropout_p', 'rand_number', 'activation'} 

Missing signature parameters in model_action: {'dropout_p', 'rand_number', 'activation'} 
Missing signature parameters in dls_action: {'normalize', 'activation', 'batch_size'} 

Missing signature parameters in model_action: {'dropout_p', 'activation'} 
Missing signature parameters in dls_action: {'normalize', 'activation', 'batch_size'} 

Missing signature parameters in model_action: {'dropout_p'} 
Missing signature parameters in dls_action: {'normalize', 'activation', 'batch_size'} 

Missing signature parameters in dls_action: {'normalize', 'activation', 'batch_size'} 

Missing signature parameters in dls_action: {'normalize', 'activation'} 

Missing signature parameters in dls_action: {'activation'} 

All good

All good

All good

Missing signature parameters in model_action: {'dropout_p', 'rand_number', 'activation'} 

Missing signature parameters in model_action: {'dropout_p', 'rand

[I 2024-10-13 21:17:20,302] A new study created in memory with name: no-name-3397c97e-fc4b-48f0-8f92-bd0c73b36f3b
..[I 2024-10-13 21:17:20,317] A new study created in memory with name: no-name-442fbd16-d26e-4e3b-ad07-0a70034df3cf
.[I 2024-10-13 21:17:20,331] A new study created in memory with name: no-name-a4ea1336-d06d-468e-9d2f-48faeccd5b0a
.[I 2024-10-13 21:17:20,337] A new study created in memory with name: no-name-e01a0498-86d9-4ca5-b9b6-794e4dab411f
.[I 2024-10-13 21:17:20,354] A new study created in memory with name: no-name-3dcf599a-9d84-4ae9-807f-e256c62fdc95
..

Missing signature parameters in dls_action: {'normalize', 'activation'} 

Missing signature parameters in dls_action: {'activation'} 

All good

All good

All good

All good

All good

All good

Missing signature parameters in model_action: {'dropout_p', 'rand_number', 'activation'} 

Missing signature parameters in model_action: {'dropout_p', 'rand_number', 'activation'} 
Missing signature parameters in dls_action: {'normalize', 'activation', 'batch_size'} 

Missing signature parameters in model_action: {'dropout_p', 'activation'} 
Missing signature parameters in dls_action: {'normalize', 'activation', 'batch_size'} 

Missing signature parameters in model_action: {'dropout_p'} 
Missing signature parameters in dls_action: {'normalize', 'activation', 'batch_size'} 

Missing signature parameters in dls_action: {'normalize', 'activation', 'batch_size'} 

Missing signature parameters in dls_action: {'normalize', 'activation'} 

Missing signature parameters in dls_action: {'activation'} 

A

.....................................................[I 2024-10-13 21:17:20,486] A new study created in memory with name: no-name-0e068e7b-14a8-497e-b35f-3d5b6722a530
.[I 2024-10-13 21:17:20,494] A new study created in memory with name: no-name-192c43f9-274e-4426-ab9d-3fe580662b3b
.

...

.[I 2024-10-13 21:17:48,100] A new study created in memory with name: no-name-385253a8-9f51-4914-b7ed-25b568422387
.[I 2024-10-13 21:17:48,120] A new study created in memory with name: no-name-5ad87ad8-fe2f-4120-b879-875a638b84a0


Missing signature parameters in model_action: {'dropout_p', 'activation', 'start_neurons', 'preprocess'} 

Missing signature parameters in model_action: {'dropout_p', 'activation', 'start_neurons', 'preprocess'} 
Missing signature parameters in dls_action: {'batch_size'} 

Missing signature parameters in model_action: {'dropout_p', 'activation', 'start_neurons', 'preprocess'} 
Missing signature parameters in dls_action: {'batch_size'} 

Missing signature parameters in model_action: {'dropout_p', 'activation', 'start_neurons', 'preprocess'} 
Missing signature parameters in dls_action: {'batch_size'} 

Missing signature parameters in model_action: {'dropout_p', 'activation', 'start_neurons', 'preprocess'} 
Missing signature parameters in dls_action: {'batch_size'} 

Missing signature parameters in model_action: {'dropout_p', 'activation', 'start_neurons', 'preprocess'} 
Missing signature parameters in dls_action: {'batch_size'} 

Missing signature parameters in model_action: {'dropout_p'

epoch,train_loss,valid_loss,custom_mse,time
0,503979423583755516772352.000000,79789998699672040356196898570240.000000,79789998699672040356196898570240.000000,00:00
1,493797433344617166667776.000000,79790008371078597273230296219648.000000,79790008371078597273230296219648.000000,00:00
2,496377059182377967812608.000000,79789998699672040356196898570240.000000,79789998699672040356196898570240.000000,00:00
3,499720387430549748187136.000000,79790003535375318814713597394944.000000,79790003535375318814713597394944.000000,00:00
4,512017556285077434925056.000000,79789998699672040356196898570240.000000,79789998699672040356196898570240.000000,00:00


...........[I 2024-10-13 21:17:50,334] A new study created in memory with name: no-name-7f45e25a-6a35-4ae6-9bf0-70a015b44185
..[I 2024-10-13 21:17:50,339] A new study created in memory with name: no-name-2b06f4bc-7a79-48b6-8bdf-d2fee3effa78
.....
----------------------------------------------------------------------
Ran 110 tests in 30.176s

OK


Dynamic verification passed successfully
.


#Class: HpoTrainer

In [22]:
class HpoTrainer:
    def __init__(self, combined_sampler: CombinedSampler, n_trials: int, device: torch.device,
                 prev_top_k_trials: list = [], pruner: optuna.pruners = None, timeout: int = 14400, eta: int = 2):
        """
        Class Explanation: HpoTrainer is a TEMPORARY objective for running a single HPO process.
        """
        if any(not isinstance(trial, (Trial, FrozenTrial, dict)) for trial in prev_top_k_trials):
            raise ValueError(f"Input 'prev_top_k_trials' must be a list where each element is one of (Trial, FrozenTrial, dict) but found: {prev_top_k_trials}")

        self.combined_sampler = combined_sampler
        self.device = device
        self.eta = eta
        self.timeout = timeout
        #Turn them into dictionaries before saving.
        self.prior_trials = [trial if isinstance(trial, dict) else trial.params for trial in prev_top_k_trials]

        #Use the user-defined n_trials for new trials (ignore the prior trial count)
        self.n_trials = n_trials

        #Select a pruner if not specified
        choices = (optuna.pruners.HyperbandPruner, optuna.pruners.SuccessiveHalvingPruner)
        self.pruner = random.choice(choices) if pruner is None else pruner

    def run(self):
        print("*****************RUNNING: HPO PROCESS*****************")

        #To get logging info on the HPO process:
        optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))

        #Create the study object with pruner
        study = optuna.create_study(
            direction='minimize',
            pruner=self.pruner(reduction_factor=self.eta)
        )

        #Enqueue the prior trials (these will run first)
        for trial in self.prior_trials:
            study.enqueue_trial(trial)

        #Run the optimization for the user-defined n_trials (new trials only)
        study.optimize(func=self.hpo_objective, n_trials=self.n_trials, timeout=self.timeout)

        #Return the optimized study
        return study

    def hpo_objective(self, trial):
        #Sanity check
        if not isinstance(trial, (Trial, FrozenTrial)):
            raise ValueError(f"Input 'trial' MUST be an instance of optuna::Trial OR optuna::FrozenTrial but found trial: {trial}.")

        #Retrieve & return the objective loss, produced by ObjectiveExecutor.
        objective_loss, _ = ObjectiveExecutor(
            trial=trial,
            combined_sampler=self.combined_sampler,
            device=self.device
        ).execute_objective(turn_off_training_log=True)

        #Return the objective loss.
        return objective_loss

#Test: HpoTrainer

In [23]:
if __name__ == '__main__':
    from typing import Dict, Any
    from torch.utils.data import Dataset, DataLoader
    class UserObjectiveForTestingHpoTrainer(UserObjective):
        def __init__(self):
            super().__init__()

    class PolynomialDataset(Dataset):
        def __init__(self, coeffs: torch.tensor, input_min: float, input_max: float, input_size: int):
            self.coeffs = coeffs.clone().detach()
            self.inputs = torch.empty(size=(input_size,)).uniform_(input_min, input_max)
            self.outputs = self._f(X=self.inputs)

        def __getitem__(self, idx):
            return self.inputs[idx], self.outputs[idx]

        def __len__(self):
            return self.inputs.size(0)

        def _f(self, X):
            powers = torch.arange(self.coeffs.size(0)).unsqueeze(0)
            Y = self.inputs.unsqueeze(-1)**powers
            return torch.matmul(Y, self.coeffs).squeeze(-1)

    degree = 6
    coeffs_min, coeffs_max = -10.0, 10.0
    coeffs = torch.empty(size=(degree+1,)).uniform_(coeffs_min, coeffs_max)

    train_dset = PolynomialDataset(coeffs=coeffs, input_min=-100.0, input_max=100.0, input_size=8000)
    valid_dset = PolynomialDataset(coeffs=coeffs, input_min=-300.12, input_max=500.78, input_size=2000)

    class ModelDefaultHpoTrainer(nn.Module):
        def __init__(self, activation: nn.Module, preprocess: bool, start_neurons: int):
            super().__init__()
            self.preprocess = preprocess
            if self.preprocess:
                start_neurons = max(start_neurons, degree+1)
            self.net = nn.Sequential(
                nn.Linear(in_features=degree+1 if self.preprocess else 1, out_features=start_neurons),
                nn.BatchNorm1d(num_features=start_neurons),
                activation(),
                nn.Linear(in_features=start_neurons, out_features=start_neurons*2),
                nn.BatchNorm1d(num_features=start_neurons*2),
                activation(),
                nn.Linear(in_features=start_neurons*2, out_features=1)
            )

        def _preprocess(self, X):
            N = X.size(0)
            X = X.unsqueeze(-1)**torch.arange(degree+1).unsqueeze(0)
            return X

        def forward(self, X):
            if X.dim() != 1:
                X = X.reshape(-1)

            if self.preprocess:
                X = self._preprocess(X)
            else:
                X = X.unsqueeze(-1)
            return self.net(X)

    class ModelDropoutHpoTrainer(nn.Module):
        def __init__(self, activation: nn.Module, dropout_p: float, preprocess: bool, start_neurons: int):
            super().__init__()
            self.preprocess = preprocess
            if self.preprocess:
                start_neurons = max(start_neurons, degree+1)
            self.net = nn.Sequential(
                nn.Linear(in_features=degree+1 if self.preprocess else 1, out_features=start_neurons),
                nn.BatchNorm1d(num_features=start_neurons),
                activation(),
                nn.Dropout(p=dropout_p),
                nn.Linear(in_features=start_neurons, out_features=start_neurons*2),
                nn.BatchNorm1d(num_features=start_neurons*2),
                activation(),
                nn.Linear(in_features=start_neurons*2, out_features=1)
            )

        def _preprocess(self, X):
            N = X.size(0)
            X = X.unsqueeze(-1)**torch.arange(degree+1).unsqueeze(0)
            return X

        def forward(self, X):
            if X.dim() != 1:
                X = X.reshape(-1)

            if self.preprocess:
                X = self._preprocess(X)
            else:
                X = X.unsqueeze(-1)
            return self.net(X)

    def model_action_default_hpo_trainer(activation: nn.Module, preprocess: bool, start_neurons: int, device: torch.device):
        if start_neurons < 1:
            raise ValueError(f"start_neurons MUST be a positive integer but found start_neurons: {start_neurons}")
        return ModelDefaultHpoTrainer(activation=activation,
                            preprocess=preprocess,
                            start_neurons=start_neurons).to(device)

    def model_action_dropout_hpo_trainer(activation: nn.Module, dropout_p: float, preprocess: bool, start_neurons: int, device: torch.device):
        if start_neurons < 1:
            raise ValueError(f"start_neurons MUST be a positive integer but found start_neurons: {start_neurons}")
        if dropout_p < 0.0 or dropout_p >= 1.0:
            raise ValueError(f"0.0 <= dropout_p < 1.0 required but found dropout_p: {dropout_p}")
        return ModelDropoutHpoTrainer(activation=activation,
                            dropout_p=dropout_p,
                            preprocess=preprocess,
                            start_neurons=start_neurons).to(device)

    def dls_action_default_hpo_trainer(batch_size: int, device=None):
        train_dataloader = DataLoader(train_dset, batch_size=batch_size, shuffle=True)
        valid_dataloader = DataLoader(valid_dset, batch_size=batch_size, shuffle=True)
        dls = DataLoaders(train_dataloader, valid_dataloader)
        return dls

    def custom_mse(Y_hat, Y):
        Y_hat = Y_hat.reshape(-1)
        Y = Y.reshape(-1)
        return nn.MSELoss()(Y_hat, Y)

#Now, adapt the test cases
class TestHpoTrainer(unittest.TestCase):
    def setUp(self):
        self.INIT_DICT = {
            ('dls_action', None): {
                'params': {'choices': [dls_action_default_hpo_trainer]},
                'sample': 'categorical',
                'action': True
            },
            ('freeze', None): {
                'params': {'choices': [True, False]},
                'sample': 'categorical'
            },
            ('lr_low', None): {
                'params': {'low': 1e-7, 'high': 9e-4, 'log': True},
                'sample': 'float'
            },
            ('lr_high', None): {
                'params': {'low': 9e-4, 'high': 1e-1, 'log': True},
                'sample': 'float'
            },
            ('lr', None): {
                'params': {'low': 1e-6, 'high': 1e-1, 'log': True},
                'sample': 'float'
            },
            ('wd', None): {
                'params': {'choices': [1e-4, 1e-3, 1e-2, 1e-1]},
                'sample': 'categorical'
            },
            ('gradient_clip', None): {
                'params': {'choices': [True, False]},
                'sample': 'categorical'
            },
            ('max_norm', None): {
                'params': {'low': 0.0, 'high': 15.0, 'log': False},
                'sample': 'float'
            },
            ('one_cycle', None): {
                'params': {'choices': [True, False]},
                'sample': 'categorical'
            },
            ('pct_start', None): {
                'params': {'low': 0.10, 'high': 0.95,  'log': False, 'step': 0.05},
                'sample': 'float'
            },
            ('n_epoch', None): {
                'params': {'choices': list(range(1,11))},
                'sample': 'categorical'
            },
            ('model_action', None): {
                'params': {'choices': [model_action_default_hpo_trainer, model_action_dropout_hpo_trainer]},
                'sample': 'categorical',
                'action': True,
                'save_state': True
            },
            ('activation', 'model_action'): {
                'params': {'choices': [nn.ReLU, nn.PReLU, nn.SiLU]},
                'sample': 'categorical'
            },
            ('loss_func', None): {
                'params': {'choices': [custom_mse]},
                'sample': 'categorical'
            },
            ('metric', None): {
                'params': {'choices': [custom_mse]},
                'sample': 'categorical'
            },
            ('preprocess', 'model_action'): {
                'params': {'choices': [True, False]},
                'sample': 'categorical'
            },
            ('start_neurons', 'model_action'): {
                'params': {'choices': [1, 2, 4, 8, 16, 32]},
                'sample': 'categorical'
            },
            ('dropout_p', 'model_action'): {
                'params': {'low': 0.01, 'high': 0.95,  'log': False, 'step': 0.05},
                'sample': 'float'
            },
            ('batch_size', 'dls_action'): {
                'params': {'choices': [2, 4, 8, 16, 32, 64]},
                'sample': 'categorical'
            }
        }
        self.cs = ConfigSpace(init_config=self.INIT_DICT)
        self.user_objective = UserObjectiveForTestingHpoTrainer()
        setattr(ObjectiveExecutor, 'execute_objective', self.user_objective.objective.__func__)
        self.trial = optuna.create_study(direction='minimize').ask()
        self.device = torch.device('cpu')

    def test_initialization(self):
        combined_sampler = CombinedSampler(INIT_CONFIG=self.cs)
        self.assertEqual(
            combined_sampler.model_action_var_types(),
            {'model_action'},
        )
        self.assertEqual(
            combined_sampler.action_var_types(),
            {'dls_action', 'model_action'},
        )

        hpo_trainer = HpoTrainer(
            combined_sampler=combined_sampler,
            n_trials=2,
            device=self.device
        )
        self.assertIsInstance(hpo_trainer, HpoTrainer)
        self.assertIsInstance(hpo_trainer.combined_sampler, CombinedSampler)
        self.assertEqual(hpo_trainer.n_trials, 2)
        self.assertEqual(hpo_trainer.device, self.device)
        self.assertEqual(hpo_trainer.eta, 2)
        self.assertEqual(hpo_trainer.timeout, 14400)

    def test_run(self):
        n_trials = [4, 2, 1]
        top_ks = random.choices(list(range(1, 10)), k=len(n_trials))
        for n, k in zip(n_trials, top_ks):
            combined_sampler = CombinedSampler(INIT_CONFIG=self.cs)

            hpo_trainer = HpoTrainer(
                combined_sampler=combined_sampler,
                n_trials=n,
                device=self.device
            )

            study = hpo_trainer.run()
            prev_trials = study.trials
            top_k = min(len(prev_trials), k)
            top_k_trials = sorted(
                prev_trials,
                key=lambda trial: trial.value if trial.value is not None else float('inf')
            )[:top_k]
            combined_sampler.update(
                top_k_trials=top_k_trials,
                update_state_window=2,
                device=self.device
            )
            #You can add assertions or checks here as needed
            print("****************************************")
            print(combined_sampler)
            print("****************************************")
            print()

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

..[I 2024-10-13 21:17:50,423] A new study created in memory with name: no-name-b0bb75cb-836a-4669-a5e6-8b0d4c0d3a59
[I 2024-10-13 21:17:50,426] A new study created in memory with name: no-name-c6d572ec-e615-4e53-82aa-075590862f5e
.................[I 2024-10-13 21:17:50,478] A new study created in memory with name: no-name-64fdc1fa-d1b8-46e3-a825-904a1f8fac11
[I 2024-10-13 21:17:50,483] A new study created in memory with name: no-name-562a4196-bc30-4054-91fb-d05c17c53f31
[I 2024-10-13 21:17:50,486] A new study created in memory with name: no-name-52d9fcaa-db9b-4212-bad2-a482716be014
[I 2024-10-13 21:17:50,488] A new study created in memory with name: no-name-d12cc965-ac5e-4e4c-891d-b100000c8ec5
[I 2024-10-13 21:17:50,490] A new study created in memory with name: no-name-dc0c620d-36a6-42c7-aef2-a8c6bdd9ac57
[I 2024-10-13 21:17:50,493] A new study created in memory with name: no-name-0f26879e-c62a-46e1-b804-d03f71781376
[I 2024-10-13 21:17:50,497] A new study created in memory with name: 

All good

Missing signature parameters in model_action: {'dropout_p', 'rand_number', 'activation'} 

Missing signature parameters in model_action: {'dropout_p', 'rand_number', 'activation'} 
Missing signature parameters in dls_action: {'normalize', 'activation', 'batch_size'} 

Missing signature parameters in model_action: {'dropout_p', 'activation'} 
Missing signature parameters in dls_action: {'normalize', 'activation', 'batch_size'} 

Missing signature parameters in model_action: {'dropout_p'} 
Missing signature parameters in dls_action: {'normalize', 'activation', 'batch_size'} 

Missing signature parameters in dls_action: {'normalize', 'activation', 'batch_size'} 

Missing signature parameters in dls_action: {'normalize', 'activation'} 

Missing signature parameters in dls_action: {'activation'} 

All good

All good

All good

Missing signature parameters in model_action: {'dropout_p', 'rand_number', 'activation'} 

Missing signature parameters in model_action: {'dropout_p', 'rand

.[I 2024-10-13 21:17:50,523] A new study created in memory with name: no-name-38f0848c-1f3e-4dfa-974f-487108ff02c8
..[I 2024-10-13 21:17:50,541] A new study created in memory with name: no-name-b6f78fef-a941-4e01-be75-59c18acf0401
.[I 2024-10-13 21:17:50,549] A new study created in memory with name: no-name-25f668fb-872b-4857-a3e8-11c2662f4711
.[I 2024-10-13 21:17:50,554] A new study created in memory with name: no-name-9c5d2ed5-226f-4084-b8d4-82bda8312f61
.[I 2024-10-13 21:17:50,569] A new study created in memory with name: no-name-7ed55495-ebdc-4bc9-8197-30e243e8462e
.......

All good

All good

All good

All good

All good

Missing signature parameters in model_action: {'dropout_p', 'rand_number', 'activation'} 

Missing signature parameters in model_action: {'dropout_p', 'rand_number', 'activation'} 
Missing signature parameters in dls_action: {'normalize', 'activation', 'batch_size'} 

Missing signature parameters in model_action: {'dropout_p', 'activation'} 
Missing signature parameters in dls_action: {'normalize', 'activation', 'batch_size'} 

Missing signature parameters in model_action: {'dropout_p'} 
Missing signature parameters in dls_action: {'normalize', 'activation', 'batch_size'} 

Missing signature parameters in dls_action: {'normalize', 'activation', 'batch_size'} 

Missing signature parameters in dls_action: {'normalize', 'activation'} 

Missing signature parameters in dls_action: {'activation'} 

All good

All good

All good

All good

All good

All good

Missing signature parameters in model_action: {'dropout_p', 'rand_number', 'activation

................................................[I 2024-10-13 21:17:50,714] A new study created in memory with name: no-name-99f9229b-6715-43e8-8063-edfa95854df2
.[I 2024-10-13 21:17:50,719] A new study created in memory with name: no-name-bb2d65cb-0bd9-409d-ab3c-90d5eb3e58f8
[I 2024-10-13 21:17:50,721] A new study created in memory with name: no-name-14ffb9b2-f2e5-40b4-a699-600f147ebcb4


...

...........[I 2024-10-13 21:20:40,887] A new study created in memory with name: no-name-4b61048c-c01f-42b1-a1d6-df0e0b4b4069


Dynamic verification passed successfully
.
A new study created in memory with name: no-name-4b61048c-c01f-42b1-a1d6-df0e0b4b4069
A new study created in memory with name: no-name-4b61048c-c01f-42b1-a1d6-df0e0b4b4069
A new study created in memory with name: no-name-4b61048c-c01f-42b1-a1d6-df0e0b4b4069


..[I 2024-10-13 21:20:40,894] A new study created in memory with name: no-name-66056870-f8e3-426e-a24c-42440ffeb741


A new study created in memory with name: no-name-66056870-f8e3-426e-a24c-42440ffeb741
A new study created in memory with name: no-name-66056870-f8e3-426e-a24c-42440ffeb741
A new study created in memory with name: no-name-66056870-f8e3-426e-a24c-42440ffeb741


.....
----------------------------------------------------------------------
Ran 112 tests in 170.495s

OK


#Class: Auto

In [28]:
class Auto:
    def __init__(self, INIT_CONFIG: ConfigSpace, storage_dir: str = '/content/trial_storage/'):
        if not isinstance(INIT_CONFIG, ConfigSpace):
            raise ValueError("Input 'INIT_CONFIG' is completely determined from the beginning. Nothing to optimize for")

        #Save it as a member.
        self.init_config = INIT_CONFIG
        #Verify syntax.
        self.init_config.verify_syntax(True)
        self.user_objective = None

        self.combined_sampler = CombinedSampler(self.init_config)

        if self.combined_sampler.is_fixed():
            raise ValueError("Input 'INIT_SOURCE' is completely determined from the beginning. Nothing to optimize for")

        self.prev_top_k_trials = []
        self.storage_dir = storage_dir
        os.makedirs(self.storage_dir, exist_ok=True)
        self._n_study = None

    def optimize(self, user_objective: Objective, n_auto: int, top_k: int, update_state_window: int=-1, prior_dynamic_verification: bool = False, device: torch.device=None, prior_trials: list = []):
        #Initialize input variables properly
        update_state_window = top_k//2 if update_state_window == -1 else min(update_state_window, top_k)
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') if device is None else device

        #ERROR_CHECK_ON: user_objective.
        try:
            with self._suppress_output():
                user_objective.verify_objective_static(init_config=self.init_config, missing_var_types_raise_error=True)
                if prior_dynamic_verification:
                    user_objective.verify_objective_dynamic(init_config=self.init_config, device=device, turn_off_training_log=True)
        except Exception as e:
            raise RuntimeError(f"Objective verification failed: {e}") from e

        #ERROR_CHECK_ON: prior_trials
        if not(isinstance(prior_trials, list) and all(isinstance(trial, dict) for trial in prior_trials)):
            type_prior_trials = type(prior_trials)
            type_individual_trials = [type(trial) for trial in prior_trials]
            raise ValueError(f"Input prior_trials must be a list and every element in prior_trials must be a dictionary but found type(prior_trials): {type_prior_trials} , type(trials): {type_individual_trials}")

        #self.prev_top_k_trials <- (properly formatted)prior_trials.
        self.prev_top_k_trials = self._prepare_prior_trials(prior_trials)

        #Once the confirmation is done, save 'user_objective'.
        self.user_objective = user_objective
        #Set .execute_objective to the ObjectiveExecutor.
        setattr(ObjectiveExecutor, 'execute_objective', self.user_objective.objective.__func__)

        #Now, do the optimization.
        list_n_trials = self._get_list_n_trials(n_auto=n_auto)
        self._n_study = len(list_n_trials)
        print("*****************************************************************")
        print(f"n_auto: {n_auto}, list_n_trials: {list_n_trials}")
        print()

        for idx, n_trials in enumerate(list_n_trials):
            hpo_pointer = HpoTrainer(
                combined_sampler=self.combined_sampler,
                n_trials=n_trials,
                device=device,
                prev_top_k_trials=self.prev_top_k_trials
            )
            optimized_study = hpo_pointer.run()
            self._update(
                idx=idx,
                optimized_study=optimized_study,
                top_k=top_k,
                update_state_window=update_state_window,
                device=device
            )

    def num_study(self):
        return self._n_study

    def get_study(self, idx: int):
        if self._n_study is None:
            raise RuntimeError("No studies have been conducted yet. Please call .optimize() before requesting a study.")
        if idx < 0 or idx >= self._n_study:
            raise ValueError(f"Input 'idx':{idx} must satisfy 0 <= idx < {self._n_study} but failed.")

        return self._load_study_from_pickle(idx)

    def _prepare_prior_trials(self, prior_trials: list):
        #NOTE: This should ONLY be called at the START of the loop, NOT inside the ._update
        list_params = [trial.params if isinstance(trial, (Trial, FrozenTrial)) else trial for trial in prior_trials]

        #Ensure: For every param in list_params, the format of each key must either be (1)'var_type>belongs_to' OR (2)(var_type, belongs_to)
        idx_to_invalid_keys = {idx: {key for key in param.keys() if not isinstance(key, (str, tuple))} for idx, param in enumerate(list_params)}
        msg = reduce(
            lambda s1,s2: s1+'\n'+s2,
            [f'Invalid keys in trial{idx}: {keys}' for idx, keys in idx_to_invalid_keys.items() if keys],
            ''
        )
        if msg:
            raise ValueError(msg)

        #Ensure: Every key is properly formatted.
        list_formatted_params = list()
        for idx, params in enumerate(list_params):
            formatted_params = dict()
            for key, val in params.items():
                if (isinstance(key, str) and '>' not in key) or (isinstance(key, str) and len(tuple(key.split('>'))) != 2):
                    raise KeyError(f"Detected invalid key format: {key}->{val} in trial{idx}.\n Note that every key in trial must either be formatted (1)'var_type>belongs_to' OR (2)(var_type, belongs_to)")
                var_type, belongs_to = tuple(key.split('>')) if isinstance(key, str) else key
                belongs_to = None if belongs_to == 'None' else belongs_to
                if (var_type, belongs_to) not in self.init_config.all_pairs():
                    raise KeyError(f"Detected invalid key in trial{idx}. Provided key: {key} translates to {(var_type, belongs_to)}, which is not in {self.init_config.all_pairs()}")

                #Retrieve the corresponding Var/ActionVar object from self.init_config.
                var = self.init_config.get_var(var_type, belongs_to)

                #***NOTE_PARTICULAR_BEHAVIOR***
                #If input 'val' is in the list of action_func(s) corresponding to action object(s), then assign corresponding idx to 'val'.
                action_funcs = [action.action_func for action in var.distribution(indices=False, current=False)] if isinstance(var, ActionVar) else []
                if val in action_funcs:
                    val = action_funcs.index(val)

                #Raise Error if the corresponding value is not appropriate.
                if not(var.contain_idx(val) or var.contain_val(val)):
                    msg = (f"Detected invalid value in trial{idx}. Provided mapping {(key, val)} translates to {((var_type, belongs_to), val)}. \
                             However, parameter {var_type}'s distribution does not contain {val}.\n Note that {val} should either belong to {var.distribution(False)} or {var.distribution(True)}")
                    raise ValueError(msg)

                mask_idx = var.convert_val_to_idx(val) if not var.contain_idx(val) else val
                formatted_params[var_type+'>'+str(belongs_to)] = mask_idx
            list_formatted_params.append(formatted_params)

        return list_formatted_params

    def _get_list_n_trials(self, n_auto: int):
        """
        ACCEPTS:
        n_auto := Positive integer power of 2. IF n_auto <= 1, THEN n_auto <- 2
                                               IF n_auto > 1 but NOT a power of 2, THEN n_auto <- (nearest power of 2) - 1.
        RETURNS:
        Exponential distribution of [2^m, 2^(m-1), ..., 2^1, 2^0] + (n_auto==power of 2)*[2^0] trials returned.
        """
        n_auto = max(n_auto, 2)
        pow2 = n_auto & (n_auto-1) == 0

        output = [1<<e for e in range(math.ceil(math.log2(n_auto)) - 1, -1, -1)]

        if pow2:
            output.append(1)

        return output

    def _update(self, idx: int, optimized_study: optuna.study, top_k: int, update_state_window: int, device: torch.device):
        prev_trials = optimized_study.trials
        top_k = min(top_k, len(prev_trials))
        update_state_window = min(update_state_window, top_k)

        top_k_trials = sorted(
            prev_trials,
            key=lambda trial: trial.value if trial.value is not None else float('inf')
        )[:top_k]

        #Save the whole study, not just top_k_trials
        self._save_study_to_pickle(idx, optimized_study)
        self.prev_top_k_trials = top_k_trials

        self.combined_sampler.update(
            top_k_trials=top_k_trials,
            update_state_window=update_state_window,
            device=device
        )

    def _save_study_to_pickle(self, idx: int, study: optuna.study):
        save_path = os.path.join(self.storage_dir, f'study_{idx}.pkl')
        with gzip.open(save_path, 'wb') as f:
            pickle.dump(study, f)
        print(f"Saved study to: {save_path}")

    def _load_study_from_pickle(self, idx: int):
        load_path = os.path.join(self.storage_dir, f'study_{idx}.pkl')
        if not os.path.exists(load_path):
            raise FileNotFoundError(f"No study found at: {load_path}")

        with gzip.open(load_path, 'rb') as f:
            study = pickle.load(f)
        print(f"Loaded study from: {load_path}")
        return study

    @contextmanager
    def _suppress_output(self):
        """
        Context manager to suppress stdout and stderr.
        Redirects both to os.devnull, effectively silencing any output within the context.
        """
        with open(os.devnull, 'w') as fnull:
            #Save the current stdout and stderr
            old_stdout = sys.stdout
            old_stderr = sys.stderr
            try:
                #Redirect stdout and stderr to fnull
                sys.stdout = fnull
                sys.stderr = fnull
                yield #yield the control back to the caller of this block.
            finally:
                #Restore original stdout and stderr
                #NOTE: 'finally' is needed so that the output stream is unaffected.
                sys.stdout = old_stdout
                sys.stderr = old_stderr

#Test: Auto

In [29]:
if __name__ == '__main__':
    from torch.utils.data import Dataset
    class PolynomialDataset(Dataset):
        def __init__(self, coeffs: torch.tensor, input_min: float, input_max: float, input_size: int):
            #coeffs.shape == [7, 1]
            self.coeffs = coeffs.clone().detach()
            #inputs.shape == [inputs]
            self.inputs = torch.empty(size=(input_size,)).uniform_(input_min, input_max)
            #Preprocess the output and retrieve it upon request.
            self.outputs = self._f(X=self.inputs)

        def __getitem__(self, idx):
            return self.inputs[idx], self.outputs[idx]

        def __len__(self):
            if self.inputs.size(0) != self.outputs.size(0):
                return ValueError(f"len(inputs): {self.inputs.size(0)} != len(outputs): {self.outputs.size(0)}")
            return self.inputs.size(0)

        def _f(self, X):
            #power.shape == [1, 7]
            powers = torch.arange(self.coeffs.size(0)).unsqueeze(0)
            Y = self.inputs.unsqueeze(-1)**powers
            return torch.matmul(Y, self.coeffs).squeeze(-1)

    """
    Action functions:
    """
    class ModelDefaultAuto(nn.Module):
        def __init__(self, activation: nn.Module, preprocess: bool, start_neurons: int):
            super().__init__()
            self.preprocess = preprocess
            if self.preprocess:
                start_neurons = max(start_neurons, 7)
            self.net = nn.Sequential(
                nn.Linear(
                    in_features=7 if self.preprocess else 1,
                    out_features=start_neurons
                ),
                nn.BatchNorm1d(num_features=start_neurons),
                activation(),
                nn.Linear(in_features=start_neurons,out_features=start_neurons*2),
                nn.BatchNorm1d(num_features=start_neurons*2),
                activation(),
                nn.Linear(in_features=start_neurons*2,out_features=1)
            )

        def _preprocess(self, X):
            N = X.size(0)
            #X <- [64,1]x[1,7] == [64,7], where each index (i,j) yields X[i]**j.
            X = X.unsqueeze(-1)**torch.arange(7).unsqueeze(0)
            if X.size() != torch.Size([N,7]):
                raise ValueError(f"X.size() MUST be torch.Size([{N},{7}]) but found X.size()=={X.size()}")
            return X

        def forward(self, X):
            if X.dim() != 1:
                raise ValueError(f"X.dim() MUST be 1 but found X.dim()=={X.dim()}")

            if self.preprocess:
                X = self._preprocess(X)
            else:
                X = X.unsqueeze(-1)
            return self.net(X)

    class ModelDropoutAuto(nn.Module):
        def __init__(self, activation: nn.Module, dropout_p: int, preprocess: int, start_neurons: Int):
            super().__init__()
            self.preprocess = preprocess
            if self.preprocess:
                start_neurons = max(start_neurons, 7)
            self.net = nn.Sequential(
                nn.Linear(
                    in_features=7 if self.preprocess else 1,
                    out_features=start_neurons
                ),
                nn.BatchNorm1d(num_features=start_neurons),
                activation(),
                nn.Dropout(p=dropout_p),
                nn.Linear(in_features=start_neurons, out_features=start_neurons*2),
                nn.BatchNorm1d(num_features=start_neurons*2),
                activation(),
                nn.Linear(in_features=start_neurons*2, out_features=1)
            )

        def _preprocess(self, X):
            N = X.size(0)
            #X <- [64, 1]x[1, 7] == [64, 7], where each index (i, j) yields X[i]**j.
            X = X.unsqueeze(-1)**torch.arange(7).unsqueeze(0)
            if X.size() != torch.Size([N, 7]):
                raise ValueError(f"X.size() MUST be torch.Size([{N},{7}]) but found X.size()=={X.size()}")
            return X

        def forward(self, X):
            if X.dim() != 1:
                raise ValueError(f"X.dim() MUST be 1 but found X.dim()=={X.dim()}")

            if self.preprocess:
                X = self._preprocess(X)
            else:
                X = X.unsqueeze(-1)
            return self.net(X)

    def model_action_default_auto(activation: nn.Module, preprocess: bool, start_neurons: int, device: torch.device):
        if start_neurons < 1:
            raise ValueError(f"start_neurons MUST be a positive integer but found start_neurons: {start_neurons}")
        return ModelDefaultAuto(
            activation=activation,
            preprocess=preprocess,
            start_neurons=start_neurons
        ).to(device)

    def model_action_dropout_auto(activation: nn.Module, dropout_p: int, preprocess: bool, start_neurons: int, device: torch.device):
        if start_neurons < 1:
            raise ValueError(f"start_neurons MUST be a positive integer but found start_neurons: {start_neurons}")
        if dropout_p < 0.0 or dropout_p >= 1.0:
            raise ValueError(f"0.0 <= dropout_p < 1.0 required but found dropout_p: {dropout_p}")
        return ModelDropoutAuto(
            activation=activation,
            dropout_p=dropout_p,
            preprocess=preprocess,
            start_neurons=start_neurons
        ).to(device)

    #Define DataLoaders and training process
    def get_dls(batch_size: int):
        coeffs_min, coeffs_max = -10.0, 10.0
        coeffs = torch.empty(size=(7,)).uniform_(coeffs_min, coeffs_max)

        train_dset = PolynomialDataset(coeffs=coeffs, input_min=-100.0, input_max=100.0, input_size=8000)
        valid_dset = PolynomialDataset(coeffs=coeffs, input_min=-300.12, input_max=500.78, input_size=2000)

        train_dataloader = DataLoader(train_dset, batch_size=batch_size, shuffle=True)
        valid_dataloader = DataLoader(valid_dset, batch_size=batch_size, shuffle=True)

        dls = DataLoaders(train_dataloader, valid_dataloader)
        return dls

    def custom_mse(Y_hat, Y):
        Y_hat = Y_hat.reshape(-1)
        Y = Y.reshape(-1)
        if Y_hat.size() != Y.size():
            raise ValueError(f"Y_hat.size(): {Y_hat.size()} != Y.size(): {Y.size()}")
        return nn.MSELoss()(Y_hat, Y)

In [30]:
class TestAutoPreparePriorTrials(unittest.TestCase):
    def setUp(self):
        #Define the full INIT_DICT as per your code
        self.INIT_DICT = {
            ('dls_action', None): {
                'params': {'choices': [get_dls]},
                'sample': 'categorical',
                'action': True
            },
            ('freeze', None): {
                'params': {'choices': [True, False]},
                'sample': 'categorical'
            },
            ('lr_low', None): {
                'params': {'low': 1e-7, 'high': 9e-4, 'log': True},
                'sample': 'float'
            },
            ('lr_high', None): {
                'params': {'low': 9e-4, 'high': 1e-1, 'log': True},
                'sample': 'float'
            },
            ('lr', None): {
                'params': {'low': 1e-6, 'high': 1e-1, 'log': True},
                'sample': 'float'
            },
            ('wd', None): {
                'params': {'choices': [1e-4, 1e-3, 1e-2, 1e-1]},
                'sample': 'categorical'
            },
            ('gradient_clip', None): {
                'params': {'choices': [True, False]},
                'sample': 'categorical'
            },
            ('max_norm', None): {
                'params': {'low': 0.0, 'high': 15.0, 'log': False},
                'sample': 'float'
            },
            ('one_cycle', None): {
                'params': {'choices': [True, False]},
                'sample': 'categorical'
            },
            ('pct_start', None): {
                'params': {'low': 0.10, 'high': 0.95,  'log': False, 'step': 0.05},
                'sample': 'float'
            },
            ('n_epoch', None): {
                'params': {'choices': [5, 10, 15]},
                'sample': 'categorical'
            },
            ('model_action', None): {
                'params': {'choices': [model_action_default_auto, model_action_dropout_auto]},
                'sample': 'categorical',
                'action': True,
                'save_state': True
            },
            ('activation', 'model_action'): {
                'params': {'choices': [nn.ReLU, nn.PReLU, nn.SiLU]},
                'sample': 'categorical'
            },
            ('loss_func', None): {
                'params': {'choices': [custom_mse]},
                'sample': 'categorical'
            },
            ('metric', None): {
                'params': {'choices': [custom_mse]},
                'sample': 'categorical'
            },
            ('preprocess', 'model_action'): {
                'params': {'choices': [True, False]},
                'sample': 'categorical'
            },
            ('start_neurons', 'model_action'): {
                'params': {'choices': [1, 2, 4, 8, 16, 32]},
                'sample': 'categorical'
            },
            ('dropout_p', 'model_action'): {
                'params': {'low': 0.05, 'high': 0.95,  'log': False, 'step': 0.05},
                'sample': 'float'
            },
            ('batch_size', 'dls_action'): {
                'params': {'choices': [32, 64, 128]},
                'sample': 'categorical'
            }
        }

        self.init_config = ConfigSpace(self.INIT_DICT)
        self.storage_dir = './test_trial_storage/'
        self.auto = Auto(self.init_config, storage_dir=self.storage_dir)

        #Create valid prior trials for testing
        self.valid_prior_trials = [
            {
                'dls_action>None': get_dls,  #Assuming index 0 corresponds to get_dls
                'freeze>None': True,
                'lr_low>None': 1e-5,
                'lr_high>None': 1e-3,
                'wd>None': 1e-3,
                'gradient_clip>None': False,
                'one_cycle>None': True,
                'n_epoch>None': 10,
                'model_action>None': 0,  #Assuming index 0 corresponds to model_action_default_auto
                'activation>model_action': nn.ReLU, #Equivalent to index 0
                'preprocess>model_action': True,
                'start_neurons>model_action': 16,
                'batch_size>dls_action': 64
            },
            {
                ('dls_action', None): get_dls,  #Assuming index 0 corresponds to get_dls
                ('freeze', None): True,
                ('lr_low', None): 1e-5,
                ('lr_high', None): 1e-3,
                ('wd', None): 1e-3,
                ('gradient_clip', None): False,
                ('one_cycle', None): True,
                ('n_epoch', None): 10,
                ('model_action', None): 0,  # Assuming index 0 corresponds to model_action_default_auto
                ('activation', 'model_action'): nn.ReLU, #Equivalent to index 0
                ('preprocess', 'model_action'): True,
                ('start_neurons', 'model_action'): 16,
                ('batch_size', 'dls_action'): 64
            },
            {
                'dls_action>None': 0,
                'freeze>None': False,
                'lr_low>None': 1e-4,
                'lr_high>None': 1e-2,
                'wd>None': 1e-2,
                'gradient_clip>None': True,
                'max_norm>None': 5.0,
                'one_cycle>None': False,
                'n_epoch>None': 15,
                'model_action>None': 1,  #Assuming index 1 corresponds to model_action_dropout_auto
                'activation>model_action': nn.SiLU, #Equivalent to index 2
                'dropout_p>model_action': 0.2,
                'preprocess>model_action': False,
                'start_neurons>model_action': 32,
                'batch_size>dls_action': 128
            },
            {
                ('dls_action', None): get_dls,
                ('freeze', None): False,
                ('lr_low', None): 1e-4,
                ('lr_high', None): 1e-2,
                ('wd', None): 1e-2,
                ('gradient_clip', None): True,
                ('max_norm', None): 5.0,
                ('one_cycle', None): False,
                ('n_epoch', None): 15,
                ('model_action', None): model_action_dropout_auto,  #Assuming index 1 corresponds to model_action_dropout_auto
                ('activation', 'model_action'): nn.SiLU, #Equivalent to index 2
                ('dropout_p', 'model_action'): 0.2,
                ('preprocess', 'model_action'): False,
                ('start_neurons', 'model_action'): 32,
                ('batch_size', 'dls_action'): 128
            },
            { #This one has BOTH action parameters provided and therefore, the rest of the variables should be pooled randomly.
              #Consequently passing.
                ('model_action', None): 0,
                ('dls_action', None): 0
            }
        ]

        #Create invalid prior trials for testing
        self.invalid_prior_trials = [
            {
                'invalid_param>None': 0, #'invalid_param' NOT in init_config --> keyerror
                'freeze>None': True,
            },
            {
                ('invalid_param', None): 0, #'invalid_param' NOT in init_config --> keyerror
                ('freeze', None): True,
            },
            {
                'dls_action>None': 1, #index 1 does NOT exist in the dls_action's distribution.
                'freeze': True,  #Missing '>None', invalid format
            },
            {
                'dls_action>None': 1, #index 1 does NOT exist in the dls_action's distribution.
                'freeze': True,  #Missing '>None', invalid format
            },
            {
                ('model_action', None): model_action_dropout_auto,
                ('activation', 'model_action'): nn.ELU, #NOT in the distribution of 'activation'
            },
            {
                ('model_action', None): model_action_dropout_auto,
                ('dls_action', None): 1, #NOT in the distribution of 'dls_action'.
            }
        ]

    def tearDown(self):
        #Clean up the storage directory after tests
        if os.path.exists(self.storage_dir):
            shutil.rmtree(self.storage_dir)

    def test_prepare_prior_trials_valid(self):
        #Test with valid prior trials
        formatted_trials = self.auto._prepare_prior_trials(self.valid_prior_trials)
        #Assert that the formatted_trials have the expected format
        self.assertIsInstance(formatted_trials, list)
        self.assertEqual(len(formatted_trials), len(self.valid_prior_trials))
        formatted_config_keys = [f"{var_type}>{belongs_to}" for var_type, belongs_to in self.init_config._config.keys()]
        for trial_dict in formatted_trials:
            self.assertIsInstance(trial_dict, dict)
            for key in trial_dict.keys():
                self.assertIn(key, formatted_config_keys)

    def test_prepare_prior_trials_invalid(self):
        for idx, trial in enumerate(self.invalid_prior_trials):
            if idx<2:
                with self.assertRaises(KeyError):
                    self.auto._prepare_prior_trials([trial])
            else:
                with self.assertRaises(ValueError):
                    self.auto._prepare_prior_trials([trial])

    def test_prepare_prior_trials_empty(self):
        #Test with an empty list of prior trials
        formatted_trials = self.auto._prepare_prior_trials([])
        self.assertEqual(formatted_trials, [])

#Run the tests
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

..[I 2024-10-13 21:39:41,574] A new study created in memory with name: no-name-22f9e277-a7f4-4c38-ad96-f71c7d4a2dff


All good

Missing signature parameters in model_action: {'dropout_p', 'rand_number', 'activation'} 

Missing signature parameters in model_action: {'dropout_p', 'rand_number', 'activation'} 
Missing signature parameters in dls_action: {'normalize', 'activation', 'batch_size'} 

Missing signature parameters in model_action: {'dropout_p', 'activation'} 
Missing signature parameters in dls_action: {'normalize', 'activation', 'batch_size'} 

Missing signature parameters in model_action: {'dropout_p'} 
Missing signature parameters in dls_action: {'normalize', 'activation', 'batch_size'} 

Missing signature parameters in dls_action: {'normalize', 'activation', 'batch_size'} 

Missing signature parameters in dls_action: {'normalize', 'activation'} 

Missing signature parameters in dls_action: {'activation'} 

All good

All good

All good

Missing signature parameters in model_action: {'dropout_p', 'rand_number', 'activation'} 

Missing signature parameters in model_action: {'dropout_p', 'rand

[I 2024-10-13 21:39:41,592] A new study created in memory with name: no-name-c6c2a712-d250-431c-95de-aa65389ba80a


A new study created in memory with name: no-name-c6c2a712-d250-431c-95de-aa65389ba80a
A new study created in memory with name: no-name-c6c2a712-d250-431c-95de-aa65389ba80a
A new study created in memory with name: no-name-c6c2a712-d250-431c-95de-aa65389ba80a
A new study created in memory with name: no-name-c6c2a712-d250-431c-95de-aa65389ba80a
A new study created in memory with name: no-name-c6c2a712-d250-431c-95de-aa65389ba80a
A new study created in memory with name: no-name-c6c2a712-d250-431c-95de-aa65389ba80a
A new study created in memory with name: no-name-c6c2a712-d250-431c-95de-aa65389ba80a
A new study created in memory with name: no-name-c6c2a712-d250-431c-95de-aa65389ba80a
A new study created in memory with name: no-name-c6c2a712-d250-431c-95de-aa65389ba80a
A new study created in memory with name: no-name-c6c2a712-d250-431c-95de-aa65389ba80a
A new study created in memory with name: no-name-c6c2a712-d250-431c-95de-aa65389ba80a
A new study created in memory with name: no-name-c6c2a

.................

...

...........[I 2024-10-13 21:47:32,785] A new study created in memory with name: no-name-f9b9a1a6-adc8-44a8-980b-9e93baeb07bc


Dynamic verification passed successfully
.
A new study created in memory with name: no-name-f9b9a1a6-adc8-44a8-980b-9e93baeb07bc
A new study created in memory with name: no-name-f9b9a1a6-adc8-44a8-980b-9e93baeb07bc
A new study created in memory with name: no-name-f9b9a1a6-adc8-44a8-980b-9e93baeb07bc
A new study created in memory with name: no-name-f9b9a1a6-adc8-44a8-980b-9e93baeb07bc
A new study created in memory with name: no-name-f9b9a1a6-adc8-44a8-980b-9e93baeb07bc
A new study created in memory with name: no-name-f9b9a1a6-adc8-44a8-980b-9e93baeb07bc
A new study created in memory with name: no-name-f9b9a1a6-adc8-44a8-980b-9e93baeb07bc
A new study created in memory with name: no-name-f9b9a1a6-adc8-44a8-980b-9e93baeb07bc
A new study created in memory with name: no-name-f9b9a1a6-adc8-44a8-980b-9e93baeb07bc
A new study created in memory with name: no-name-f9b9a1a6-adc8-44a8-980b-9e93baeb07bc
A new study created in memory with name: no-name-f9b9a1a6-adc8-44a8-980b-9e93baeb07bc
A new study

..[I 2024-10-13 21:47:32,815] A new study created in memory with name: no-name-1aab9267-001d-4847-bd53-0bec7621c987


A new study created in memory with name: no-name-1aab9267-001d-4847-bd53-0bec7621c987
A new study created in memory with name: no-name-1aab9267-001d-4847-bd53-0bec7621c987
A new study created in memory with name: no-name-1aab9267-001d-4847-bd53-0bec7621c987
A new study created in memory with name: no-name-1aab9267-001d-4847-bd53-0bec7621c987
A new study created in memory with name: no-name-1aab9267-001d-4847-bd53-0bec7621c987
A new study created in memory with name: no-name-1aab9267-001d-4847-bd53-0bec7621c987
A new study created in memory with name: no-name-1aab9267-001d-4847-bd53-0bec7621c987
A new study created in memory with name: no-name-1aab9267-001d-4847-bd53-0bec7621c987
A new study created in memory with name: no-name-1aab9267-001d-4847-bd53-0bec7621c987
A new study created in memory with name: no-name-1aab9267-001d-4847-bd53-0bec7621c987
A new study created in memory with name: no-name-1aab9267-001d-4847-bd53-0bec7621c987
A new study created in memory with name: no-name-1aab9

.....
FAIL: test_inaccurate_prior_trials (__main__.TestAutoEntireRun)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-27-49468e7041fa>", line 108, in test_inaccurate_prior_trials
    with self.assertRaises(ValueError) as context:
AssertionError: ValueError not raised

----------------------------------------------------------------------
Ran 117 tests in 471.286s

FAILED (failures=1)


In [31]:
class TestAutoEntireRun(unittest.TestCase):
    def setUp(self):
        #Define the full INIT_DICT as per your code
        self.INIT_DICT = {
            ('dls_action', None): {
                'params': {'choices': [get_dls]},
                'sample': 'categorical',
                'action': True
            },
            ('freeze', None): {
                'params': {'choices': [True, False]},
                'sample': 'categorical'
            },
            ('lr_low', None): {
                'params': {'low': 1e-7, 'high': 9e-4, 'log': True},
                'sample': 'float'
            },
            ('lr_high', None): {
                'params': {'low': 9e-4, 'high': 1e-1, 'log': True},
                'sample': 'float'
            },
            ('lr', None): {
                'params': {'low': 1e-6, 'high': 1e-1, 'log': True},
                'sample': 'float'
            },
            ('wd', None): {
                'params': {'choices': [1e-4, 1e-3, 1e-2, 1e-1]},
                'sample': 'categorical'
            },
            ('gradient_clip', None): {
                'params': {'choices': [True, False]},
                'sample': 'categorical'
            },
            ('max_norm', None): {
                'params': {'low': 0.0, 'high': 15.0, 'log': False},
                'sample': 'float'
            },
            ('one_cycle', None): {
                'params': {'choices': [True, False]},
                'sample': 'categorical'
            },
            ('pct_start', None): {
                'params': {'low': 0.10, 'high': 0.95,  'log': False, 'step': 0.05},
                'sample': 'float'
            },
            ('n_epoch', None): {
                'params': {'choices': [5, 10, 15]},
                'sample': 'categorical'
            },
            ('model_action', None): {
                'params': {'choices': [model_action_default_auto, model_action_dropout_auto]},
                'sample': 'categorical',
                'action': True,
                'save_state': True
            },
            ('activation', 'model_action'): {
                'params': {'choices': [nn.ReLU, nn.PReLU, nn.SiLU]},
                'sample': 'categorical'
            },
            ('loss_func', None): {
                'params': {'choices': [custom_mse]},
                'sample': 'categorical'
            },
            ('metric', None): {
                'params': {'choices': [custom_mse]},
                'sample': 'categorical'
            },
            ('preprocess', 'model_action'): {
                'params': {'choices': [True, False]},
                'sample': 'categorical'
            },
            ('start_neurons', 'model_action'): {
                'params': {'choices': [1, 2, 4, 8, 16, 32]},
                'sample': 'categorical'
            },
            ('dropout_p', 'model_action'): {
                'params': {'low': 0.05, 'high': 0.95,  'log': False, 'step': 0.05},
                'sample': 'float'
            },
            ('batch_size', 'dls_action'): {
                'params': {'choices': [32, 64, 128]},
                'sample': 'categorical'
            }
        }

        self.init_config = ConfigSpace(self.INIT_DICT)
        self.user_objective = UserObjective()
        self.storage_dir = './test_trial_storage/'

        self.auto = Auto(self.init_config, self.storage_dir)

        #Initializers for self.auto.optimize()
        self.n_auto = 4
        self.top_k = 4
        self.update_state_window = 2
        self.device = torch.device('cpu')

    def tearDown(self):
        if os.path.exists(self.storage_dir):
            shutil.rmtree(self.storage_dir)

    def test_entire_run(self):
        #Run the optimization
        prior_trials = [
            {('dls_action', None): get_dls},
            #NOTE: Even if the user passes in an inaccurate signature parameter for the/any chosen Action, the program should be able to 'hop over' it.
            {('dls_action', None): get_dls, ('model_action', None): model_action_default_auto, ('dropout_p', 'model_action'): 0.05},
            {('dls_action', None): get_dls, ('model_action', None): model_action_dropout_auto, ('dropout_p', 'model_action'): 0.15}
        ]

        self.auto.optimize(
            user_objective=self.user_objective,
            n_auto=self.n_auto,
            top_k=self.top_k,
            update_state_window=self.update_state_window,
            device=self.device,
            prior_trials=prior_trials,
            prior_dynamic_verification=True
        )

        #Check that the number of studies is as expected
        expected_n_study = len(self.auto._get_list_n_trials(self.n_auto))
        self.assertEqual(self.auto.num_study(), expected_n_study)

        #Check that studies have been saved and can be loaded
        for idx in range(self.auto.num_study()):
            study = self.auto.get_study(idx)
            self.assertIsInstance(study, optuna.study.Study)
            #Check that the study has trials
            self.assertTrue(len(study.trials) > 0)

        #Check that the combined_sampler has been updated
        self.assertIsNotNone(self.auto.combined_sampler)

#Run the tests
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

..[I 2024-10-13 21:47:32,886] A new study created in memory with name: no-name-29d3aa39-adc6-4b1c-ba11-3fafda0a7352


All good

Missing signature parameters in model_action: {'dropout_p', 'rand_number', 'activation'} 

Missing signature parameters in model_action: {'dropout_p', 'rand_number', 'activation'} 
Missing signature parameters in dls_action: {'normalize', 'activation', 'batch_size'} 

Missing signature parameters in model_action: {'dropout_p', 'activation'} 
Missing signature parameters in dls_action: {'normalize', 'activation', 'batch_size'} 

Missing signature parameters in model_action: {'dropout_p'} 
Missing signature parameters in dls_action: {'normalize', 'activation', 'batch_size'} 

Missing signature parameters in dls_action: {'normalize', 'activation', 'batch_size'} 

Missing signature parameters in dls_action: {'normalize', 'activation'} 

Missing signature parameters in dls_action: {'activation'} 

All good

All good

All good

Missing signature parameters in model_action: {'dropout_p', 'rand_number', 'activation'} 

Missing signature parameters in model_action: {'dropout_p', 'rand

[I 2024-10-13 21:47:32,916] A new study created in memory with name: no-name-25eeee2b-cedd-4914-b228-bdec5f81a663


A new study created in memory with name: no-name-25eeee2b-cedd-4914-b228-bdec5f81a663
A new study created in memory with name: no-name-25eeee2b-cedd-4914-b228-bdec5f81a663
A new study created in memory with name: no-name-25eeee2b-cedd-4914-b228-bdec5f81a663
A new study created in memory with name: no-name-25eeee2b-cedd-4914-b228-bdec5f81a663
A new study created in memory with name: no-name-25eeee2b-cedd-4914-b228-bdec5f81a663
A new study created in memory with name: no-name-25eeee2b-cedd-4914-b228-bdec5f81a663
A new study created in memory with name: no-name-25eeee2b-cedd-4914-b228-bdec5f81a663
A new study created in memory with name: no-name-25eeee2b-cedd-4914-b228-bdec5f81a663
A new study created in memory with name: no-name-25eeee2b-cedd-4914-b228-bdec5f81a663
A new study created in memory with name: no-name-25eeee2b-cedd-4914-b228-bdec5f81a663
A new study created in memory with name: no-name-25eeee2b-cedd-4914-b228-bdec5f81a663
A new study created in memory with name: no-name-25eee

.......

...

...........[I 2024-10-13 21:53:44,429] A new study created in memory with name: no-name-380f6cc8-e48e-4b0e-8794-49677b1f8cb2


Dynamic verification passed successfully
.
A new study created in memory with name: no-name-380f6cc8-e48e-4b0e-8794-49677b1f8cb2
A new study created in memory with name: no-name-380f6cc8-e48e-4b0e-8794-49677b1f8cb2
A new study created in memory with name: no-name-380f6cc8-e48e-4b0e-8794-49677b1f8cb2
A new study created in memory with name: no-name-380f6cc8-e48e-4b0e-8794-49677b1f8cb2
A new study created in memory with name: no-name-380f6cc8-e48e-4b0e-8794-49677b1f8cb2
A new study created in memory with name: no-name-380f6cc8-e48e-4b0e-8794-49677b1f8cb2
A new study created in memory with name: no-name-380f6cc8-e48e-4b0e-8794-49677b1f8cb2
A new study created in memory with name: no-name-380f6cc8-e48e-4b0e-8794-49677b1f8cb2
A new study created in memory with name: no-name-380f6cc8-e48e-4b0e-8794-49677b1f8cb2
A new study created in memory with name: no-name-380f6cc8-e48e-4b0e-8794-49677b1f8cb2
A new study created in memory with name: no-name-380f6cc8-e48e-4b0e-8794-49677b1f8cb2
A new study

..[I 2024-10-13 21:53:44,473] A new study created in memory with name: no-name-7f0b28e4-3634-4072-97cf-e2917572d606


A new study created in memory with name: no-name-7f0b28e4-3634-4072-97cf-e2917572d606
A new study created in memory with name: no-name-7f0b28e4-3634-4072-97cf-e2917572d606
A new study created in memory with name: no-name-7f0b28e4-3634-4072-97cf-e2917572d606
A new study created in memory with name: no-name-7f0b28e4-3634-4072-97cf-e2917572d606
A new study created in memory with name: no-name-7f0b28e4-3634-4072-97cf-e2917572d606
A new study created in memory with name: no-name-7f0b28e4-3634-4072-97cf-e2917572d606
A new study created in memory with name: no-name-7f0b28e4-3634-4072-97cf-e2917572d606
A new study created in memory with name: no-name-7f0b28e4-3634-4072-97cf-e2917572d606
A new study created in memory with name: no-name-7f0b28e4-3634-4072-97cf-e2917572d606
A new study created in memory with name: no-name-7f0b28e4-3634-4072-97cf-e2917572d606
A new study created in memory with name: no-name-7f0b28e4-3634-4072-97cf-e2917572d606
A new study created in memory with name: no-name-7f0b2

.....
FAIL: test_inaccurate_prior_trials (__main__.TestAutoEntireRun)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-31-49468e7041fa>", line 108, in test_inaccurate_prior_trials
    with self.assertRaises(ValueError) as context:
AssertionError: ValueError not raised

----------------------------------------------------------------------
Ran 117 tests in 371.645s

FAILED (failures=1)
